In [ ]:
from datetime import datetime # noqa: F401
from functools import reduce
from pathlib import Path # noqa: F401
import funcoes_br as f_br
import polars as pl

In [ ]:
# Carregando os arquivos compilados consolidados - anual
dre = f_br.df_dre(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_DRE_con_2017_2024.parquet'))
bpa = f_br.df_bp(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_BPA_con_2017_2024.parquet'))
bpp = f_br.df_bp(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_BPP_con_2017_2024.parquet'))
dfc = f_br.df_dfc(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_DFC_MI_con_2017_2024.parquet'))
dva = f_br.df_dva(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_DVA_con_2017_2024.parquet'))

# Carregando os arquivos compilados individuais - anual
dre_ind = f_br.df_dre(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_DRE_ind_2017_2024.parquet'))
bpa_ind = f_br.df_bp(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_BPA_ind_2017_2024.parquet'))
bpp_ind = f_br.df_bp(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_BPP_ind_2017_2024.parquet'))
dfc_ind = f_br.df_dfc(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_DFC_MI_ind_2017_2024.parquet'))
dva_ind = f_br.df_dva(Path('C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_DVA_ind_2017_2024.parquet'))

# Composição de capital social histórico - anual
composicao_capital = f_br.df_composicao_capital(Path(
    'C://B3//historico-arquivos//dfp-b3//dfp-cia-aberta//consolidados//dfp_cia_aberta_composicao_capital_2020_2024.parquet'
))

# Carregando os arquivos compilados consolidados - trimestral
""
dre_trimestral = f_br.df_dre(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_DRE_con_2020_2025.parquet'))
bpa_trimestral = f_br.df_bp(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_BPA_con_2020_2025.parquet'))
bpp_trimestral = f_br.df_bp(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_BPP_con_2020_2025.parquet'))
dfc_trimestral = f_br.df_dfc(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_DFC_MI_con_2020_2025.parquet'))
dva_trimestral = f_br.df_dva(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_DVA_con_2020_2025.parquet'))

# Carregando os arquivos compilados individuais - trimestral
dre_ind_trimestral = f_br.df_dre(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_DRE_ind_2020_2025.parquet'))
bpa_ind_trimestral = f_br.df_bp(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_BPA_ind_2020_2025.parquet'))
bpp_ind_trimestral = f_br.df_bp(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_BPP_ind_2020_2025.parquet'))
dfc_ind_trimestral = f_br.df_dfc(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_DFC_MI_ind_2020_2025.parquet'))
dva_ind_trimestral = f_br.df_dva(Path('C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_DVA_ind_2020_2025.parquet'))

# Composição de capital social histórico - trimestral
composicao_capital_trimestral = f_br.df_composicao_capital(Path(
    'C://B3//historico-arquivos//itr-b3//itr-cia-aberta//consolidados//itr_cia_aberta_composicao_capital_2020_2025.parquet'
))

# Composição de capital social mais recente - contém a coluna ticker ('PETR4')
df_capital_social = pl.read_csv(
    Path('C://B3//historico-arquivos//capital-social-b3//capital_social_3T2025.csv'),
    separator=';',
    encoding='utf8'
)

# Formatando as colunas
df_capital_social = (
    df_capital_social
    .with_columns([
        pl.col('Capital R$')
        .str.replace_all('.', '', literal=True) # Remove todos os pontos LITERAIS (separador de milhar)
        .str.replace_all(',', '.')
        .cast(pl.Float64),

        pl.col('Aprovado Em')
        .str.strptime(pl.Date, format='%d/%m/%Y'),

        pl.col('Qtde Ações Ordinárias')
        .str.replace_all('.', '', literal=True)
        .cast(pl.Int64),

        pl.col('Qtde Ações Preferenciais')
        .str.replace_all('.', '', literal=True)
        .cast(pl.Int64),

        pl.col('Qtde Total De Ações')
        .str.replace_all('.', '', literal=True)
        .cast(pl.Int64),
    ])
    .select([
        'Código',
        'Qtde Ações Ordinárias',
        'Qtde Ações Preferenciais',
        'Qtde Total De Ações'
    ])
)

# Renomeando as colunas
df_capital_social = df_capital_social.rename({
    'Código': 'ticker',
    'Qtde Ações Ordinárias': 'num_on',
    'Qtde Ações Preferenciais': 'num_pn',
    'Qtde Total De Ações': 'num_total',
})

# Existem alguns tickers que estão duplicados - removendo essa duplicidade p/ calcular o VM atual
df_capital_social = df_capital_social.unique(subset=['ticker'], keep='first')

# Para visualizar quais são os tickers duplicados
# linhas_duplicadas = df_capital_social.filter(
#     pl.col('ticker').is_duplicated()
# )

In [ ]:
# Padrão de substituição - esta expressão regular substitui as principais letras acentuadas e cedilha pela sua contraparte não-acentuada.
LIMPEZA_MAP = (
    pl.col('denom_cia')
    .str.replace_all('[áàãâä]', 'a', literal=False)
    .str.replace_all('[éèêë]', 'e', literal=False)
    .str.replace_all('[íìîï]', 'i', literal=False)
    .str.replace_all('[óòõôö]', 'o', literal=False)
    .str.replace_all('[úùûü]', 'u', literal=False)
    .str.replace_all('[ç]', 'c', literal=False)
    .str.to_lowercase()  # Converte tudo para minúsculas
    .alias('_chave_ordenacao') # Dá um nome para a coluna temporária
)

# Empresas em recuperação judicial
df_empresas_recuperacao_judicial = (
    composicao_capital_trimestral
    .filter(pl.col('denom_cia').str.contains('EM RECUPERAÇÃO JUDICIAL'))
    .select(
        'denom_cia',
        LIMPEZA_MAP  # Adiciona a chave de ordenação
    )
    .unique()
    # Ordenando de forma alfabética usando a coluna temporária '_chave_ordenacao'
    .sort('_chave_ordenacao', descending=False)
    # Remove a coluna temporária para a coluna original
    .select('denom_cia')
)

df_empresas_recuperacao_judicial

# **Setor de Aluguel**

## RENT3

In [ ]:
# Número de ações (2017 a 2019) - RENT
dict_composicao_capital_17_18_19_rent = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        667_149_000,
        667_149_000,
        758_467_000,
    ]
}

## MILS3

In [ ]:
# Número de ações (2017 a 2019) - MILS
dict_composicao_capital_17_18_19_mils = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        175_586_000,
        175_586_000,
        251_866_000,
    ]
}

# **Setor de Bebidas**

## ABEV3

In [ ]:
# Número de ações (2017 a 2019) - ABEV
dict_composicao_capital_17_18_19_abev = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        15_717_615_000,
        15_722_147_000,
        15_733_575_000,
    ]
}

# **Setor de Celulose**

## KLBN3

In [ ]:
# Número de ações (2017 a 2019) - KLBN
dict_composicao_capital_17_18_19_klbn = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        1_859_852_065,
        1_984_594_655,
        1_984_594_655,
    ],

    'num_pn':[
        2_926_236_825,
        3_425_207_185,
        3_425_207_185,
    ]
}

## SUZB3

In [ ]:
# Número de ações (2017 a 2019) - SUZB
dict_composicao_capital_17_18_19_suzb = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        1_105_826_000,
        1_105_826_000,
        1_361_263_584,
    ]
}

## RANI3

In [ ]:
# Número de ações (2017 a 2019) - RANI
dict_composicao_capital_17_18_19_rani = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        166_720_000,
        166_720_000,
        166_720_000,
    ]
}

# **Setor de Energia Elétrica - Distribuição**

## EQTL3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_eqtl';
    - capex no dicionário 'dict_capex_eqtl';
    - R&D no dicionário 'dict_rd_eqtl'.

In [ ]:
# Número de ações (2017 a 2019) - EQTL
dict_composicao_capital_17_18_19_eqtl = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        198_743_467,
        198_769_717,
        1_010_186_085,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_eqtl = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_eqtl)

# Composição de capital (2020 a 2024) - EQTL
composicao_capital_eqtl = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('EQUATORIAL')).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_eqtl = pl.concat([df_composicao_capital_17_18_19_eqtl, composicao_capital_eqtl])

##############################################################################
# Preços de fechamento - EQTL
df_eqtl_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='EQTL3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_eqtl.tail())
print('=' * 80)
print(df_eqtl_fechamento_anual.tail())

In [ ]:
# EQTL3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='EQTL3.SA', ticker_pn=None)

# Valor de mercado
vm_eqtl = f_br.indicador_vm(df_capital_social=df_composicao_capital_eqtl, df_fechamento=df_eqtl_fechamento_anual, novo_mercado=True)

# LPA
lpa_eqtl = f_br.indicador_lpa(df_capital_social=df_composicao_capital_eqtl, df_dre=dre, cod_cvm=20010, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_eqtl = f_br.indicador_pl_lp(df_fechamento=df_eqtl_fechamento_anual, tipo_acao='on', df_lpa=lpa_eqtl)[0]

# L/P
lp_eqtl = f_br.indicador_pl_lp(df_fechamento=df_eqtl_fechamento_anual, tipo_acao='on', df_lpa=lpa_eqtl)[1]

# EBITDA
ebitda_eqtl = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=20010, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_eqtl = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=20010, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_eqtl = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=20010, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_eqtl = f_br.indicador_divida_liquida(df_caixa_total=caixa_eqtl, df_divida_bruta=divida_bruta_eqtl)

# DL/EBITDA
dl_ebitda_eqtl = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_eqtl, df_ebitda=ebitda_eqtl)

# DL/PL
dl_pl_eqtl = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=20010, cod_pl='2.03', df_divida_liquida=divida_liquida_eqtl)

# EV/EBITDA
ev_ebitda_eqtl = f_br.indicador_ev_ebitda(df_vm=vm_eqtl, df_divida_liquida=divida_liquida_eqtl, df_ebitda=ebitda_eqtl)

# VPA
vpa_eqtl = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_eqtl, cod_cvm=20010, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_eqtl = f_br.indicador_p_vpa(df_fechamento=df_eqtl_fechamento_anual, tipo_acao='on', df_vpa=vpa_eqtl)

# Margem líquida
ml_eqtl = f_br.indicador_ml(df_dre=dre, cod_cvm=20010, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_eqtl = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=20010, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_eqtl = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=20010, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_eqtl)

# Dividendos
proventos_eqtl = f_br.indicador_proventos(df_dva=dva, cod_cvm=20010, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_eqtl = f_br.indicador_payout(df_dre=dre, cod_cvm=20010, df_proventos=proventos_eqtl, cod_ll='3.11')

# Buyback
dict_buyback_eqtl = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback': [
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        57_538,
    ]
}

buyback_eqtl = f_br.dict_to_polars_df(dict_buyback_eqtl)

# FCO
fco_eqtl = f_br.indicador_fco(df_dfc=dfc, cod_cvm=20010, cod_fco='6.01')

# FCI
fci_eqtl = f_br.indicador_fci(df_dfc=dfc, cod_cvm=20010, cod_fci='6.02')

# FCF
fcf_eqtl = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=20010, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisições de Imobilizado e Intangível'
]

capex_eqtl_1 = f_br.indicador_capex(df_dfc=dfc, cod_cvm=20010, lst_cod_capex=lst_cod_capex)
capex_eqtl_1 = capex_eqtl_1.rename({'capex':'vl_conta_1'})

# Itens do capex em que o código varia durante os anos
dict_capex_eqtl = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    # Soma dos três itens ('Aquisições no ativo contratual - Distribuidoras', 'Aquisições no ativo contratual - Transmissoras' e 'Aquisição de participação em coligada')
    'vl_conta_2': [
        0 + 0 + 0,
        848_009 + 945_803 + 0,
        1_553_189 + 0 + 0,
        1_657_699 + 0 + 0,
        2_477_267 + 0 + 0,
        4_847_790 + 0 + 0,
        0 + 0 + 0,
        0 + 0 + 6_869_274,
    ]
}

capex_eqtl_2 = f_br.dict_to_polars_df(dict_capex_eqtl)
capex_eqtl_2 = capex_eqtl_2.rename({'capex':'vl_conta_2'})

capex_eqtl = capex_eqtl_1.join(capex_eqtl_2, on='dt_refer', how='inner')

capex_eqtl = capex_eqtl.with_columns(
    (pl.col('vl_conta_1') + pl.col('vl_conta_2')).alias('capex')
).select('dt_refer', 'capex')

# Free cash flow
free_cash_flow_eqtl = f_br.indicador_free_cash_flow(df_fco=fco_eqtl, df_capex=capex_eqtl)

# Net capex
net_capex_eqtl = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=20010, cod_depreciacao='6.01.01.02', df_capex=capex_eqtl)

# R&D
# A partir de 2022, essa informação está no arquivo "Release de resultados", no item "Investimento em P&D e Eficiência Energética", na tabela de "Indicadores de ESG".
# De 2021 para trás, essa informação está no arquivo DFP, na nota explicativa "Pesquisa e desenvolvimento e eficiência energética".
dict_rd_eqtl = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        99_448,
        136_218,
        171_583,
        196_918,
        232_575,
        51_465,
        83_581,
        1_506+830+18_473+28_175 # Em 2024, soma dos 4 trimestres - não veio anualizada
    ]
}

rd_eqtl = f_br.dict_to_polars_df(dict_rd_eqtl)

# Adjusted net capex
adjusted_net_capex_eqtl = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_eqtl, df_rd=rd_eqtl)

# Working capital
working_capital_eqtl = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=20010,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_eqtl = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=20010, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_eqtl = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_eqtl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_eqtl
)

# Effective tax rate
effective_tax_rate_eqtl = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=20010, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_eqtl = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=20010, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_eqtl, anual=True)

* Itens que compõem o 'new borrowing':
    - 'new_borrowing_item_1' - 'Captação de Empréstimos e Financiamentos' - '6.03.01';
    - 'new_borrowing_item_2' - 'Captação de debêntures' - '6.03.04'.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_new_borrowing_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# New borrowing

# DFC da EQTL
dfc_eqtl = dfc.filter(pl.col('cd_cvm') == 20010)

################################################################
# 'Captação de Empréstimos e Financiamentos'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2017, 12, 31),
    datetime(2018, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_1 = (
    dfc_eqtl
    .filter(
        (pl.col('cd_conta') == '6.03.01') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_new_borrowing_item_1 = {
    'dt_refer': [
        '2019-12-31'
    ],

    'vl_conta_1': [
        5_178_251
    ]
}

new_borrowing_item_1_complementos = f_br.dict_to_polars_df(dict_new_borrowing_item_1)
new_borrowing_item_1 = pl.concat([new_borrowing_item_1_complementos, new_borrowing_item_1])
new_borrowing_item_1 = new_borrowing_item_1.sort('dt_refer')

################################################################
# 'Captação de debêntures'
DATAS_ALVO_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_2 = (
    dfc_eqtl
    .filter(
        (pl.col('cd_conta') == '6.03.04') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_new_borrowing_item_2 = {
    'dt_refer': [
        '2017-12-31'
    ],

    'vl_conta_2': [
        1_323_000
    ]
}

new_borrowing_item_2_complementos = f_br.dict_to_polars_df(dict_new_borrowing_item_2)
new_borrowing_item_2 = pl.concat([new_borrowing_item_2_complementos, new_borrowing_item_2])
new_borrowing_item_2 = new_borrowing_item_2.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_new_borrowing = [new_borrowing_item_1, new_borrowing_item_2]
new_borrowing_eqtl = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_new_borrowing)

# Somando as colunas
new_borrowing_eqtl = new_borrowing_eqtl.with_columns(
    ((pl.col('vl_conta_1') + pl.col('vl_conta_2'))).alias('new_borrowing')
).select('dt_refer', 'new_borrowing')

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Amortização de Empréstimos e Financiamentos' - '6.03.02';
    - 'debt_paid_item_2' - 'Amortização de debêntures' - '6.03.03';
    - 'debt_paid_item_3' - 'Valores pagos referente a acordos com plano de recuperação judicial' - '6.03.08';
    - 'debt_paid_item_4' - 'Amortização do passivo de arrendamento' - '6.03.11'.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid

# DFC da EQTL
dfc_eqtl = dfc.filter(pl.col('cd_cvm') == 20010)

################################################################
# 'Amortização de Empréstimos e Financiamentos'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2017, 12, 31),
    datetime(2018, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_eqtl
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2019-12-31'
    ],

    'vl_conta_1': [
        -3_245_314
    ]
}

debt_paid_item_1_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_1)
debt_paid_item_1 = pl.concat([debt_paid_item_1_complementos, debt_paid_item_1])
debt_paid_item_1 = debt_paid_item_1.sort('dt_refer')

################################################################
# 'Amortização de debêntures'
DATAS_ALVO_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_eqtl
    .filter(
        (pl.col('cd_conta') == '6.03.03') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31'
    ],

    'vl_conta_2': [
        -100_450
    ]
}

debt_paid_item_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_paid_item_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

################################################################
# 'Valores pagos referente a acordos com plano de recuperação judicial'
DATAS_ALVO_3 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2018, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_3 = (
    dfc_eqtl
    .filter(
        (pl.col('cd_conta') == '6.03.08') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_3))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_3')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2017-12-31',
        '2019-12-31',
    ],

    'vl_conta_3': [
        -166_857,
        -31_935,
    ]
}

debt_paid_item_3_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_3)
debt_paid_item_3 = pl.concat([debt_paid_item_3_complementos, debt_paid_item_3])
debt_paid_item_3 = debt_paid_item_3.sort('dt_refer')

################################################################
# 'Amortização do passivo de arrendamento'
DATAS_ALVO_4 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_4 = (
    dfc_eqtl
    .filter(
        (pl.col('cd_conta') == '6.03.11') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_4))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_4')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'vl_conta_4': [
        0,
        0,
        0,
    ]
}

debt_paid_item_4_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_4)
debt_paid_item_4 = pl.concat([debt_paid_item_4_complementos, debt_paid_item_4])
debt_paid_item_4 = debt_paid_item_4.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_item_1, debt_paid_item_2, debt_paid_item_3, debt_paid_item_4]
debt_paid_eqtl = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_eqtl = debt_paid_eqtl.with_columns(
    ((pl.col('vl_conta_1') + pl.col('vl_conta_2') + pl.col('vl_conta_3') + pl.col('vl_conta_4')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_eqtl = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=20010,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_eqtl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_eqtl,
    df_new_borrowing=new_borrowing_eqtl,
    df_debt_paid=debt_paid_eqtl,
)

# FCFF
fcff_eqtl = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_eqtl,
    df_capex=capex_eqtl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_eqtl,
)

In [ ]:
# Lista dos df
lst_df_eqtl = [
    vm_eqtl,
    lpa_eqtl,
    pl_eqtl,
    lp_eqtl,
    ebitda_eqtl,
    divida_bruta_eqtl,
    caixa_eqtl,
    divida_liquida_eqtl,
    dl_ebitda_eqtl,
    dl_pl_eqtl,
    ev_ebitda_eqtl,
    vpa_eqtl,
    p_vp_eqtl,
    ml_eqtl,
    roe_eqtl,
    roic_eqtl,
    proventos_eqtl,
    payout_eqtl,
    buyback_eqtl,
    fco_eqtl,
    fci_eqtl,
    fcf_eqtl,
    capex_eqtl,
    free_cash_flow_eqtl,
    net_capex_eqtl,
    rd_eqtl,
    adjusted_net_capex_eqtl,
    working_capital_eqtl,
    change_in_non_cash_wc_eqtl,
    reinvestment_rate_eqtl,
    effective_tax_rate_eqtl,
    fcfe_eqtl,
    fcff_eqtl
]

# Juntando os dfs
df_indicadores_eqtl = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_eqtl
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_eqtl = df_indicadores_eqtl.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_eqtl

In [ ]:
# Transformando em um arquivo csv
df_indicadores_eqtl.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//eqtl//eqtl_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_trimestral_eqtl';
    - capex no dicionário 'dict_capex_trimestral_eqtl';
    - R&D no dicionário 'dict_rd_trimestral_eqtl'.

In [ ]:
# Composição de capital - EQTL
df_composicao_capital_trimestral_eqtl = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('EQUATORIAL')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - EQTL
df_eqtl_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='EQTL3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_eqtl['dt_refer']
)

print(df_composicao_capital_trimestral_eqtl.tail())
print('=' * 80)
print(df_eqtl_fechamento_trimestral.tail())

In [ ]:
# EQTL3

# Valor de mercado
vm_trimestral_eqtl = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_eqtl[3:],
    df_fechamento=df_eqtl_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_eqtl = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_eqtl,
    df_dre=dre_trimestral,
    cod_cvm=20010,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_eqtl = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_eqtl,
    df_dre=dre_trimestral,
    cod_cvm=20010,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_eqtl = f_br.indicador_pl_lp(
    df_fechamento=df_eqtl_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_eqtl
)[0]

# L/P
lp_trimestral_eqtl = f_br.indicador_pl_lp(
    df_fechamento=df_eqtl_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_eqtl
)[1]

# EBITDA
ebitda_trimestral_eqtl = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_eqtl = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_eqtl = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=20010,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_eqtl = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=20010,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_eqtl = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_eqtl,
    df_divida_bruta=divida_bruta_trimestral_eqtl
)

# DL/EBITDA
dl_ebitda_trimestral_eqtl = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_eqtl,
    df_ebitda=ebitda_trimestral_acum_eqtl
)

# DL/PL
dl_pl_trimestral_eqtl = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=20010,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_eqtl
)

# EV/EBITDA
ev_ebitda_trimestral_eqtl = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_eqtl,
    df_divida_liquida=divida_liquida_trimestral_eqtl,
    df_ebitda=ebitda_trimestral_acum_eqtl
)

# VPA
vpa_trimestral_eqtl = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_eqtl[3:],
    cod_cvm=20010,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_eqtl = f_br.indicador_p_vpa(
    df_fechamento=df_eqtl_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_eqtl
)

# Margem líquida
ml_trimestral_eqtl = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=20010,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_eqtl = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=20010,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_eqtl = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_eqtl
)

# Proventos
proventos_trimestral_eqtl = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=20010,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_eqtl = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=20010,
    df_proventos=proventos_trimestral_eqtl,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_eqtl = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'buyback_nao_acum': [
        # 2021
        600_271, 0, (610_991 - 600_271),
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 29_756, 0,
        # 2025
        0, 11_571, 0,
    ]
}

# Dados complementares
buyback_trimestral_eqtl = f_br.dict_to_polars_df(dict_buyback_trimestral_eqtl)

# FCO
fco_trimestral_eqtl = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_eqtl = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_eqtl = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_eqtl = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_eqtl = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_eqtl = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisições de Imobilizado e Intangível'
]

capex_trimestral_eqtl_1 = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_eqtl_1 = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    lst_cod_capex=lst_cod_capex
)[1][3:]

capex_trimestral_acum_eqtl_1 = capex_trimestral_acum_eqtl_1.rename({'capex_acum':'capex_1_acum'})

# CAPEX - o código do capex ('Aquisições no ativo contratual - Distribuidoras', 'Aquisições no ativo contratual - Transmissoras', 'Aquisição de participação em coligada') não são fixos
dict_capex_trimestral_eqtl = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    # 'Aquisições no ativo contratual - Distribuidoras'
    'capex_2': [
        # 2020
        0, 0, 0,
        # 2021
        395_751, 816_138, 1_584_222,
        # 2022
        658_537, 1_735_421, 3_128_043,
        # 2023
        2_016_059, 3_750_349, 5_479_094,
        # 2024
        0, 1_638_115, 4_111_792,
        # 2025
        0, 4_266_423, 6_572_217,
    ],

    # 'Aquisições no ativo contratual - Transmissoras'
    'capex_3': [
        # 2020
        0, 0, 0,
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ],

    # 'Aquisição de participação em coligada' ou 'Aquisição de participação em investida'
    'capex_4': [
        # 2020
        0, 0, 0,
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 6_869_274,
        # 2025
        0, 0, 0,
    ]
}

capex_trimestral_eqtl = f_br.dict_to_polars_df(dict_capex_trimestral_eqtl)

# Calculando o capex não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_capex_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se capex_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('capex_2') == 0)
        .then(0)
        .otherwise(pl.col('capex_2') - pl.col('capex_2').shift(1))
    )
    .otherwise(pl.col('capex_2')) # Mantém o valor original para outros meses
    .alias('capex_2_nao_acum')
)

expr_capex_3 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se capex_3 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('capex_3') == 0)
        .then(0)
        .otherwise(pl.col('capex_3') - pl.col('capex_3').shift(1))
    )
    .otherwise(pl.col('capex_3')) # Mantém o valor original para outros meses
    .alias('capex_3_nao_acum')
)

expr_capex_4 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se capex_4 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('capex_4') == 0)
        .then(0)
        .otherwise(pl.col('capex_4') - pl.col('capex_4').shift(1))
    )
    .otherwise(pl.col('capex_4')) # Mantém o valor original para outros meses
    .alias('capex_4_nao_acum')
)

capex_2_trimestral_nao_acum = capex_trimestral_eqtl.with_columns([expr_capex_2]).select('dt_refer', 'capex_2_nao_acum').fill_null(0)
capex_3_trimestral_nao_acum = capex_trimestral_eqtl.with_columns([expr_capex_3]).select('dt_refer', 'capex_3_nao_acum').fill_null(0)
capex_4_trimestral_nao_acum = capex_trimestral_eqtl.with_columns([expr_capex_4]).select('dt_refer', 'capex_4_nao_acum').fill_null(0)

# Calculando o capex trimestral acumulado
expr_capex_2_acum = (
    pl.col('capex_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('capex_2_acum')
)

expr_capex_3_acum = (
    pl.col('capex_3_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('capex_3_acum')
)

expr_capex_4_acum = (
    pl.col('capex_4_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('capex_4_acum')
)

capex_2_trimestral_acum = capex_2_trimestral_nao_acum.with_columns([expr_capex_2_acum]).select('dt_refer', 'capex_2_acum').fill_null(0)
capex_3_trimestral_acum = capex_3_trimestral_nao_acum.with_columns([expr_capex_3_acum]).select('dt_refer', 'capex_3_acum').fill_null(0)
capex_4_trimestral_acum = capex_4_trimestral_nao_acum.with_columns([expr_capex_4_acum]).select('dt_refer', 'capex_4_acum').fill_null(0)

# Juntando os dfs
capex_trimestral_acum_eqtl = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), [capex_trimestral_acum_eqtl_1, capex_2_trimestral_acum, capex_3_trimestral_acum, capex_4_trimestral_acum]
)

capex_trimestral_acum_eqtl = capex_trimestral_acum_eqtl.with_columns(
    (pl.col('capex_1_acum') + pl.col('capex_2_acum') + pl.col('capex_3_acum') + pl.col('capex_4_acum')).alias('capex')
).select('dt_refer', 'capex')[3:]


# Free cash flow
free_cash_flow_trimestral_eqtl = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_eqtl,
    df_capex=capex_trimestral_acum_eqtl
)

# Net capex
net_capex_trimestral_eqtl = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_eqtl
)

# R&D
# De 2021 para trás, essa informação está no arquivo DFP, na nota explicativa "Pesquisa e desenvolvimento e eficiência energética".
# A partir de 2022, essa informação está no arquivo "Release de resultados" é o item "Investimento em P&D e Eficiência Energética" na tabela de "Indicadores de ESG".

dict_rd_trimestral_eqtl = {
    'dt_refer':[
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2020
        0, 0, 0,
        # 2021
        0, 0, 0,
        # 2022
        13_370, 14_402, 38_201,
        # 2023
        14_216, 18_278, 32_365,
        # 2024
        1_506, 830, 18_473,
        # 2025
        29_359, 28_642, 31_318,
    ]
}

rd_trimestral_eqtl = f_br.dict_to_polars_df(dict_rd_trimestral_eqtl)

# Calculando o R&D acumulado
expr_rd_acum = (
    pl.col('rd')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
)

rd_trimestral_acum_eqtl = rd_trimestral_eqtl.with_columns([expr_rd_acum]).select('dt_refer', 'rd').fill_null(0)[3:]

# Adjusted net capex
adjusted_net_capex_trimestral_eqtl = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_eqtl,
    df_rd=rd_trimestral_acum_eqtl
)

# Working capital
working_capital_trimestral_eqtl = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=20010,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_eqtl = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_eqtl = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_eqtl = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_eqtl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_eqtl
)

# Effective tax rate
effective_tax_rate_trimestral_eqtl = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=20010,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_eqtl = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=20010,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_eqtl,
    anual=False
)

* Itens que compõem o 'new borrowing':
    - 'new_borrowing_item_1' - 'Captação de empréstimos e financiamentos' ou 'Ingresso de instrumentos de dívida' - código varia durante os anos;
    - 'new_borrowing_item_2' - 'Captação de debêntures' - código varia durante os anos.

* No itens 'new_borrowing_item_1' e 'new_borrowing_item_2', adicionar manualmente os dados.

In [ ]:
# New borrowing

# DFC da EQTL
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 20010)

################################################################
# 'Captação de empréstimos e financiamentos'
# '6.03.01' (03/20, 06/20, 09/20, 03/21, 09/21, 03/22, 09/22, 03/23, 09/23, 03/24, 09/24, 03/25, 09/25)
# '6.03.02' (06/21, 06/22, 06/23, 06/24, 06/25)
dict_new_borrowing_item_1 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_1': [
        # 2020
        696_111, 1_085_149, 1_639_823,
        # 2021
        730_175, 1_108_887, 2_985_428,
        # 2022
        865_161, 1_429_161, 1_454_000,
        # 2023
        534_992, 1_593_058, 3_927_874,
        # 2024
        428_859, 1_311_042, 7_602_522,
        # 2025
        985_162, 2_660_619, 5_480_526,
    ]
}

new_borrowing_item_1 = f_br.dict_to_polars_df(dict_new_borrowing_item_1)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_new_borrowing_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

new_borrowing_nao_acum_item_1 = new_borrowing_item_1.with_columns([expr_new_borrowing_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_new_borrowing_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

new_borrowing_acum_item_1 = new_borrowing_nao_acum_item_1.with_columns([expr_new_borrowing_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
new_borrowing_acum_item_1 = new_borrowing_acum_item_1[3:]

################################################################
# 'Captação de debêntures'
# '6.03.04' (03/20, 06/20, 09/20, 03/21, 09/21, 03/22, 09/22, 03/23, 09/23, 03/24, 09/24, 03/25, 09/25)
# '6.03.03' (06/21, 06/22, 06/23, 06/24, 06/25)
dict_new_borrowing_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        # 2020
        0, 0, 0,
        # 2021
        0, 800_000, 2_300_000,
        # 2022
        1_987_090, 3_631_674, 3_625_571,
        # 2023
        8_466_564, 9_426_282, 9_801_282,
        # 2024
        0, 4_810_248, 7_671_902,
        # 2025
        1_500_000, 2_200_000, 8_648_816,
    ]
}

new_borrowing_item_2 = f_br.dict_to_polars_df(dict_new_borrowing_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_new_borrowing_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

new_borrowing_nao_acum_item_2 = new_borrowing_item_2.with_columns([expr_new_borrowing_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_new_borrowing_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

new_borrowing_acum_item_2 = new_borrowing_nao_acum_item_2.with_columns([expr_new_borrowing_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
new_borrowing_acum_item_2 = new_borrowing_acum_item_2[3:]

################################################################
# Lista dos dfs para concatenar
dfs_new_borrowing_trimestral = [
    new_borrowing_acum_item_1,
    new_borrowing_acum_item_2
]

new_borrowing_trimestral_eqtl = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_new_borrowing_trimestral)

# Somando as colunas
new_borrowing_trimestral_eqtl = new_borrowing_trimestral_eqtl.with_columns(
    (pl.col('vl_conta_1_acum') + pl.col('vl_conta_2_acum')).alias('new_borrowing')
).select('dt_refer', 'new_borrowing')

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Amortização de empréstimos e financiamentos' - código varia durante os anos;
    - 'debt_paid_item_2' - 'Amortização de debêntures' - código varia durante os anos;
    - 'debt_paid_item_3' - 'Amortização do passivo de arrendamento' - código varia durante os anos;
    - 'debt_paid_item_4' - 'Valores pagos referente a acordos com plano de recuperação judicial' - código varia durante os anos.

* Nos itens 'debt_paid_item_1', 'debt_paid_item_2', 'debt_paid_item_3' e 'debt_paid_item_4', adicionar manualmente os dados.

In [ ]:
# Debt paid

# DFC da EQTL
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 20010)

################################################################
# 'Amortização de empréstimos e financiamentos'
# '6.03.01' (06/21, 06/22, 06/23, 06/24, 06/25)
# '6.03.02' (03/20, 06/20, 09/20, 03/21, 09/21, 03/22, 09/22, 03/23, 09/23, 03/24, 09/24, 03/25, 09/25)
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_1': [
        # 2020
        -121_168, -240_900, -1_267_859,
        # 2021
        -1_024_095, -1_330_989, -2_284_336,
        # 2022
        -737_701, -1_706_148, -2_551_483,
        # 2023
        -7_204_580, -7_823_694, -8_474_240,
        # 2024
        -1_144_135, -2_477_505, -5_920_638,
        # 2025
        -2_815_191, -4_323_940, -4_950_376,
    ]
}

debt_paid_item_1 = f_br.dict_to_polars_df(dict_debt_paid_item_1)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

debt_paid_nao_acum_item_1 = debt_paid_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

debt_paid_acum_item_1 = debt_paid_nao_acum_item_1.with_columns([expr_debt_paid_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
debt_paid_acum_item_1 = debt_paid_acum_item_1[3:]

################################################################
# 'Amortização de debêntures'
# '6.03.03' (03/20, 06/20, 09/20, 03/21, 09/21, 03/22, 09/22, 03/23, 09/23, 03/24, 09/24, 03/25, 09/25)
# '6.03.04' (06/21, 06/22, 06/23, 06/24, 06/25)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        # 2020
        -695_500, -786_398, -786_398,
        # 2021
        0, 0, 0,
        # 2022
        -35_389, -894_253, -2_020_059,
        # 2023
        -1_631_749, -1_970_728, -2_296_564,
        # 2024
        -2_058_056, -3_440_172, -5_239_093,
        # 2025
        -1_127_511, -1_304_566, -2_515_189,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# 'Amortização do passivo de arrendamento'
# '6.03.05' (03/20, 06/21, 06/22, 06/23, 06/24, 06/25)
# '6.03.07' (06/20, 09/20, 09/21, 09/22, 09/23, 09/24, 09/25)
# '6.03.11' (03/21, 03/22, 03/23, 03/24, 03/25,)
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_3': [
        # 2020
        -6_086, -10_212, 0,
        # 2021
        -4_253, -9_148, -16_547,
        # 2022
        -13_963, -21_170, -21_736,
        # 2023
        -9_484, -18_330, -19_765,
        # 2024
        -6_127, -16_142, -20_607,
        # 2025
        -5_941, -9_799, -17_754,
    ]
}

debt_paid_item_3 = f_br.dict_to_polars_df(dict_debt_paid_item_3)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_3 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_3 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_3') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_3') - pl.col('vl_conta_3').shift(1))
    )
    .otherwise(pl.col('vl_conta_3')) # Mantém o valor original para outros meses
    .alias('vl_conta_3_nao_acum')
)

debt_paid_nao_acum_item_3 = debt_paid_item_3.with_columns([expr_debt_paid_nao_acum_item_3]).select('dt_refer', 'vl_conta_3_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_3 = (
    pl.col('vl_conta_3_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_3_acum')
)

debt_paid_acum_item_3 = debt_paid_nao_acum_item_3.with_columns([expr_debt_paid_acum_item_3]).select('dt_refer', 'vl_conta_3_acum').fill_null(0)
debt_paid_acum_item_3 = debt_paid_acum_item_3[3:]

################################################################
# 'Valores pagos referente a acordos com plano de recuperação judicial'
# '6.03.06' (06/21, 06/22, 06/23, 06/24, 06/25)
# '6.03.08' (03/20, 06/20, 09/20, 03/21, 09/21, 03/22, 09/22, 03/23, 09/23, 03/24, 09/24, 03/25, 09/25)
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_4': [
        # 2020
        -24_489, -27_247, -47_061,
        # 2021
        -24_835, 0, -92_883,
        # 2022
        0, -2_193, -8_774,
        # 2023
        -8_774, -15_354, -19_741,
        # 2024
        -6_580, -13_160, -21_934,
        # 2025
        -6_580, -13_160, -19_741,
    ]
}

debt_paid_item_4 = f_br.dict_to_polars_df(dict_debt_paid_item_4)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_4 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_4 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_4') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_4') - pl.col('vl_conta_4').shift(1))
    )
    .otherwise(pl.col('vl_conta_4')) # Mantém o valor original para outros meses
    .alias('vl_conta_4_nao_acum')
)

debt_paid_nao_acum_item_4 = debt_paid_item_4.with_columns([expr_debt_paid_nao_acum_item_4]).select('dt_refer', 'vl_conta_4_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_4 = (
    pl.col('vl_conta_4_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_4_acum')
)

debt_paid_acum_item_4 = debt_paid_nao_acum_item_4.with_columns([expr_debt_paid_acum_item_4]).select('dt_refer', 'vl_conta_4_acum').fill_null(0)
debt_paid_acum_item_4 = debt_paid_acum_item_4[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2,
    debt_paid_acum_item_3,
    debt_paid_acum_item_4,
]
debt_paid_trimestral_eqtl = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_eqtl = debt_paid_trimestral_eqtl.with_columns(
    ((
        pl.col('vl_conta_1_acum') +
        pl.col('vl_conta_2_acum') +
        pl.col('vl_conta_3_acum') +
        pl.col('vl_conta_4_acum')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_trimestral_eqtl = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_eqtl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_eqtl,
    df_new_borrowing=new_borrowing_trimestral_eqtl,
    df_debt_paid=debt_paid_trimestral_eqtl,
)

# FCFF
fcff_trimestral_eqtl = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20010,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_eqtl,
    df_capex=capex_trimestral_acum_eqtl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_eqtl,
)

In [ ]:
# Lista dos df
lst_df_trimestral_eqtl = [
    vm_trimestral_eqtl,
    lpa_trimestral_eqtl,
    lpa_trimestral_acum_eqtl.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_eqtl,
    lp_trimestral_eqtl,
    ebitda_trimestral_eqtl,
    ebitda_trimestral_acum_eqtl.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_eqtl,
    caixa_trimestral_eqtl,
    divida_liquida_trimestral_eqtl,
    dl_ebitda_trimestral_eqtl,
    dl_pl_trimestral_eqtl,
    ev_ebitda_trimestral_eqtl,
    vpa_trimestral_eqtl,
    p_vp_trimestral_eqtl,
    ml_trimestral_eqtl,
    roe_trimestral_eqtl,
    roic_trimestral_eqtl,
    proventos_trimestral_eqtl,
    payout_trimestral_eqtl,
    buyback_trimestral_eqtl,
    fco_trimestral_eqtl,
    fci_trimestral_eqtl,
    fcf_trimestral_eqtl,
    fco_trimestral_acum_eqtl.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_eqtl.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_eqtl.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_eqtl,
    capex_trimestral_acum_eqtl.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_eqtl,
    net_capex_trimestral_eqtl,
    rd_trimestral_eqtl,
    rd_trimestral_acum_eqtl.rename({'rd': 'rd_acum'}),
    adjusted_net_capex_trimestral_eqtl,
    working_capital_trimestral_eqtl,
    change_in_non_cash_wc_trimestral_eqtl,
    change_in_non_cash_wc_trimestral_acum_eqtl.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_eqtl,
    effective_tax_rate_trimestral_eqtl,
    fcfe_trimestral_eqtl,
    fcff_trimestral_eqtl
]

# Juntando os dfs
df_indicadores_trimestral_eqtl = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_eqtl
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_eqtl = df_indicadores_trimestral_eqtl.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_eqtl

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_eqtl.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//eqtl//eqtl_indicadores_trimestral.csv'),
    separator=';'
)

## CPLE3

In [ ]:
# Número de ações (2017 a 2019) - CPLE
dict_composicao_capital_17_18_19_cple = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        145_031_080,
        145_031_080,
        145_031_080,
    ],

    'num_pn':[
        128_624_295,
        128_624_295,
        128_624_295,
    ]
}

## CMIG4

In [ ]:
# Número de ações (2017 a 2019) - CMIG
dict_composicao_capital_17_18_19_cmig = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        420_765_000,
        487_614_000,
        487_614_000,
    ],

    'num_pn':[
        838_077_000,
        971_138_000,
        971_138_000,
    ]
}

# **Setor de Energia Elétrica - Geração**

## AXIA3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_axia';
    - R&D no dicionário 'dict_rd_axia'.

In [ ]:
# Número de ações - AXIA
dict_composicao_capital_17_18_19_axia = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        1_087_050_297,
        1_087_050_297,
        1_087_050_297,
    ],

    'num_pn':[
        265_583_803,
        265_583_803,
        265_583_803,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_axia = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_axia)

# Calculando o total de ações
df_composicao_capital_17_18_19_axia = df_composicao_capital_17_18_19_axia.with_columns(
    (pl.col('num_on') + pl.col('num_pn')).alias('num_total')
)

# Composição de capital (2020 a 2024) - AXIA
composicao_capital_axia = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('ELETROBRAS'))

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Arrumando a escala dos anos específicos
composicao_capital_axia = composicao_capital_axia.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on'),

    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_pn') * 1000)
      .otherwise(pl.col('num_pn'))
      .alias('num_pn'),

    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_total') * 1000)
      .otherwise(pl.col('num_total'))
      .alias('num_total'),
]).select('dt_refer', 'num_on', 'num_pn', 'num_total')

# Juntando os dfs
df_composicao_capital_axia = pl.concat([df_composicao_capital_17_18_19_axia, composicao_capital_axia])

##############################################################################
# Preços de fechamento - AXIA
df_axia_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit=None,
    list_ticker_on_pn=['AXIA3.SA', 'AXIA6.SA'],
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_axia.tail())
print('=' * 80)
print(df_axia_fechamento_anual.tail())

In [ ]:
# AXIA

# Valor de mercado atual
vm_atual_axia = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='AXIA3.SA', ticker_pn='AXIA6.SA')

# Valor de mercado
vm_axia = f_br.indicador_vm(df_capital_social=df_composicao_capital_axia, df_fechamento=df_axia_fechamento_anual, novo_mercado=False)

# LPA
lpa_axia = f_br.indicador_lpa(df_capital_social=df_composicao_capital_axia, df_dre=dre, cod_cvm=2437, cod_ll='3.11.01', novo_mercado=False)

# P/L
pl_axia = f_br.indicador_pl_lp(df_fechamento=df_axia_fechamento_anual, tipo_acao='on', df_lpa=lpa_axia)[0]

# L/P
lp_axia = f_br.indicador_pl_lp(df_fechamento=df_axia_fechamento_anual, tipo_acao='on', df_lpa=lpa_axia)[1]

# EBITDA
ebitda_axia = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=2437, cod_ebit='3.05', cod_depreciacao='6.01.01.05')

# Dívida bruta
divida_bruta_axia = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=2437, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_axia = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=2437, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_axia = f_br.indicador_divida_liquida(df_caixa_total=caixa_axia, df_divida_bruta=divida_bruta_axia)

# DL/EBITDA
dl_ebitda_axia = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_axia, df_ebitda=ebitda_axia)

# DL/PL
dl_pl_axia = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=2437, cod_pl='2.03', df_divida_liquida=divida_liquida_axia)

# EV/EBITDA
ev_ebitda_axia = f_br.indicador_ev_ebitda(df_vm=vm_axia, df_divida_liquida=divida_liquida_axia, df_ebitda=ebitda_axia)

# VPA
vpa_axia = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_axia, cod_cvm=2437, cod_pl='2.03', novo_mercado=False)

# P/VP
p_vp_axia = f_br.indicador_p_vpa(df_fechamento=df_axia_fechamento_anual, tipo_acao='on', df_vpa=vpa_axia)

# Margem líquida
ml_axia = f_br.indicador_ml(df_dre=dre, cod_cvm=2437, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_axia = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=2437, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_axia = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=2437, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_axia)

# Dividendos
proventos_axia = f_br.indicador_proventos(df_dva=dva, cod_cvm=2437, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_axia = f_br.indicador_payout(df_dre=dre, cod_cvm=2437, df_proventos=proventos_axia, cod_ll='3.11')

# Buyback - o código do buyback não é fixo (2023: '6.03.07'; 2024: '6.03.04')
dict_buyback_axia = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1_927_218,
        115_099
    ]
}

buyback_axia = f_br.dict_to_polars_df(dict_buyback_axia)

# FCO
fco_axia = f_br.indicador_fco(df_dfc=dfc, cod_cvm=2437, cod_fco='6.01')

# FCI
fci_axia = f_br.indicador_fci(df_dfc=dfc, cod_cvm=2437, cod_fci='6.02')

# FCF
fcf_axia = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=2437, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.03', # 'Aquisição de ativo imobilizado'
    '6.02.04'  # 'Aquisição de ativo intangível'
]

capex_axia = f_br.indicador_capex(df_dfc=dfc, cod_cvm=2437, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_axia = f_br.indicador_free_cash_flow(df_fco=fco_axia, df_capex=capex_axia)

# Net capex
net_capex_axia = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=2437, cod_depreciacao='6.01.01.05', df_capex=capex_axia)

# R&D
dict_rd_axia = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd': [
        240_812+671_791,
        425_669+698_917,
        397_125+730_246,
        371_364+744_395,
        235_492+649_303,
        583_855+464_330,
        461_631+432_322,
        455_880+942_348
    ]
}

rd_axia = f_br.dict_to_polars_df(dict_rd_axia)

# Adjusted net capex
adjusted_net_capex_axia = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_axia, df_rd=rd_axia)

# Working capital
working_capital_axia = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=2437,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_axia = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=2437, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_axia = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_axia,
    df_change_in_non_cash_wc=change_in_non_cash_wc_axia
)

# Effective tax rate
effective_tax_rate_axia = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=2437, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_axia = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=2437, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_axia, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Empréstimos e financiamentos obtidos / debentures obitdas'
]

new_borrowing_axia = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=2437, lst_cod_new_borrowing=lst_cod_new_borrowing)

# Arrumando o número de 2023 (esse dado está no código '6.03.02')
new_borrowing_axia = new_borrowing_axia.with_columns([
    pl.when(pl.col('dt_refer').is_in(datetime(2023, 12, 31)))
      .then(11_821_045)
      .otherwise(pl.col('new_borrowing'))
      .alias('new_borrowing')
])

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de principal sobre empréstimos' - '6.03.02';
    - 'debt_paid_item_2' - 'Juros pagos sobre empréstimos' - código varia durante os anos.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* No 'debt_paid_item_2', adicionar manualmente os dados.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da AXIA
dfc_axia = dfc.filter(pl.col('cd_cvm') == 2437)

################################################################
# 'Pagamento de principal sobre empréstimos '
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2017, 12, 31),
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_axia
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

################################################################
# 'Pagamento de arrendamentos financeiros'
# '6.03.08' (2023)
# '6.03.07' (2024)
# '6.03.06' (2022, 2021, 2020, 2019)
# Adicionar manualmente é mais fácil do que criar três dfs diferentes
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_2': [
        0.0,
        0.0,
        -547_226,
        -556_876,
        -600_470,
        -721_074,
        -765_525,
        -757_196,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_item_1, debt_paid_item_2]
debt_paid_axia = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_axia = debt_paid_axia.with_columns(
    ((pl.col('vl_conta_1') + pl.col('vl_conta_2')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

# FCFE
fcfe_axia = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=2437,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.05',
    df_capex=capex_axia,
    df_change_in_non_cash_wc=change_in_non_cash_wc_axia,
    df_new_borrowing=new_borrowing_axia,
    df_debt_paid=debt_paid_axia,
)

# FCFF
fcff_axia = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.05',
    df_taxes_on_operating_income=taxes_on_operating_income_axia,
    df_capex=capex_axia,
    df_change_in_non_cash_wc=change_in_non_cash_wc_axia,
)

In [ ]:
# Lista dos df
lst_df_axia = [
    vm_axia,
    lpa_axia,
    pl_axia,
    lp_axia,
    ebitda_axia,
    divida_bruta_axia,
    caixa_axia,
    divida_liquida_axia,
    dl_ebitda_axia,
    dl_pl_axia,
    ev_ebitda_axia,
    vpa_axia,
    p_vp_axia,
    ml_axia,
    roe_axia,
    roic_axia,
    proventos_axia,
    payout_axia,
    buyback_axia,
    fco_axia,
    fci_axia,
    fcf_axia,
    capex_axia,
    free_cash_flow_axia,
    net_capex_axia,
    rd_axia,
    adjusted_net_capex_axia,
    working_capital_axia,
    change_in_non_cash_wc_axia,
    reinvestment_rate_axia,
    effective_tax_rate_axia,
    fcfe_axia,
    fcff_axia
]

# Juntando os dfs
df_indicadores_axia = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_axia
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_axia = df_indicadores_axia.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_axia

In [ ]:
# Transformando em um arquivo csv
df_indicadores_axia.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//axia//axia_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - R&D no dicionário 'dict_rd_trimestral_axia'.

In [ ]:
# Composição de capital - AXIA
df_composicao_capital_trimestral_axia = composicao_capital_trimestral.filter(
    composicao_capital_trimestral['denom_cia'].str.contains('ELETROBRAS')
)

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),
]

# Arrumando a escala dos anos específicos
df_composicao_capital_trimestral_axia = df_composicao_capital_trimestral_axia.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on'),

    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_pn') * 1000)
      .otherwise(pl.col('num_pn'))
      .alias('num_pn'),

    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_total') * 1000)
      .otherwise(pl.col('num_total'))
      .alias('num_total'),
]).select('dt_refer', 'num_on', 'num_pn', 'num_total')

# Calculando o total de ações
df_composicao_capital_trimestral_axia = df_composicao_capital_trimestral_axia.with_columns(
    (pl.col('num_on') + pl.col('num_pn')).alias('num_total')
).select('dt_refer', 'num_on', 'num_pn', 'num_total')

##############################################################################
# Preços de fechamento - AXIA
df_axia_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit=None,
    list_ticker_on_pn=['AXIA3.SA', 'AXIA6.SA'],
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_axia['dt_refer']
)

print(df_composicao_capital_trimestral_axia.tail())
print('=' * 80)
print(df_axia_fechamento_trimestral.tail())

In [ ]:
# AXIA

# Valor de mercado
vm_trimestral_axia = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_axia[3:],
    df_fechamento=df_axia_fechamento_trimestral[3:],
    novo_mercado=False
)

# LPA
lpa_trimestral_axia = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_axia,
    df_dre=dre_trimestral,
    cod_cvm=2437,
    cod_ll='3.11.01',
    novo_mercado=False
)[0]

lpa_trimestral_acum_axia = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_axia,
    df_dre=dre_trimestral,
    cod_cvm=2437,
    cod_ll='3.11.01',
    novo_mercado=False
)[1]

# P/L
pl_trimestral_axia = f_br.indicador_pl_lp(
    df_fechamento=df_axia_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_axia
)[0]

# L/P
lp_trimestral_axia = f_br.indicador_pl_lp(
    df_fechamento=df_axia_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_axia
)[1]

# EBITDA
ebitda_trimestral_axia = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.05'
)[0]

ebitda_trimestral_acum_axia = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.05'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_axia = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=2437,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_axia = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=2437,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_axia = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_axia,
    df_divida_bruta=divida_bruta_trimestral_axia
)

# DL/EBITDA
dl_ebitda_trimestral_axia = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_axia,
    df_ebitda=ebitda_trimestral_acum_axia
)

# DL/PL
dl_pl_trimestral_axia = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=2437,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_axia
)

# EV/EBITDA
ev_ebitda_trimestral_axia = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_axia,
    df_divida_liquida=divida_liquida_trimestral_axia,
    df_ebitda=ebitda_trimestral_acum_axia
)

# VPA
vpa_trimestral_axia = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_axia[3:],
    cod_cvm=2437,
    cod_pl='2.03',
    novo_mercado=False
)

# P/VP
p_vp_trimestral_axia = f_br.indicador_p_vpa(
    df_fechamento=df_axia_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_axia
)

# Margem líquida
ml_trimestral_axia = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=2437,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_axia = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=2437,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_axia = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_axia
)

# Proventos
proventos_trimestral_axia = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=2437,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_axia = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=2437,
    df_proventos=proventos_trimestral_axia,
    cod_ll='3.11'
)

# Buyback
buyback_trimestral_axia = f_br.indicador_buyback_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_buyback='6.03.04'
)

DATAS_BUYBACK = [
    # Todas as datas que o buyback está errado
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
]

# Arrumando alguns números
buyback_trimestral_axia = buyback_trimestral_axia.with_columns([
    pl
    .when(pl.col('dt_refer').is_in(DATAS_BUYBACK))
    .then(0)
    .otherwise(pl.col('buyback_nao_acum'))
    .alias('buyback_nao_acum')
])[3:]

# FCO
fco_trimestral_axia = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_axia = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_axia = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_axia = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_axia = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_axia = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.03', # 'Aquisição de ativo imobilizado'
    '6.02.04'  # 'Aquisição de ativo intangível'
]

capex_trimestral_axia = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_axia = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_axia = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_axia,
    df_capex=capex_trimestral_acum_axia
)

# Net capex
net_capex_trimestral_axia = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_depreciacao='6.01.01.05',
    df_capex=capex_trimestral_acum_axia
)

# R&D - a AXIA não divulga esse nº no trimestre, somente no anual
dict_rd_trimestral_axia = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd': [
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

rd_trimestral_axia = f_br.dict_to_polars_df(dict_rd_trimestral_axia)

# Adjusted net capex
adjusted_net_capex_trimestral_axia = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_axia,
    df_rd=rd_trimestral_axia
)

# Working capital
working_capital_trimestral_axia = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=2437,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)[3:]

# Change in non-cash wc
change_in_non_cash_wc_trimestral_axia = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_axia = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_axia = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_axia,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_axia
)

# Effective tax rate
effective_tax_rate_trimestral_axia = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=2437,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_axia = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=2437,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_axia,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Empréstimos e financiamentos obtidos / debentures obitdas'
]

new_borrowing_trimestral_axia = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de principal sobre empréstimos ' - '6.03.02';
    - 'debt_paid_item_2' - 'Juros pagos sobre empréstimos' - código varia durante os anos.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* No 'debt_paid_item_2', adicionar manualmente os dados.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da AXIA
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 2437)

################################################################
# 'Empréstimos, financiamentos e debêntures obtidos'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),

]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

debt_paid_nao_acum_item_1 = debt_paid_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

debt_paid_acum_item_1 = debt_paid_nao_acum_item_1.with_columns([expr_debt_paid_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
debt_paid_acum_item_1 = debt_paid_acum_item_1[3:]

################################################################
# 'Pagamento de arrendamentos financeiros'
# '6.03.08' (03/25; 06/25; 09/25)
# '6.03.07' (03/23; 06/23; 09/23; 03/24; 06/24)
# '6.03.06' (03/21; 06/21; 03/22; 06/22)
# '6.03.05' (09/21; 09/22)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        -139_451, -274_564, -409_831,
        -161_148, -322_728, -380_978,
        -182_239, -359_789, -540_369,
        -190_934, -389_484, -574_095,
        -16_088, -30_891, -48_167,
        -6_063, -25_868, -39_360,

    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2
]
debt_paid_trimestral_axia = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_axia = debt_paid_trimestral_axia.with_columns(
    ((pl.col('vl_conta_1_acum') + pl.col('vl_conta_2_acum')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

# FCFE
fcfe_trimestral_axia = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.05',
    df_capex=capex_trimestral_acum_axia,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_axia,
    df_new_borrowing=new_borrowing_trimestral_axia,
    df_debt_paid=debt_paid_trimestral_axia,
)

# FCFF
fcff_trimestral_axia = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=2437,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.05',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_axia,
    df_capex=capex_trimestral_acum_axia,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_axia,
)

In [ ]:
# Lista dos df
lst_df_trimestral_axia = [
    vm_trimestral_axia,
    lpa_trimestral_axia,
    lpa_trimestral_acum_axia.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_axia,
    lp_trimestral_axia,
    ebitda_trimestral_axia,
    ebitda_trimestral_acum_axia.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_axia,
    caixa_trimestral_axia,
    divida_liquida_trimestral_axia,
    dl_ebitda_trimestral_axia,
    dl_pl_trimestral_axia,
    ev_ebitda_trimestral_axia,
    vpa_trimestral_axia,
    p_vp_trimestral_axia,
    ml_trimestral_axia,
    roe_trimestral_axia,
    roic_trimestral_axia,
    proventos_trimestral_axia,
    payout_trimestral_axia,
    buyback_trimestral_axia,
    fco_trimestral_axia,
    fci_trimestral_axia,
    fcf_trimestral_axia,
    fco_trimestral_acum_axia.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_axia.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_axia.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_axia,
    capex_trimestral_acum_axia.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_axia,
    net_capex_trimestral_axia,
    rd_trimestral_axia,
    # rd_trimestral_acum_axia.rename({'rd': 'rd_acum'}),     # Não divulga os nº nos trimestres
    adjusted_net_capex_trimestral_axia,
    working_capital_trimestral_axia,
    change_in_non_cash_wc_trimestral_axia,
    change_in_non_cash_wc_trimestral_acum_axia.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_axia,
    effective_tax_rate_trimestral_axia,
    fcfe_trimestral_axia,
    fcff_trimestral_axia
]

# Juntando os dfs
df_indicadores_trimestral_axia = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_axia
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_axia = df_indicadores_trimestral_axia.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_axia

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_axia.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//axia//axia_indicadores_trimestral.csv'),
    separator=';'
)

## EGIE3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_egie';
    - capex no dicionário 'dict_capex_egie';
    - R&D no dicionário 'dict_rd_egie'.

In [ ]:
# Número de ações (2017 a 2019) - EGIE
dict_composicao_capital_17_18_19_egie = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        652_742_192,
        815_927_740,
        815_927_740,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_egie = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_egie)

# Composição de capital (2020 a 2024) - EGIE
composicao_capital_egie = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('ENGIE')).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_egie = pl.concat([df_composicao_capital_17_18_19_egie, composicao_capital_egie])

##############################################################################
# Preços de fechamento - EGIE
df_egie_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='EGIE3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_egie.tail())
print('=' * 80)
print(df_egie_fechamento_anual.tail())

In [ ]:
# EGIE3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='EGIE3.SA', ticker_pn=None)

# Valor de mercado
vm_egie = f_br.indicador_vm(df_capital_social=df_composicao_capital_egie, df_fechamento=df_egie_fechamento_anual, novo_mercado=True)

# LPA
lpa_egie = f_br.indicador_lpa(df_capital_social=df_composicao_capital_egie, df_dre=dre, cod_cvm=17329, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_egie = f_br.indicador_pl_lp(df_fechamento=df_egie_fechamento_anual, tipo_acao='on', df_lpa=lpa_egie)[0]

# L/P
lp_egie = f_br.indicador_pl_lp(df_fechamento=df_egie_fechamento_anual, tipo_acao='on', df_lpa=lpa_egie)[1]

# EBITDA
ebitda_egie = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=17329, cod_ebit='3.05', cod_depreciacao='6.01.01.03')

# Dívida bruta
divida_bruta_egie = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=17329, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_egie = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=17329, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_egie = f_br.indicador_divida_liquida(df_caixa_total=caixa_egie, df_divida_bruta=divida_bruta_egie)

# DL/EBITDA
dl_ebitda_egie = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_egie, df_ebitda=ebitda_egie)

# DL/PL
dl_pl_egie = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=17329, cod_pl='2.03', df_divida_liquida=divida_liquida_egie)

# EV/EBITDA
ev_ebitda_egie = f_br.indicador_ev_ebitda(df_vm=vm_egie, df_divida_liquida=divida_liquida_egie, df_ebitda=ebitda_egie)

# VPA
vpa_egie = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_egie, cod_cvm=17329, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_egie = f_br.indicador_p_vpa(df_fechamento=df_egie_fechamento_anual, tipo_acao='on', df_vpa=vpa_egie)

# Margem líquida
ml_egie = f_br.indicador_ml(df_dre=dre, cod_cvm=17329, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_egie = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=17329, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_egie = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=17329, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_egie)

# Dividendos
proventos_egie = f_br.indicador_proventos(df_dva=dva, cod_cvm=17329, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_egie = f_br.indicador_payout(df_dre=dre, cod_cvm=17329, df_proventos=proventos_egie, cod_ll='3.11')

# Buyback
dict_buyback_egie = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
}

buyback_egie = f_br.dict_to_polars_df(dict_buyback_egie)

# FCO
fco_egie = f_br.indicador_fco(df_dfc=dfc, cod_cvm=17329, cod_fco='6.01')

# FCI
fci_egie = f_br.indicador_fci(df_dfc=dfc, cod_cvm=17329, cod_fci='6.02')

# FCF
fcf_egie = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=17329, cod_fcf='6.03')

# CAPEX - o código do capex ('Aplicação no imobilizado e intangível' e 'Aquisição de subsidiárias') não são fixos - 2024 ('6.02.09'); 2023 ('6.02.08'); 2022 ('6.02.06'); 2021 ('6.02.07'); 2020, 2019 e 2017 ('6.02.04'); 2018 ('6.02.02')
dict_capex_egie = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'capex': [
        1_904_866 + 1_051_482,
        3_305_514,
        1_165_832,
        1_020_131,
        1_229_753,
        1_528_874 + 659_959,
        2_565_799,
        6_646_060 + 2_361_046,
    ]
}

capex_egie = f_br.dict_to_polars_df(dict_capex_egie)

# Free cash flow
free_cash_flow_egie = f_br.indicador_free_cash_flow(df_fco=fco_egie, df_capex=capex_egie)

# Net capex
net_capex_egie = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=17329, cod_depreciacao='6.01.01.03', df_capex=capex_egie)

# R&D
dict_rd_egie = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        36_406,
        42_260,
        43_766,
        53_175,
        49_443,
        52_433,
        54_717,
        49_810,
    ]
}

df_rd_egie = f_br.dict_to_polars_df(dict_rd_egie)

# Adjusted net capex
adjusted_net_capex_egie = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_egie, df_rd=df_rd_egie)

# Working capital
working_capital_egie = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=17329,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_egie = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=17329, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_egie = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_egie,
    df_change_in_non_cash_wc=change_in_non_cash_wc_egie
)

# Effective tax rate
effective_tax_rate_egie = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=17329, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_egie = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=17329, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_egie, anual=True)

* Itens que compõem o 'new borrowing':
    - 'new_borrowing_item_1' - 'Captação de empréstimos e financiamentos' ou 'Ingresso de instrumentos de dívida' - '6.03.01'
    - 'new_borrowing_item_2' - 'Emissão de debêntures' - '6.03.02'

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_new_borrowing_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# New borrowing - não utilizei a função 'indicador_new_borrowing', porque os dados de alguns itens estavam errados

# DFC da EGIE
dfc_egie = dfc.filter(pl.col('cd_cvm') == 17329)

################################################################
# 'Captação de empréstimos e financiamentos' ou 'Ingresso de instrumentos de dívida' estão no mesmo código
DATAS_ALVO_NB_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2017, 12, 31),
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_1 = (
    dfc_egie
    .filter(
        (pl.col('cd_conta') == '6.03.01') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_NB_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

################################################################
# 'Emissão de debêntures'
DATAS_ALVO_NB_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2017, 12, 31),
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_2 = (
    dfc_egie
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_NB_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_new_borrowing_item_2 = {
    'dt_refer': [
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_2': [
        0.0,
        0.0,
        0.0,
    ]
}

new_borrowing_2_complementos = f_br.dict_to_polars_df(dict_new_borrowing_item_2)
new_borrowing_item_2 = pl.concat([new_borrowing_2_complementos, new_borrowing_item_2])
new_borrowing_item_2 = new_borrowing_item_2.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_new_borrowing = [
    new_borrowing_item_1,
    new_borrowing_item_2
]

new_borrowing_egie = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_new_borrowing)

# Somando as colunas
new_borrowing_egie = new_borrowing_egie.with_columns(
    (pl.col('vl_conta_1') + pl.col('vl_conta_2')).alias('new_borrowing')
).select('dt_refer', 'new_borrowing')

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de instrumentos de dívida, líquido de hedge' - código varia durante os anos;
    - 'debt_paid_item_2' - 'Pagamento de debêntures, líquido de hedge' - '6.03.05';
    - 'debt_paid_item_3' - 'Pagamento de parcelas de concessões' - código varia durante os anos;
    - 'debt_paid_item_4' - 'Pagamento de arrendamentos' - '6.03.08'.


* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho;
* Nos itens 'debt_paid_item_1' e 'debt_paid_item_3', adicionar manualmente os dados.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da EGIE
dfc_egie = dfc.filter(pl.col('cd_cvm') == 17329)

################################################################
# 'Pagamento de instrumentos de dívida, líquido de hedge'
# '6.03.02' (2024 e 2023)
# '6.03.03' (2022, 2021, 2019, 2018 e 2017)
# '6.03.04' (2020)
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_1': [
        -505_399.0,
        -2_290_194.0,
        -2_172_993.0,
        -1_437_609.0,
        -1_258_671.0,
        -3_489_879.0,
        -1_497_918.0,
        -2_462_243.0,
    ]
}

debt_paid_item_1 = f_br.dict_to_polars_df(dict_debt_paid_item_1)

################################################################
# 'Pagamento de debêntures, líquido de hedge'
DATAS_ALVO_DP_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_egie
    .filter(
        (pl.col('cd_conta') == '6.03.05') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_2': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
}

debt_debt_paid_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_debt_paid_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

################################################################
# 'Pagamento de parcelas de concessões'
# '6.03.04' (2020 e 2021)
# '6.03.06' (2017, 2018 e 2019)
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_3': [
        -68_719.0,
        -70_885.0,
        -79_755.0,
        -153_879.0,
        -243_432.0,
        0.0,
        0.0,
        0.0,
    ]
}

debt_paid_item_3 = f_br.dict_to_polars_df(dict_debt_paid_item_3)

################################################################
# 'Pagamento de arrendamentos'
DATAS_ALVO_DP_4 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_4 = (
    dfc_egie
    .filter(
        (pl.col('cd_conta') == '6.03.08') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_4))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_4')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_4': [
        0.0,
        0.0,
        -16_743,
        0.0,
        0.0,
        0.0,
    ]
}

debt_debt_paid_4_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_4)
debt_paid_item_4 = pl.concat([debt_debt_paid_4_complementos, debt_paid_item_4])
debt_paid_item_4 = debt_paid_item_4.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [
    debt_paid_item_1,
    debt_paid_item_2,
    debt_paid_item_3,
    debt_paid_item_4,
]

debt_paid_egie = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_egie = debt_paid_egie.with_columns(
    ((
        pl.col('vl_conta_1') +
        pl.col('vl_conta_2') +
        pl.col('vl_conta_3') +
        pl.col('vl_conta_4')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')
################################################################

# FCFE
fcfe_egie = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=17329,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.03',
    df_capex=capex_egie,
    df_change_in_non_cash_wc=change_in_non_cash_wc_egie,
    df_new_borrowing=new_borrowing_egie,
    df_debt_paid=debt_paid_egie,
)

# FCFF
fcff_egie = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.03',
    df_taxes_on_operating_income=taxes_on_operating_income_egie,
    df_capex=capex_egie,
    df_change_in_non_cash_wc=change_in_non_cash_wc_egie,
)

In [ ]:
# Lista dos df
lst_df_egie = [
    vm_egie,
    lpa_egie,
    pl_egie,
    lp_egie,
    ebitda_egie,
    divida_bruta_egie,
    caixa_egie,
    divida_liquida_egie,
    dl_ebitda_egie,
    dl_pl_egie,
    ev_ebitda_egie,
    vpa_egie,
    p_vp_egie,
    ml_egie,
    roe_egie,
    roic_egie,
    proventos_egie,
    payout_egie,
    buyback_egie,
    fco_egie,
    fci_egie,
    fcf_egie,
    capex_egie,
    free_cash_flow_egie,
    net_capex_egie,
    df_rd_egie,
    adjusted_net_capex_egie,
    working_capital_egie,
    change_in_non_cash_wc_egie,
    reinvestment_rate_egie,
    effective_tax_rate_egie,
    fcfe_egie,
    fcff_egie
]

# Juntando os dfs
df_indicadores_egie = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_egie
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_egie = df_indicadores_egie.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_egie

In [ ]:
# Transformando em um arquivo csv
df_indicadores_egie.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//egie//egie_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_trimestral_egie';
    - capex no dicionário 'dict_capex_trimestral_egie';
    - R&D no dicionário 'dict_rd_trimestral_egie'.

In [ ]:
# Composição de capital - EGIE
df_composicao_capital_trimestral_egie = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('ENGIE')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - EGIE
df_egie_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='EGIE3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_egie['dt_refer']
)

print(df_composicao_capital_trimestral_egie.tail())
print('=' * 80)
print(df_egie_fechamento_trimestral.tail())

In [ ]:
# EGIE3

# Valor de mercado
vm_trimestral_egie = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_egie[3:],
    df_fechamento=df_egie_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_egie = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_egie,
    df_dre=dre_trimestral,
    cod_cvm=17329,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_egie = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_egie,
    df_dre=dre_trimestral,
    cod_cvm=17329,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_egie = f_br.indicador_pl_lp(
    df_fechamento=df_egie_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_egie
)[0]

# L/P
lp_trimestral_egie = f_br.indicador_pl_lp(
    df_fechamento=df_egie_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_egie
)[1]

# EBITDA
ebitda_trimestral_egie = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.03'
)[0]

ebitda_trimestral_acum_egie = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.03'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_egie = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=17329,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_egie = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=17329,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_egie = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_egie,
    df_divida_bruta=divida_bruta_trimestral_egie
)

# DL/EBITDA
dl_ebitda_trimestral_egie = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_egie,
    df_ebitda=ebitda_trimestral_acum_egie
)

# DL/PL
dl_pl_trimestral_egie = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=17329,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_egie
)

# EV/EBITDA
ev_ebitda_trimestral_egie = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_egie,
    df_divida_liquida=divida_liquida_trimestral_egie,
    df_ebitda=ebitda_trimestral_acum_egie
)

# VPA
vpa_trimestral_egie = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_egie[3:],
    cod_cvm=17329,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_egie = f_br.indicador_p_vpa(
    df_fechamento=df_egie_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_egie
)

# Margem líquida
ml_trimestral_egie = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=17329,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_egie = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=17329,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_egie = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_egie
)

# Proventos
proventos_trimestral_egie = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=17329,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_egie = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=17329,
    df_proventos=proventos_trimestral_egie,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_egie = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'buyback': [
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

buyback_trimestral_egie = f_br.dict_to_polars_df(dict_buyback_trimestral_egie)

# FCO
fco_trimestral_egie = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_egie = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_egie = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_egie = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_egie = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_egie = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_fcf='6.03'
)[1][3:]

# CAPEX - o código do capex ('Aplicação no imobilizado e intangível' e 'Aquisição de subsidiárias') não são fixos
dict_capex_trimestral_egie = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    # 'Aplicação no imobilizado e intangível'
    'capex': [
        # 2020
        281_521, 444_953, 523_215,
        # 2021
        265_845, 541_033, 762_906,
        # 2022
        86_610, 459_145, 949_228,
        # 2023
        371_301, 914_616, 1_601_003,
        # 2024
        1_083_192, 3_096_437, 4_899_075,
        # 2025
        755_308, 1_167_741, 1_411_372
    ],

    # 'Aquisição de subsidiárias'
    'capex_2': [
        # 2020
        316_776, 328_195, 328_889,
        # 2021
        0, 0, 0,
        # 2022
        656_157, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        2_361_046, 0, 0,
        # 2025
        0, 0, 2_286_053,
    ]
}

capex_trimestral_egie = f_br.dict_to_polars_df(dict_capex_trimestral_egie)

# Calculando o capex não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_capex = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se capex é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('capex') == 0)
        .then(0)
        .otherwise(pl.col('capex') - pl.col('capex').shift(1))
    )
    .otherwise(pl.col('capex')) # Mantém o valor original para outros meses
    .alias('capex_nao_acum')
)

expr_capex_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se capex_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('capex_2') == 0)
        .then(0)
        .otherwise(pl.col('capex_2') - pl.col('capex_2').shift(1))
    )
    .otherwise(pl.col('capex_2')) # Mantém o valor original para outros meses
    .alias('capex_2_nao_acum')
)

capex_trimestral_nao_acum = capex_trimestral_egie.with_columns([expr_capex]).select('dt_refer', 'capex_nao_acum').fill_null(0)
capex_2_trimestral_nao_acum = capex_trimestral_egie.with_columns([expr_capex_2]).select('dt_refer', 'capex_2_nao_acum').fill_null(0)

# Calculando o capex trimestral acumulado
expr_capex_acum = (
    pl.col('capex_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('capex_acum')
)

expr_capex_2_acum = (
    pl.col('capex_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('capex_2_acum')
)

capex_trimestral_acum = capex_trimestral_nao_acum.with_columns([expr_capex_acum]).select('dt_refer', 'capex_acum').fill_null(0)
capex_2_trimestral_acum = capex_2_trimestral_nao_acum.with_columns([expr_capex_2_acum]).select('dt_refer', 'capex_2_acum').fill_null(0)

# Juntando os dfs
capex_trimestral_acum_egie = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), [capex_trimestral_acum, capex_2_trimestral_acum]
)

capex_trimestral_acum_egie = capex_trimestral_acum_egie.with_columns(
    (pl.col('capex_acum') + pl.col('capex_2_acum')).alias('capex')
).select('dt_refer', 'capex')[3:]

# Free cash flow
free_cash_flow_trimestral_egie = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_egie,
    df_capex=capex_trimestral_acum_egie
)

# Net capex
net_capex_trimestral_egie = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_depreciacao='6.01.01.03',
    df_capex=capex_trimestral_acum_egie
)

# R&D - essa informação está no arquivo DFP/ITR, na nota explicativa "RECEITA OPERACIONAL LÍQUIDA".
dict_rd_trimestral_egie = {
    'dt_refer':[
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2020
        15_717, 11_012, 13_173,
        # 2021
        13_885, 7_004, 5_362,
        # 2022
        8_461, 9_700, 8_238,
        # 2023
        9_918, 9_897, 7_738,
        # 2024
        10_556, 7_863, 8_867,
        # 2025
        12_389, 12_565, 12_132,
    ]
}

rd_trimestral_egie = f_br.dict_to_polars_df(dict_rd_trimestral_egie)

# Calculando o R&D acumulado
expr_rd_acum = (
    pl.col('rd')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
)

rd_trimestral_acum_egie = rd_trimestral_egie.with_columns([expr_rd_acum]).select('dt_refer', 'rd').fill_null(0)[3:]

# Adjusted net capex
adjusted_net_capex_trimestral_egie = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_egie,
    df_rd=rd_trimestral_acum_egie
)

# Working capital
working_capital_trimestral_egie = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=17329,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_egie = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_egie = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_egie = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_egie,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_egie
)

# Effective tax rate
effective_tax_rate_trimestral_egie = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=17329,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_egie = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=17329,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_egie,
    anual=False
)

* Itens que compõem o 'new borrowing':
    - 'new_borrowing_item_1' - 'Captação de empréstimos e financiamentos' ou 'Ingresso de instrumentos de dívida' - '6.03.01'
    - 'new_borrowing_item_2' - 'Emissão de debêntures' - '6.03.02'

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_new_borrowing_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# New borrowing

# DFC da EGIE
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 17329)

################################################################
# 'Captação de empréstimos e financiamentos' ou 'Ingresso de instrumentos de dívida' estão no mesmo código
DATAS_ALVO_NB_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_1 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.01') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_NB_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Calculando o new borrowing não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_new_borrowing_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

new_borrowing_nao_acum_item_1 = new_borrowing_item_1.with_columns([expr_new_borrowing_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o new borrowing trimestral acumulado
expr_new_borrowing_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

new_borrowing_acum_item_1 = new_borrowing_nao_acum_item_1.with_columns([expr_new_borrowing_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
new_borrowing_acum_item_1 = new_borrowing_acum_item_1[3:]

################################################################
# 'Emissão de debêntures'
DATAS_ALVO_NB_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_2 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_NB_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_new_borrowing_item_2 = {
    'dt_refer': [
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
    ]
}

new_borrowing_item_2_complementos = f_br.dict_to_polars_df(dict_new_borrowing_item_2)
new_borrowing_item_2 = pl.concat([new_borrowing_item_2_complementos, new_borrowing_item_2])
new_borrowing_item_2 = new_borrowing_item_2.sort('dt_refer')

# Calculando o new borrowing não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_new_borrowing_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

new_borrowing_nao_acum_item_2 = new_borrowing_item_2.with_columns([expr_new_borrowing_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o new borrowing trimestral acumulado
expr_new_borrowing_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

new_borrowing_acum_item_2 = new_borrowing_nao_acum_item_2.with_columns([expr_new_borrowing_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
new_borrowing_acum_item_2 = new_borrowing_acum_item_2[3:]

################################################################
# Lista dos dfs para concatenar
dfs_new_borrowing_trimestral = [
    new_borrowing_acum_item_1,
    new_borrowing_acum_item_2
]

new_borrowing_trimestral_egie = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_new_borrowing_trimestral)

# Somando as colunas
new_borrowing_trimestral_egie = new_borrowing_trimestral_egie.with_columns(
    (pl.col('vl_conta_1_acum') + pl.col('vl_conta_2_acum')).alias('new_borrowing')
).select('dt_refer', 'new_borrowing')

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de instrumentos de dívida, líquido de hedge' - código varia durante os anos;
    - 'debt_paid_item_2' - 'Pagamento de debêntures, líquido de hedge' - código varia durante os anos;
    - 'debt_paid_item_3' - 'Pagamento de parcelas de concessões' - código varia durante os anos;
    - 'debt_paid_item_4' - 'Pagamento de arrendamentos' - código varia durante os anos.

* Nos itens 'debt_paid_item_1', 'debt_paid_item_2', 'debt_paid_item_3' e 'debt_paid_item_4', adicionar manualmente os dados.

In [ ]:
# Debt paid

# DFC da EGIE
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 17329)

################################################################
# 'Pagamento de instrumentos de dívida, líquido de hedge'
# '6.03.02' (03/22, 06/23, 09/23, 03/24, 06/24, 09/24, 03/25, 06/25, 09/25)
# '6.03.03' (03/20, 06/02, 03/21, 09/22)
# '6.03.04' (09/20, 06/21, 06/22, 03/23)
# '6.03.05' (09/21)
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_1': [
        # 2020
        -119_899, -1_211_434, -545_630,
        # 2021
        -67_899, -1_182_324, -1_753_430,
        # 2022
        -73_264, -1_572_909, -1_661_838,
        # 2023
        -110_129, -897_698, -1_129_247,
        # 2024
        -767_361, -1_081_700, -1_788_706,
        # 2025
        -151_668, -468_489, -1_872_732,
    ]
}

debt_paid_item_1 = f_br.dict_to_polars_df(dict_debt_paid_item_1)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

debt_paid_nao_acum_item_1 = debt_paid_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

debt_paid_acum_item_1 = debt_paid_nao_acum_item_1.with_columns([expr_debt_paid_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
debt_paid_acum_item_1 = debt_paid_acum_item_1[3:]

################################################################
# 'Pagamento de debêntures, líquido de hedge'
# '6.03.04' (09/22)
# '6.03.05' (09/20; 06/21; 06/22)
# '6.03.06' (09/21)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        # 2020
        0, 0, -596_130,
        # 2021
        0, -110_273, -211_168,
        # 2022
        0, -139_981, -252_865,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# 'Pagamento de parcelas de concessões'
# '6.03.03' (03/22)
# '6.03.04' (03/20, 03/21)
# '6.03.05' (06/20, 09/22)
# '6.03.06' (09/20, 06/21, 06/22)
# '6.03.07' (09/21)
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],
    'vl_conta_3': [
        # 2020
        -36_417, -72_871, -109_468,
        # 2021
        -59_380, -118_829, -178_813,
        # 2022
        -71_273, -142_680, -214_762,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

debt_paid_item_3 = f_br.dict_to_polars_df(dict_debt_paid_item_3)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_3 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_3 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_3') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_3') - pl.col('vl_conta_3').shift(1))
    )
    .otherwise(pl.col('vl_conta_3')) # Mantém o valor original para outros meses
    .alias('vl_conta_3_nao_acum')
)

debt_paid_nao_acum_item_3 = debt_paid_item_3.with_columns([expr_debt_paid_nao_acum_item_3]).select('dt_refer', 'vl_conta_3_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_3 = (
    pl.col('vl_conta_3_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_3_acum')
)

debt_paid_acum_item_3 = debt_paid_nao_acum_item_3.with_columns([expr_debt_paid_acum_item_3]).select('dt_refer', 'vl_conta_3_acum').fill_null(0)
debt_paid_acum_item_3 = debt_paid_acum_item_3[3:]

################################################################
# 'Pagamento de arrendamentos'
# '6.03.05' (03/22)
# '6.03.06' (03/20, 06/20; 03/21)
# '6.03.07' (09/22)
# '6.03.08' (06/21; 06/22)
# '6.03.09' (09/21)
# '6.03.10' (09/20)
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_4': [
        # 2020
        -5_143, -10_163, -15_228,
        # 2021
        -5_532, -10_580, -14_617,
        # 2022
        -3_780, -10_240, -16_228,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

debt_paid_item_4 = f_br.dict_to_polars_df(dict_debt_paid_item_4)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_4 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_4 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_4') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_4') - pl.col('vl_conta_4').shift(1))
    )
    .otherwise(pl.col('vl_conta_4')) # Mantém o valor original para outros meses
    .alias('vl_conta_4_nao_acum')
)

debt_paid_nao_acum_item_4 = debt_paid_item_4.with_columns([expr_debt_paid_nao_acum_item_4]).select('dt_refer', 'vl_conta_4_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_4 = (
    pl.col('vl_conta_4_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_4_acum')
)

debt_paid_acum_item_4 = debt_paid_nao_acum_item_4.with_columns([expr_debt_paid_acum_item_4]).select('dt_refer', 'vl_conta_4_acum').fill_null(0)
debt_paid_acum_item_4 = debt_paid_acum_item_4[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2,
    debt_paid_acum_item_3,
    debt_paid_acum_item_4,
]
debt_paid_trimestral_egie = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_egie = debt_paid_trimestral_egie.with_columns(
    ((
        pl.col('vl_conta_1_acum') +
        pl.col('vl_conta_2_acum') +
        pl.col('vl_conta_3_acum') +
        pl.col('vl_conta_4_acum')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_trimestral_egie = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.03',
    df_capex=capex_trimestral_acum_egie,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_egie,
    df_new_borrowing=new_borrowing_trimestral_egie,
    df_debt_paid=debt_paid_trimestral_egie,
)

# FCFF
fcff_trimestral_egie = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=17329,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.03',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_egie,
    df_capex=capex_trimestral_acum_egie,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_egie,
)

In [ ]:
# Lista dos df
lst_df_trimestral_egie = [
    vm_trimestral_egie,
    lpa_trimestral_egie,
    lpa_trimestral_acum_egie.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_egie,
    lp_trimestral_egie,
    ebitda_trimestral_egie,
    ebitda_trimestral_acum_egie.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_egie,
    caixa_trimestral_egie,
    divida_liquida_trimestral_egie,
    dl_ebitda_trimestral_egie,
    dl_pl_trimestral_egie,
    ev_ebitda_trimestral_egie,
    vpa_trimestral_egie,
    p_vp_trimestral_egie,
    ml_trimestral_egie,
    roe_trimestral_egie,
    roic_trimestral_egie,
    proventos_trimestral_egie,
    payout_trimestral_egie,
    buyback_trimestral_egie,
    fco_trimestral_egie,
    fci_trimestral_egie,
    fcf_trimestral_egie,
    fco_trimestral_acum_egie.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_egie.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_egie.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_egie,
    capex_trimestral_acum_egie.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_egie,
    net_capex_trimestral_egie,
    rd_trimestral_egie,
    rd_trimestral_acum_egie.rename({'rd': 'rd_acum'}),
    adjusted_net_capex_trimestral_egie,
    working_capital_trimestral_egie,
    change_in_non_cash_wc_trimestral_egie,
    change_in_non_cash_wc_trimestral_acum_egie.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_egie,
    effective_tax_rate_trimestral_egie,
    fcfe_trimestral_egie,
    fcff_trimestral_egie
]

# Juntando os dfs
df_indicadores_trimestral_egie = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_egie
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_egie = df_indicadores_trimestral_egie.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_egie

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_egie.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//egie//egie_indicadores_trimestral.csv'),
    separator=';'
)

## ENEV3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_enev';
    - capex no dicionário 'dict_capex_enev';
    - R&D no dicionário 'dict_rd_enev'.

In [ ]:
# Número de ações (2017 a 2019) - ENEV
dict_composicao_capital_17_18_19_enev = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        314_990_000,
        314_990_000,
        315_483_000,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_enev = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_enev)

# Composição de capital (2020 a 2024) - ENEV
composicao_capital_enev = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('ENEVA')).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_enev = pl.concat([df_composicao_capital_17_18_19_enev, composicao_capital_enev])

##############################################################################
# Preços de fechamento - ENEV
df_enev_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='ENEV3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_enev.tail())
print('=' * 80)
print(df_enev_fechamento_anual.tail())

In [ ]:
# ENEV3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='ENEV3.SA', ticker_pn=None)

# Valor de mercado
vm_enev = f_br.indicador_vm(df_capital_social=df_composicao_capital_enev, df_fechamento=df_enev_fechamento_anual, novo_mercado=True)

# LPA
lpa_enev = f_br.indicador_lpa(df_capital_social=df_composicao_capital_enev, df_dre=dre, cod_cvm=21237, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_enev = f_br.indicador_pl_lp(df_fechamento=df_enev_fechamento_anual, tipo_acao='on', df_lpa=lpa_enev)[0]

# L/P
lp_enev = f_br.indicador_pl_lp(df_fechamento=df_enev_fechamento_anual, tipo_acao='on', df_lpa=lpa_enev)[1]

# EBITDA
ebitda_enev = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=21237, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_enev = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=21237, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_enev = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=21237, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_enev = f_br.indicador_divida_liquida(df_caixa_total=caixa_enev, df_divida_bruta=divida_bruta_enev)

# DL/EBITDA
dl_ebitda_enev = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_enev, df_ebitda=ebitda_enev)

# DL/PL
dl_pl_enev = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=21237, cod_pl='2.03', df_divida_liquida=divida_liquida_enev)

# EV/EBITDA
ev_ebitda_enev = f_br.indicador_ev_ebitda(df_vm=vm_enev, df_divida_liquida=divida_liquida_enev, df_ebitda=ebitda_enev)

# VPA
vpa_enev = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_enev, cod_cvm=21237, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_enev = f_br.indicador_p_vpa(df_fechamento=df_enev_fechamento_anual, tipo_acao='on', df_vpa=vpa_enev)

# Margem líquida
ml_enev = f_br.indicador_ml(df_dre=dre, cod_cvm=21237, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_enev = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=21237, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_enev = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=21237, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_enev)

# Dividendos
proventos_enev = f_br.indicador_proventos(df_dva=dva, cod_cvm=21237, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_enev = f_br.indicador_payout(df_dre=dre, cod_cvm=21237, df_proventos=proventos_enev, cod_ll='3.11')

# Buyback
dict_buyback_enev = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback': [
        0,
        0,
        0,
        0,
        154_258,
        0,
        0,
        0,
    ]
}

buyback_enev = f_br.dict_to_polars_df(dict_buyback_enev)

# FCO
fco_enev = f_br.indicador_fco(df_dfc=dfc, cod_cvm=21237, cod_fco='6.01')

# FCI
fci_enev = f_br.indicador_fci(df_dfc=dfc, cod_cvm=21237, cod_fci='6.02')

# FCF
fcf_enev = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=21237, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisição de imobilizado e intangível'
]

capex_enev_1 = f_br.indicador_capex(df_dfc=dfc, cod_cvm=21237, lst_cod_capex=lst_cod_capex)
capex_enev_1 = capex_enev_1.rename({'capex':'vl_conta_1'})

# Itens do capex em que o código varia durante os anos
dict_capex_enev = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    # Aquisição de empresas
    'vl_conta_2': [
        0,
        148_866, # 'Aquisição de controlada'
        0,
        0,
        0,
        (340_131 + 232_463 + 17_798 + 5_291_101), # ('Focus Energia', 'CGFT', 'Amapari' e 'Celse')
        0,
        0,
    ]
}

capex_enev_2 = f_br.dict_to_polars_df(dict_capex_enev)
capex_enev_2 = capex_enev_2.rename({'capex':'vl_conta_2'})

capex_enev = capex_enev_1.join(capex_enev_2, on='dt_refer', how='inner')

capex_enev = capex_enev.with_columns(
    (pl.col('vl_conta_1') + pl.col('vl_conta_2')).alias('capex')
).select('dt_refer', 'capex')

# Free cash flow
free_cash_flow_enev = f_br.indicador_free_cash_flow(df_fco=fco_enev, df_capex=capex_enev)

# Net capex
net_capex_enev = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=21237, cod_depreciacao='6.01.01.02', df_capex=capex_enev)

# R&D
# Essa informação está no arquivo DFP, na nota explicativa "Receita operacional líquida", na tabela "Prática contábil", na linha "P&D".
dict_rd_enev = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        0,
        28_729,
        30_138,
        29_936,
        47_625,
        44_768,
        68_968,
        72_740, # Essa informação estava na nota explicativa "2 . Informações por segmento do negócio"
    ]
}

rd_enev = f_br.dict_to_polars_df(dict_rd_enev)

# Adjusted net capex
adjusted_net_capex_enev = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_enev, df_rd=rd_enev)

# Working capital
working_capital_enev = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=21237,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_enev = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=21237, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_enev = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=21237,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_enev,
    df_change_in_non_cash_wc=change_in_non_cash_wc_enev
)

# Effective tax rate
effective_tax_rate_enev = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=21237, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_enev = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=21237, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_enev, anual=True)

* Itens que compõem o 'new borrowing':
    - 'new_borrowing_item_1' - 'Captações de financiamentos e debêntures' - '6.03.04'
    - 'new_borrowing_item_2' - 'Antecipação de recebíveis futuros' - '6.03.14'

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_new_borrowing_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# New borrowing

lst_cod_new_borrowing = [
    '6.03.04' # 'Captações de financiamentos e debêntures'
]

new_borrowing_item_1 = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=21237, lst_cod_new_borrowing=lst_cod_new_borrowing)
new_borrowing_item_1 = new_borrowing_item_1.rename({'new_borrowing':'vl_conta_1'})

################################################################
# DFC da ENEV
dfc_enev = dfc.filter(pl.col('cd_cvm') == 21237)

################################################################
# 'Antecipação de recebíveis futuros'
DATAS_ALVO_NB_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
new_borrowing_item_2 = (
    dfc_enev
    .filter(
        (pl.col('cd_conta') == '6.03.14') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_NB_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_new_borrowing_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
    ],

    'vl_conta_2': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
}

new_borrowing_2_complementos = f_br.dict_to_polars_df(dict_new_borrowing_item_2)
new_borrowing_item_2 = pl.concat([new_borrowing_2_complementos, new_borrowing_item_2])
new_borrowing_item_2 = new_borrowing_item_2.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_new_borrowing = [
    new_borrowing_item_1,
    new_borrowing_item_2
]

new_borrowing_enev = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_new_borrowing)

# Somando as colunas
new_borrowing_enev = new_borrowing_enev.with_columns(
    (pl.col('vl_conta_1') + pl.col('vl_conta_2')).alias('new_borrowing')
).select('dt_refer', 'new_borrowing')

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamentos de financiamentos e debêntures - juros' ou 'Juros pagos' - '6.03.09';
    - 'debt_paid_item_2' - 'Pagamento de empréstimos, financiamentos e debêntures - principal' - '6.03.02';
    - 'debt_paid_item_3' - 'Pagamento de passivo de arrendamento' - '6.03.08';
    - 'debt_paid_item_4' - 'Pagamentos de antecipação de recebíveis futuros - principal' - '6.03.15'.
    - 'debt_paid_item_5' - 'Pagamentos de antecipação de recebíveis futuros - juros' - '6.03.16'.


* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid

lst_cod_debt_paid = [
    '6.03.09' # 'Pagamentos de financiamentos e debêntures - juros' ou 'Juros pagos'
]

debt_paid_item_1 = f_br.indicador_debt_paid(df_dfc=dfc, cod_cvm=21237, lst_cod_debt_paid=lst_cod_debt_paid)
debt_paid_item_1 = debt_paid_item_1.rename({'debt_paid':'vl_conta_1'})

################################################################
# DFC da ENEV
dfc_enev = dfc.filter(pl.col('cd_cvm') == 21237)

################################################################
# 'Pagamento de empréstimos, financiamentos e debêntures - principal'
DATAS_ALVO_DP_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_enev
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
    ],

    'vl_conta_2': [
        -808_799,
        -2_184_868,
    ]
}

debt_debt_paid_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_debt_paid_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

################################################################
# 'Pagamento de passivo de arrendamento'
DATAS_ALVO_DP_3 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_3 = (
    dfc_enev
    .filter(
        (pl.col('cd_conta') == '6.03.08') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_3))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_3')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
    ],

    'vl_conta_3': [
        0.0,
        0.0,
    ]
}

debt_debt_paid_3_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_3)
debt_paid_item_3 = pl.concat([debt_debt_paid_3_complementos, debt_paid_item_3])
debt_paid_item_3 = debt_paid_item_3.sort('dt_refer')

################################################################
# 'Pagamentos de antecipação de recebíveis futuros - principal'
DATAS_ALVO_DP_4 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_4 = (
    dfc_enev
    .filter(
        (pl.col('cd_conta') == '6.03.15') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_4))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_4')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
    ],

    'vl_conta_4': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
}

debt_debt_paid_4_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_4)
debt_paid_item_4 = pl.concat([debt_debt_paid_4_complementos, debt_paid_item_4])
debt_paid_item_4 = debt_paid_item_4.sort('dt_refer')

################################################################
# 'Pagamentos de antecipação de recebíveis futuros - juros'
DATAS_ALVO_DP_5 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_5 = (
    dfc_enev
    .filter(
        (pl.col('cd_conta') == '6.03.16') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_5))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_5')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_5 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
    ],

    'vl_conta_5': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
}

debt_debt_paid_5_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_5)
debt_paid_item_5 = pl.concat([debt_debt_paid_5_complementos, debt_paid_item_5])
debt_paid_item_5 = debt_paid_item_5.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [
    debt_paid_item_1,
    debt_paid_item_2,
    debt_paid_item_3,
    debt_paid_item_4,
    debt_paid_item_5,
]

debt_paid_enev = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_enev = debt_paid_enev.with_columns(
    ((
        pl.col('vl_conta_1') +
        pl.col('vl_conta_2') +
        pl.col('vl_conta_3') +
        pl.col('vl_conta_4') +
        pl.col('vl_conta_5')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_enev = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=21237,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_enev,
    df_change_in_non_cash_wc=change_in_non_cash_wc_enev,
    df_new_borrowing=new_borrowing_enev,
    df_debt_paid=debt_paid_enev,
)

# FCFF
fcff_enev = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=21237,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_enev,
    df_capex=capex_enev,
    df_change_in_non_cash_wc=change_in_non_cash_wc_enev,
)

In [ ]:
# Lista dos df
lst_df_enev = [
    vm_enev,
    lpa_enev,
    pl_enev,
    lp_enev,
    ebitda_enev,
    divida_bruta_enev,
    caixa_enev,
    divida_liquida_enev,
    dl_ebitda_enev,
    dl_pl_enev,
    ev_ebitda_enev,
    vpa_enev,
    p_vp_enev,
    ml_enev,
    roe_enev,
    roic_enev,
    proventos_enev,
    payout_enev,
    buyback_enev,
    fco_enev,
    fci_enev,
    fcf_enev,
    capex_enev,
    free_cash_flow_enev,
    net_capex_enev,
    rd_enev,
    adjusted_net_capex_enev,
    working_capital_enev,
    change_in_non_cash_wc_enev,
    reinvestment_rate_enev,
    effective_tax_rate_enev,
    fcfe_enev,
    fcff_enev
]

# Juntando os dfs
df_indicadores_enev = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_enev
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_enev = df_indicadores_enev.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_enev

In [ ]:
# Transformando em um arquivo csv
df_indicadores_enev.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//energia-eletrica//enev//enev_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

In [ ]:
# Composição de capital - ENEV
df_composicao_capital_trimestral_enev = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('ENEVA')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - ENEV
df_enev_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='ENEV3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_enev['dt_refer']
)

print(df_composicao_capital_trimestral_enev.tail())
print('=' * 80)
print(df_enev_fechamento_trimestral.tail())

# **Setor de Energia Elétrica - Transmissão**

## ALUP11

In [ ]:
# Número de ações (2017 a 2019) - ALUP
dict_composicao_capital_17_18_19_alup = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        596_955_970,
        596_955_970,
        596_955_970,
    ],

    'num_pn':[
        282_155_299,
        282_155_299,
        282_155_299,
    ]
}

## ISAE3

In [ ]:
# Número de ações (2017 a 2019) - ISAE
dict_composicao_capital_17_18_19_isae = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        64_484_433,
        64_484_433,
        257_937_732,
    ],

    'num_pn':[
        100_236_393,
        100_236_393,
        400_945_572,
    ]
}

## TAEE11

In [ ]:
# Número de ações (2017 a 2019) - TAEE
dict_composicao_capital_17_18_19_taee = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        590_714_000,
        590_714_000,
        590_714_000,
    ],

    'num_pn':[
        442_783_000,
        442_783_000,
        442_783_000,
    ]
}

# **Setor de Farmácia**

## RADL3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_radl'.

In [ ]:
# Número de ações (2017 a 2019) - RADL
dict_composicao_capital_17_18_19_radl = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        330_386_000,
        330_386_000,
        330_386_000,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_radl = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_radl)

# Composição de capital (2020 a 2024) - EGIE
composicao_capital_radl = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('RAIADROGASIL')).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_radl = pl.concat([df_composicao_capital_17_18_19_radl, composicao_capital_radl])

##############################################################################
# Preços de fechamento - RADL
df_radl_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='RADL3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_radl.tail())
print('=' * 80)
print(df_radl_fechamento_anual.tail())

In [ ]:
# RADL3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='RADL3.SA', ticker_pn=None)

# Valor de mercado
vm_radl = f_br.indicador_vm(df_capital_social=df_composicao_capital_radl, df_fechamento=df_radl_fechamento_anual, novo_mercado=True)

# LPA
lpa_radl = f_br.indicador_lpa(df_capital_social=df_composicao_capital_radl, df_dre=dre, cod_cvm=5258, cod_ll='3.11', novo_mercado=True)

# P/L
pl_radl = f_br.indicador_pl_lp(df_fechamento=df_radl_fechamento_anual, tipo_acao='on', df_lpa=lpa_radl)[0]

# L/P
lp_radl = f_br.indicador_pl_lp(df_fechamento=df_radl_fechamento_anual, tipo_acao='on', df_lpa=lpa_radl)[1]

# EBITDA
ebitda_radl = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=5258, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_radl = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=5258, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_radl = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=5258, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_radl = f_br.indicador_divida_liquida(df_caixa_total=caixa_radl, df_divida_bruta=divida_bruta_radl)

# DL/EBITDA
dl_ebitda_radl = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_radl, df_ebitda=ebitda_radl)

# DL/PL
dl_pl_radl = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=5258, cod_pl='2.03', df_divida_liquida=divida_liquida_radl)

# EV/EBITDA
ev_ebitda_radl = f_br.indicador_ev_ebitda(df_vm=vm_radl, df_divida_liquida=divida_liquida_radl, df_ebitda=ebitda_radl)

# VPA
vpa_radl = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_radl, cod_cvm=5258, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_radl = f_br.indicador_p_vpa(df_fechamento=df_radl_fechamento_anual, tipo_acao='on', df_vpa=vpa_radl)

# Margem líquida
ml_radl = f_br.indicador_ml(df_dre=dre, cod_cvm=5258, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_radl = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=5258, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_radl = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=5258, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_radl)

# Dividendos
proventos_radl = f_br.indicador_proventos(df_dva=dva, cod_cvm=5258, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_radl = f_br.indicador_payout(df_dre=dre, cod_cvm=5258, df_proventos=proventos_radl, cod_ll='3.11')

# Buyback
buyback_radl = f_br.indicador_buyback(df_dfc=dfc, cod_cvm=5258, cod_buyback='6.03.03')

# FCO
fco_radl = f_br.indicador_fco(df_dfc=dfc, cod_cvm=5258, cod_fco='6.01')

# FCI
fci_radl = f_br.indicador_fci(df_dfc=dfc, cod_cvm=5258, cod_fci='6.02')

# FCF
fcf_radl = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=5258, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.03', # 'Aquisições de Imobilizado e Intangível'
]

capex_radl = f_br.indicador_capex(df_dfc=dfc, cod_cvm=5258, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_radl = f_br.indicador_free_cash_flow(df_fco=fco_radl, df_capex=capex_radl)

# Net capex
net_capex_radl = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=5258, cod_depreciacao='6.01.01.02', df_capex=capex_radl)

# R&D
dict_rd_radl = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
}

rd_radl = f_br.dict_to_polars_df(dict_rd_radl)

# Adjusted net capex
adjusted_net_capex_radl = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_radl, df_rd=rd_radl)

# Working capital
working_capital_radl = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=5258,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_radl = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=5258, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_radl = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_radl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_radl
)

# Effective tax rate
effective_tax_rate_radl = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=5258, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_radl = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=5258, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_radl, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Empréstimos e Financiamentos Tomados'
]

new_borrowing_radl = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=5258, lst_cod_new_borrowing=lst_cod_new_borrowing)

# Debt paid
lst_cod_debt_paid = [
    '6.03.02', # 'Pagamentos de Empréstimos e Financiamentos'
    #'6.03.05', # 'Pagamentos de Arrendamentos'
]

debt_paid_radl = f_br.indicador_debt_paid(df_dfc=dfc, cod_cvm=5258, lst_cod_debt_paid=lst_cod_debt_paid)

# FCFE
fcfe_radl = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=5258,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_radl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_radl,
    df_new_borrowing=new_borrowing_radl,
    df_debt_paid=debt_paid_radl,
)

# FCFF
fcff_radl = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_radl,
    df_capex=capex_radl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_radl,
)

In [ ]:
# Lista dos df
lst_df_radl = [
    vm_radl,
    lpa_radl,
    pl_radl,
    lp_radl,
    ebitda_radl,
    divida_bruta_radl,
    caixa_radl,
    divida_liquida_radl,
    dl_ebitda_radl,
    dl_pl_radl,
    ev_ebitda_radl,
    vpa_radl,
    p_vp_radl,
    ml_radl,
    roe_radl,
    roic_radl,
    proventos_radl,
    payout_radl,
    buyback_radl,
    fco_radl,
    fci_radl,
    fcf_radl,
    capex_radl,
    free_cash_flow_radl,
    net_capex_radl,
    rd_radl,
    adjusted_net_capex_radl,
    working_capital_radl,
    change_in_non_cash_wc_radl,
    reinvestment_rate_radl,
    effective_tax_rate_radl,
    fcfe_radl,
    fcff_radl
]

# Juntando os dfs
df_indicadores_radl = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_radl
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_radl = df_indicadores_radl.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_radl

In [ ]:
# Transformando em um arquivo csv
df_indicadores_radl.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//farmacia//radl//radl_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - Buyback no dicionário 'dict_buyback_trimestral_radl'.
    - P&D no dicionário 'dict_rd_trimestral_radl'.

In [ ]:
# Composição de capital - RADL
df_composicao_capital_trimestral_radl = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('RAIADROGASIL')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - RADL
df_radl_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='RADL3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_radl['dt_refer']
)

print(df_composicao_capital_trimestral_radl.tail())
print('=' * 80)
print(df_radl_fechamento_trimestral.tail())

In [ ]:
# RADL3

# Valor de mercado
vm_trimestral_radl = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_radl[3:],
    df_fechamento=df_radl_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_radl = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_radl,
    df_dre=dre_trimestral,
    cod_cvm=5258,
    cod_ll='3.11',
    novo_mercado=True
)[0]

lpa_trimestral_acum_radl = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_radl,
    df_dre=dre_trimestral,
    cod_cvm=5258,
    cod_ll='3.11',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_radl = f_br.indicador_pl_lp(
    df_fechamento=df_radl_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_radl
)[0]

# L/P
lp_trimestral_radl = f_br.indicador_pl_lp(
    df_fechamento=df_radl_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_radl
)[1]

# EBITDA
ebitda_trimestral_radl = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_radl = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_radl = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=5258,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_radl = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=5258,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_radl = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_radl,
    df_divida_bruta=divida_bruta_trimestral_radl
)

# DL/EBITDA
dl_ebitda_trimestral_radl = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_radl,
    df_ebitda=ebitda_trimestral_acum_radl
)

# DL/PL
dl_pl_trimestral_radl = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=5258,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_radl
)

# EV/EBITDA
ev_ebitda_trimestral_radl = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_radl,
    df_divida_liquida=divida_liquida_trimestral_radl,
    df_ebitda=ebitda_trimestral_acum_radl
)

# VPA
vpa_trimestral_radl = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_radl[3:],
    cod_cvm=5258,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_radl = f_br.indicador_p_vpa(
    df_fechamento=df_radl_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_radl
)

# Margem líquida
ml_trimestral_radl = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=5258,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_radl = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=5258,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_radl = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_radl
)

# Proventos
proventos_trimestral_radl = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=5258,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_radl = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=5258,
    df_proventos=proventos_trimestral_radl,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_radl = {
    'dt_refer':[
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2020
        0, 0, 0,
        # 2021
        0, 0, 73_228,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

buyback_trimestral_radl = f_br.dict_to_polars_df(dict_buyback_trimestral_radl)

# FCO
fco_trimestral_radl = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_radl = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_radl = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_radl = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_radl = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_radl = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisições de Imobilizado e Intangível'
]

capex_trimestral_radl = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_radl = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_radl = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_radl,
    df_capex=capex_trimestral_acum_radl
)

# Net capex
net_capex_trimestral_radl = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_radl
)

# R&D
dict_rd_trimestral_radl = {
    'dt_refer':[
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2020
        0, 0, 0,
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

rd_trimestral_radl = f_br.dict_to_polars_df(dict_rd_trimestral_radl)

# Calculando o R&D acumulado
expr_rd_acum = (
    pl.col('rd')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
)

rd_trimestral_acum_radl = rd_trimestral_radl.with_columns([expr_rd_acum]).select('dt_refer', 'rd').fill_null(0)[3:]

# Adjusted net capex
adjusted_net_capex_trimestral_radl = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_radl,
    df_rd=rd_trimestral_acum_radl
)

# Working capital
working_capital_trimestral_radl = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=5258,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_radl = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_radl = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_radl = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_radl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_radl
)

# Effective tax rate
effective_tax_rate_trimestral_radl = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=5258,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_radl = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=5258,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_radl,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Empréstimos e Financiamentos Tomados'
]

new_borrowing_trimestral_radl = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamentos de Empréstimos e Financiamentos' - '6.03.02';
    - 'debt_paid_item_2' - 'Pagamentos de arrendamento' - código varia durante os anos.
* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.
* No item 'debt_paid_item_2', adicionar manualmente os dados.

In [ ]:
# Debt paid

# 'Pagamentos de Empréstimos e Financiamentos'
debt_paid_item_1 = f_br.indicador_debt_paid_trimestral(df_dfc=dfc_trimestral, cod_cvm=5258, lst_cod_debt_paid=['6.03.02'])[3:]
debt_paid_item_1 = debt_paid_item_1.rename({'debt_paid':'vl_conta_1_acum'})

################################################################
# DFC da RADL
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 5258)

################################################################
# 'Pagamentos de arrendamento'
# '6.03.04' (03/20, 06/20, 09/20, 09/21, 09/22, 09/23, 09/24 e 09/25)
# '6.03.05' (03/21, 06/21, 03/22, 06/22, 03/23, 06/23, 03/24, 06/24, 03/25 e 06/25)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        # 2020
        -159_825, -239_544, -341_420,
        # 2021
        -149_778, -320_440, -488_283,
        # 2022
        -181_472, -360_627, -589_528,
        # 2023
        -205_697, -408_402, -612_756,
        # 2024
        -204_698, -412_748, -639_121,
        # 2025
        -200_343, -429_479, -673_926,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2,
]
debt_paid_trimestral_radl = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_radl = debt_paid_trimestral_radl.with_columns(
    ((pl.col('vl_conta_1_acum') + pl.col('vl_conta_2_acum')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_trimestral_radl = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_radl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_radl,
    df_new_borrowing=new_borrowing_trimestral_radl ,
    df_debt_paid=debt_paid_trimestral_radl,
)

# FCFF
fcff_trimestral_radl = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5258,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_radl,
    df_capex=capex_trimestral_acum_radl,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_radl,
)

In [ ]:
# Lista dos df
lst_df_trimestral_radl = [
    vm_trimestral_radl,
    lpa_trimestral_radl,
    lpa_trimestral_acum_radl.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_radl,
    lp_trimestral_radl,
    ebitda_trimestral_radl,
    ebitda_trimestral_acum_radl.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_radl,
    caixa_trimestral_radl,
    divida_liquida_trimestral_radl,
    dl_ebitda_trimestral_radl,
    dl_pl_trimestral_radl,
    ev_ebitda_trimestral_radl,
    vpa_trimestral_radl,
    p_vp_trimestral_radl,
    ml_trimestral_radl,
    roe_trimestral_radl,
    roic_trimestral_radl,
    proventos_trimestral_radl,
    payout_trimestral_radl,
    buyback_trimestral_radl,
    fco_trimestral_radl,
    fci_trimestral_radl,
    fcf_trimestral_radl,
    fco_trimestral_acum_radl.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_radl.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_radl.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_radl,
    capex_trimestral_acum_radl.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_radl,
    net_capex_trimestral_radl,
    rd_trimestral_radl,
    rd_trimestral_acum_radl.rename({'rd': 'rd_acum'}),
    adjusted_net_capex_trimestral_radl,
    working_capital_trimestral_radl,
    change_in_non_cash_wc_trimestral_radl,
    change_in_non_cash_wc_trimestral_acum_radl.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_radl,
    effective_tax_rate_trimestral_radl,
    fcfe_trimestral_radl,
    fcff_trimestral_radl
]

# Juntando os dfs
df_indicadores_trimestral_radl = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_radl
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_radl = df_indicadores_trimestral_radl.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_radl

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_radl.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//farmacia//radl//radl_indicadores_trimestral.csv'),
    separator=';'
)

# **Setor de Máquinas e Equipamentos**

## WEGE3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_wege'.

In [ ]:
# Número de ações (2017 a 2019) - WEGE
dict_composicao_capital_17_18_19_wege = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        1_614_353_076,
        2_098_658_999,
        2_098_658_999,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_wege = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_wege)

# Composição de capital (2020 a 2024) - WEGE
composicao_capital_wege = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('WEG')).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_wege = pl.concat([df_composicao_capital_17_18_19_wege, composicao_capital_wege])

##############################################################################
# Preços de fechamento - WEGE
df_wege_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='WEGE3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_wege.tail())
print('=' * 80)
print(df_wege_fechamento_anual.tail())

In [ ]:
# WEGE3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='WEGE3.SA', ticker_pn=None)

# Valor de mercado
vm_wege = f_br.indicador_vm(df_capital_social=df_composicao_capital_wege, df_fechamento=df_wege_fechamento_anual, novo_mercado=True)

# LPA
lpa_wege = f_br.indicador_lpa(df_capital_social=df_composicao_capital_wege, df_dre=dre, cod_cvm=5410, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_wege = f_br.indicador_pl_lp(df_fechamento=df_wege_fechamento_anual, tipo_acao='on', df_lpa=lpa_wege)[0]

# L/P
lp_wege = f_br.indicador_pl_lp(df_fechamento=df_wege_fechamento_anual, tipo_acao='on', df_lpa=lpa_wege)[1]

# EBITDA
ebitda_wege = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=5410, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_wege = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=5410, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_wege = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=5410, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_wege = f_br.indicador_divida_liquida(df_caixa_total=caixa_wege, df_divida_bruta=divida_bruta_wege)

# DL/EBITDA
dl_ebitda_wege = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_wege, df_ebitda=ebitda_wege)

# DL/PL
dl_pl_wege = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=5410, cod_pl='2.03', df_divida_liquida=divida_liquida_wege)

# EV/EBITDA
ev_ebitda_wege = f_br.indicador_ev_ebitda(df_vm=vm_wege, df_divida_liquida=divida_liquida_wege, df_ebitda=ebitda_wege)

# VPA
vpa_wege = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_wege, cod_cvm=5410, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_wege = f_br.indicador_p_vpa(df_fechamento=df_wege_fechamento_anual, tipo_acao='on', df_vpa=vpa_wege)

# Margem líquida
ml_wege = f_br.indicador_ml(df_dre=dre, cod_cvm=5410, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_wege = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=5410, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_wege = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=5410, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_wege)

# Dividendos
proventos_wege = f_br.indicador_proventos(df_dva=dva, cod_cvm=5410, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_wege = f_br.indicador_payout(df_dre=dre, cod_cvm=5410, df_proventos=proventos_wege, cod_ll='3.11')

# Buyback
buyback_wege = f_br.indicador_buyback(df_dfc=dfc, cod_cvm=5410, cod_buyback='6.03.05')

# FCO
fco_wege = f_br.indicador_fco(df_dfc=dfc, cod_cvm=5410, cod_fco='6.01')

# FCI
fci_wege = f_br.indicador_fci(df_dfc=dfc, cod_cvm=5410, cod_fci='6.02')

# FCF
fcf_wege = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=5410, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.02', # 'Imobilizado'
    '6.02.03', # 'Intangível'
    '6.02.09'  # 'Aquisição de empresa - combinação de negócio (líquido do caixa)'
]

capex_wege = f_br.indicador_capex(df_dfc=dfc, cod_cvm=5410, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_wege = f_br.indicador_free_cash_flow(df_fco=fco_wege, df_capex=capex_wege)

# Net capex
net_capex_wege = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=5410, cod_depreciacao='6.01.01.02', df_capex=capex_wege)

# R&D
dict_rd_wege = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        259_300,
        307_600,
        339_300,
        468_900,
        548_800,
        646_900,
        832_500,
        1_082_500,
    ]
}

rd_wege = f_br.dict_to_polars_df(dict_rd_wege)

# Adjusted net capex
adjusted_net_capex_wege = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_wege, df_rd=rd_wege)

# Working capital
working_capital_wege = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=5410,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_wege = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=5410, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_wege = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_wege,
    df_change_in_non_cash_wc=change_in_non_cash_wc_wege
)

# Effective tax rate
effective_tax_rate_wege = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=5410, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_wege = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=5410, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_wege, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Captação de empréstimos e financiamentos obtidos'
]

new_borrowing_wege = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=5410, lst_cod_new_borrowing=lst_cod_new_borrowing)

# Debt paid
lst_cod_debt_paid = [
    '6.03.02' # 'Pagamento de empréstimos e financiamentos'
]

debt_paid_wege = f_br.indicador_debt_paid(df_dfc=dfc, cod_cvm=5410, lst_cod_debt_paid=lst_cod_debt_paid)

# FCFE
fcfe_wege = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=5410,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_wege,
    df_change_in_non_cash_wc=change_in_non_cash_wc_wege,
    df_new_borrowing=new_borrowing_wege,
    df_debt_paid=debt_paid_wege,
)

# FCFF
fcff_wege = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_wege,
    df_capex=capex_wege,
    df_change_in_non_cash_wc=change_in_non_cash_wc_wege,
)

In [ ]:
# Lista dos df
lst_df_wege = [
    vm_wege,
    lpa_wege,
    pl_wege,
    lp_wege,
    ebitda_wege,
    divida_bruta_wege,
    caixa_wege,
    divida_liquida_wege,
    dl_ebitda_wege,
    dl_pl_wege,
    ev_ebitda_wege,
    vpa_wege,
    p_vp_wege,
    ml_wege,
    roe_wege,
    roic_wege,
    proventos_wege,
    payout_wege,
    buyback_wege,
    fco_wege,
    fci_wege,
    fcf_wege,
    capex_wege,
    free_cash_flow_wege,
    net_capex_wege,
    rd_wege,
    adjusted_net_capex_wege,
    working_capital_wege,
    change_in_non_cash_wc_wege,
    reinvestment_rate_wege,
    effective_tax_rate_wege,
    fcfe_wege,
    fcff_wege
]

# Juntando os dfs
df_indicadores_wege = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_wege
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_wege = df_indicadores_wege.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_wege

In [ ]:
# Transformando em um arquivo csv
df_indicadores_wege.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//maq-equip//wege//wege_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_trimestral_wege'.

In [ ]:
# Composição de capital - WEGE
df_composicao_capital_trimestral_wege = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('WEG')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - WEGE
df_wege_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='WEGE3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_wege['dt_refer']
)

print(df_composicao_capital_trimestral_wege.tail())
print('=' * 80)
print(df_wege_fechamento_trimestral.tail())

In [ ]:
# WEGE3

# Valor de mercado
vm_trimestral_wege = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_wege[3:],
    df_fechamento=df_wege_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_wege = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_wege,
    df_dre=dre_trimestral,
    cod_cvm=5410,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_wege = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_wege,
    df_dre=dre_trimestral,
    cod_cvm=5410,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_wege = f_br.indicador_pl_lp(
    df_fechamento=df_wege_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_wege
)[0]

# L/P
lp_trimestral_wege = f_br.indicador_pl_lp(
    df_fechamento=df_wege_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_wege
)[1]

# EBITDA
ebitda_trimestral_wege = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_wege = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_wege = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=5410,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_wege = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=5410,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_wege = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_wege,
    df_divida_bruta=divida_bruta_trimestral_wege
)

# DL/EBITDA
dl_ebitda_trimestral_wege = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_wege,
    df_ebitda=ebitda_trimestral_acum_wege
)

# DL/PL
dl_pl_trimestral_wege = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=5410,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_wege
)

# EV/EBITDA
ev_ebitda_trimestral_wege = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_wege,
    df_divida_liquida=divida_liquida_trimestral_wege,
    df_ebitda=ebitda_trimestral_acum_wege
)

# VPA
vpa_trimestral_wege = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_wege[3:],
    cod_cvm=5410,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_wege = f_br.indicador_p_vpa(
    df_fechamento=df_wege_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_wege
)

# Margem líquida
ml_trimestral_wege = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=5410,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_wege = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=5410,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_wege = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_wege
)

# Proventos
proventos_trimestral_wege = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=5410,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_wege = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=5410,
    df_proventos=proventos_trimestral_wege,
    cod_ll='3.11'
)

# Buyback
buyback_trimestral_wege = f_br.indicador_buyback_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_buyback='6.03.05'
)

# FCO
fco_trimestral_wege = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_wege = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_wege = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_wege = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_wege = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_wege = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Imobilizado'
    '6.02.02', # 'Intangível'
]

capex_trimestral_wege = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_wege = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_wege = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_wege,
    df_capex=capex_trimestral_acum_wege
)

# Net capex
net_capex_trimestral_wege = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_wege
)

# R&D - esse dado está no arquivo 'release de resultados'
dict_rd_trimestral_wege = {
    'dt_refer':[
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2020
        98_200,
        76_000,  # (174200 - 98200)
        147_900, # (322100 - (174200 - 98200) - 98200)
        # 2021
        142_400,
        166_300,
        149_300,
        # 2022
        157_300,
        146_500,
        193_900,
        # 2023
        223_400,
        204_300, # (427700 - 223400)
        216_700, # (644400 - (427700 - 223400) - 223400)
        # 2024
        272_800,
        255_400, # (528200 - 272800)
        270_800, # (799000 - (528200 - 272800) - 272800)
        # 2025
        312_200,
        361_600, # (673800 - 312200)
        394_300, # (1068100 - (673800 - 312200) - 312200)
    ]
}

rd_trimestral_wege = f_br.dict_to_polars_df(dict_rd_trimestral_wege)

# Calculando o R&D acumulado
expr_rd_acum = (
    pl.col('rd')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
)

rd_trimestral_acum_wege = rd_trimestral_wege.with_columns([expr_rd_acum]).select('dt_refer', 'rd').fill_null(0)[3:]

# Adjusted net capex
adjusted_net_capex_trimestral_wege = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_wege,
    df_rd=rd_trimestral_acum_wege
)

# Working capital
working_capital_trimestral_wege = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=5410,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_wege = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_wege = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_wege = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_wege,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_wege
)

# Effective tax rate
effective_tax_rate_trimestral_wege = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=5410,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_wege = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=5410,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_wege,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Captação de empréstimos e financiamentos obtidos'
]

new_borrowing_trimestral_wege = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

# Debt paid
lst_cod_debt_paid = [
    '6.03.02' # 'Pagamento de empréstimos e financiamentos'
]

debt_paid_trimestral_wege = f_br.indicador_debt_paid_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    lst_cod_debt_paid=lst_cod_debt_paid
)[3:]

# FCFE
fcfe_trimestral_wege = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_wege,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_wege,
    df_new_borrowing=new_borrowing_trimestral_wege ,
    df_debt_paid=debt_paid_trimestral_wege,
)

# FCFF
fcff_trimestral_wege = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=5410,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_wege,
    df_capex=capex_trimestral_acum_wege,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_wege,
)

In [ ]:
# Lista dos df
lst_df_trimestral_wege = [
    vm_trimestral_wege,
    lpa_trimestral_wege,
    lpa_trimestral_acum_wege.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_wege,
    lp_trimestral_wege,
    ebitda_trimestral_wege,
    ebitda_trimestral_acum_wege.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_wege,
    caixa_trimestral_wege,
    divida_liquida_trimestral_wege,
    dl_ebitda_trimestral_wege,
    dl_pl_trimestral_wege,
    ev_ebitda_trimestral_wege,
    vpa_trimestral_wege,
    p_vp_trimestral_wege,
    ml_trimestral_wege,
    roe_trimestral_wege,
    roic_trimestral_wege,
    proventos_trimestral_wege,
    payout_trimestral_wege,
    buyback_trimestral_wege,
    fco_trimestral_wege,
    fci_trimestral_wege,
    fcf_trimestral_wege,
    fco_trimestral_acum_wege.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_wege.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_wege.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_wege,
    capex_trimestral_acum_wege.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_wege,
    net_capex_trimestral_wege,
    rd_trimestral_wege,
    rd_trimestral_acum_wege.rename({'rd': 'rd_acum'}),
    adjusted_net_capex_trimestral_wege,
    working_capital_trimestral_wege,
    change_in_non_cash_wc_trimestral_wege,
    change_in_non_cash_wc_trimestral_acum_wege.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_wege,
    effective_tax_rate_trimestral_wege,
    fcfe_trimestral_wege,
    fcff_trimestral_wege
]

# Juntando os dfs
df_indicadores_trimestral_wege = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_wege
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_wege = df_indicadores_trimestral_wege.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_wege

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_wege.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//maq-equip//wege//wege_indicadores_trimestral.csv'),
    separator=';'
)

# **Setor de Material Aeronáutico e de Defesa**

## EMBJ3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_embj'.

In [ ]:
# Número de ações (2017 a 2019) - EMBJ
dict_composicao_capital_17_18_19_embj = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        740_465_000,
        740_465_000,
        740_465_000,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_embj = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_embj)

# Composição de capital (2020 a 2024) - EMBJ
composicao_capital_embj = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('EMBRAER')).select('dt_refer', 'num_on')

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Arrumando a escala dos anos específicos
composicao_capital_embj = composicao_capital_embj.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on')
]).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_embj = pl.concat([df_composicao_capital_17_18_19_embj, composicao_capital_embj])

##############################################################################
# Preços de fechamento - EMBJ
# Baixei os dados históricos no Investing, porque no yfinance não tem os dados completos, porque mudou de ticker (EMBR3 -> EMBJ3)
df_embj_fechamento_anual = f_br.ativo_fechamento_anual_csv(
    path_csv_on_unit='C://B3//historico-arquivos//precos-historicos//embj3_2017_2025.csv',
    path_csv_pn=None,
    ticker_on_unit='EMBJ3',
    primeiro_ano=2017,
    ultimo_ano=2024,
)

print(df_composicao_capital_embj.tail())
print('=' * 80)
print(df_embj_fechamento_anual.tail())

In [ ]:
# EMBJ3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='EMBJ3.SA', ticker_pn=None)

# Valor de mercado
vm_embj = f_br.indicador_vm(df_capital_social=df_composicao_capital_embj, df_fechamento=df_embj_fechamento_anual, novo_mercado=True)

# LPA
lpa_embj = f_br.indicador_lpa(df_capital_social=df_composicao_capital_embj, df_dre=dre, cod_cvm=20087, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_embj = f_br.indicador_pl_lp(df_fechamento=df_embj_fechamento_anual, tipo_acao='on', df_lpa=lpa_embj)[0]

# L/P
lp_embj = f_br.indicador_pl_lp(df_fechamento=df_embj_fechamento_anual, tipo_acao='on', df_lpa=lpa_embj)[1]

# EBITDA
ebitda_embj = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=20087, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_embj = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=20087, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_embj = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=20087, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_embj = f_br.indicador_divida_liquida(df_caixa_total=caixa_embj, df_divida_bruta=divida_bruta_embj)

# DL/EBITDA
dl_ebitda_embj = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_embj, df_ebitda=ebitda_embj)

# DL/PL
dl_pl_embj = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=20087, cod_pl='2.03', df_divida_liquida=divida_liquida_embj)

# EV/EBITDA
ev_ebitda_embj = f_br.indicador_ev_ebitda(df_vm=vm_embj, df_divida_liquida=divida_liquida_embj, df_ebitda=ebitda_embj)

# VPA
vpa_embj = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_embj, cod_cvm=20087, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_embj = f_br.indicador_p_vpa(df_fechamento=df_embj_fechamento_anual, tipo_acao='on', df_vpa=vpa_embj)

# Margem líquida
ml_embj = f_br.indicador_ml(df_dre=dre, cod_cvm=20087, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_embj = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=20087, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_embj = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=20087, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_embj)

# Dividendos
proventos_embj = f_br.indicador_proventos(df_dva=dva, cod_cvm=20087, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_embj = f_br.indicador_payout(df_dre=dre, cod_cvm=20087, df_proventos=proventos_embj, cod_ll='3.11')

# Buyback
dict_buyback_embj = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback':[
        48_395,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
}

buyback_embj = f_br.dict_to_polars_df(dict_buyback_embj)

# FCO
fco_embj = f_br.indicador_fco(df_dfc=dfc, cod_cvm=20087, cod_fco='6.01')

# FCI
fci_embj = f_br.indicador_fci(df_dfc=dfc, cod_cvm=20087, cod_fci='6.02')

# FCF
fcf_embj = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=20087, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.03', # 'Adições ao intangível'
    '6.02.04'  # 'Adições investimentos em subsidiárias e coligadas, líquido do caixa adquirido'
]

capex_embj_1 = f_br.indicador_capex(df_dfc=dfc, cod_cvm=20087, lst_cod_capex=lst_cod_capex)
capex_embj_1 = capex_embj_1.rename({'capex':'vl_conta_1'})

# O item 'Aquisições de imobilizado' está com dados errados em 2018 e 2017
capex_embj_2 = f_br.indicador_capex(df_dfc=dfc, cod_cvm=20087, lst_cod_capex=['6.02.01'])

capex_embj_2 = capex_embj_2.with_columns(
    pl.when(pl.col('dt_refer') == pl.datetime(2017, 12, 31))
      .then(762836)
      .when(pl.col('dt_refer') == pl.datetime(2018, 12, 31))
      .then(565126)
      .otherwise(pl.col('capex'))
      .alias('capex')
)

capex_embj_2 = capex_embj_2.rename({'capex':'vl_conta_2'})

capex_embj = capex_embj_1.join(capex_embj_2, on='dt_refer', how='inner')

capex_embj = capex_embj.with_columns(
    (pl.col('vl_conta_1') + pl.col('vl_conta_2')).alias('capex')
).select('dt_refer', 'capex')

# Free cash flow
free_cash_flow_embj = f_br.indicador_free_cash_flow(df_fco=fco_embj, df_capex=capex_embj)

# Net capex
net_capex_embj = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=20087, cod_depreciacao='6.01.01.02', df_capex=capex_embj)

# R&D
rd_embj = f_br.indicador_rd(df_dre=dre, cod_cvm=20087, cod_rd='3.04.05.01', anual=True)

# Adjusted net capex
adjusted_net_capex_embj = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_embj, df_rd=rd_embj)

# Working capital
working_capital_embj = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=20087,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_embj = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=20087, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_embj = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_embj,
    df_change_in_non_cash_wc=change_in_non_cash_wc_embj
)

# Effective tax rate
effective_tax_rate_embj = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=20087, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_embj = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=20087, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_embj, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Empréstimos obtidos'
]

new_borrowing_embj = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=20087, lst_cod_new_borrowing=lst_cod_new_borrowing)

# O item 'Empréstimos obtidos' está com dados errados em 2018 e 2017
new_borrowing_embj = new_borrowing_embj.with_columns(
    pl.when(pl.col('dt_refer') == pl.datetime(2017, 12, 31))
      .then(3036788)
      .when(pl.col('dt_refer') == pl.datetime(2018, 12, 31))
      .then(438197)
      .otherwise(pl.col('new_borrowing'))
      .alias('new_borrowing')
)

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Financiamentos pagos' - '6.03.02';
    - 'debt_paid_item_2' - 'Pagamentos de arrendamentos' - código varia durante os anos.

* No item 'debt_paid_item_2', adicionar manualmente os dados.

In [ ]:
# Debt paid

# 'Financiamentos pagos'
debt_paid_item_1 = f_br.indicador_debt_paid(df_dfc=dfc, cod_cvm=20087, lst_cod_debt_paid=['6.03.02'])

# O item 'Financiamentos pagos' está com dados errados em 2018 e 2017
debt_paid_item_1 = debt_paid_item_1.with_columns(
    pl.when(pl.col('dt_refer') == pl.datetime(2017, 12, 31))
      .then(-1730456)
      .when(pl.col('dt_refer') == pl.datetime(2018, 12, 31))
      .then(-2219084)
      .otherwise(pl.col('debt_paid'))
      .alias('debt_paid')
)

debt_paid_item_1 = debt_paid_item_1.rename({'debt_paid':'vl_conta_1'})

################################################################
# DFC da EMBJ
dfc_embj = dfc.filter(pl.col('cd_cvm') == 20087)

################################################################
# 'Pagamentos de arrendamentos'
# '6.03.07' (2023 e 2024)
# '6.03.06' (2019, 2020, 2021 e 2022)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'vl_conta_2': [
        0,
        0,
        -47_279,
        -46_494,
        -56_069,
        -60_874,
        -68_247,
        -102_719,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_item_1, debt_paid_item_2]
debt_paid_embj = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_embj = debt_paid_embj.with_columns(
    ((pl.col('vl_conta_1') + pl.col('vl_conta_2')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_embj = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=20087,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_embj,
    df_change_in_non_cash_wc=change_in_non_cash_wc_embj,
    df_new_borrowing=new_borrowing_embj,
    df_debt_paid=debt_paid_embj,
)

# FCFF
fcff_embj = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_embj,
    df_capex=capex_embj,
    df_change_in_non_cash_wc=change_in_non_cash_wc_embj,
)

In [ ]:
# Lista dos df
lst_df_embj = [
    vm_embj,
    lpa_embj,
    pl_embj,
    lp_embj,
    ebitda_embj,
    divida_bruta_embj,
    caixa_embj,
    divida_liquida_embj,
    dl_ebitda_embj,
    dl_pl_embj,
    ev_ebitda_embj,
    vpa_embj,
    p_vp_embj,
    ml_embj,
    roe_embj,
    roic_embj,
    proventos_embj,
    payout_embj,
    buyback_embj,
    fco_embj,
    fci_embj,
    fcf_embj,
    capex_embj,
    free_cash_flow_embj,
    net_capex_embj,
    rd_embj,
    adjusted_net_capex_embj,
    working_capital_embj,
    change_in_non_cash_wc_embj,
    reinvestment_rate_embj,
    effective_tax_rate_embj,
    fcfe_embj,
    fcff_embj
]

# Juntando os dfs
df_indicadores_embj = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_embj
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_embj = df_indicadores_embj.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_embj

In [ ]:
# Transformando em um arquivo csv
df_indicadores_embj.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//material-aeronautico//embj//embj_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - buyback no dicionário 'dict_buyback_trimestral_embj'.

In [ ]:
# Composição de capital - EMBJ
df_composicao_capital_trimestral_embj = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('EMBRAER')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - EMBJ
# Baixei os dados históricos no Investing, porque no yfinance não tem os dados completos, porque mudou de ticker (EMBR3 -> EMBJ3)
df_embj_fechamento_trimestral = f_br.ativo_fechamento_trimestral_csv(
    ticker_on_unit='EMBJ3',
    list_ticker_on_pn=None,
    path_csv_on_unit='C://B3//historico-arquivos//precos-historicos//embj3_2017_2025.csv',
    path_csv_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_embj['dt_refer']
)

print(df_composicao_capital_trimestral_embj.tail())
print('=' * 80)
print(df_embj_fechamento_trimestral.tail())


In [ ]:
# EMBJ3

# Valor de mercado
vm_trimestral_embj = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_embj[3:],
    df_fechamento=df_embj_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_embj = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_embj,
    df_dre=dre_trimestral,
    cod_cvm=20087,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_embj = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_embj,
    df_dre=dre_trimestral,
    cod_cvm=20087,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_embj = f_br.indicador_pl_lp(
    df_fechamento=df_embj_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_embj
)[0]

# L/P
lp_trimestral_embj = f_br.indicador_pl_lp(
    df_fechamento=df_embj_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_embj
)[1]

# EBITDA
ebitda_trimestral_embj = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_embj = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_embj = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=20087,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_embj = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=20087,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_embj = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_embj,
    df_divida_bruta=divida_bruta_trimestral_embj
)

# DL/EBITDA
dl_ebitda_trimestral_embj = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_embj,
    df_ebitda=ebitda_trimestral_acum_embj
)

# DL/PL
dl_pl_trimestral_embj = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=20087,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_embj
)

# EV/EBITDA
ev_ebitda_trimestral_embj = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_embj,
    df_divida_liquida=divida_liquida_trimestral_embj,
    df_ebitda=ebitda_trimestral_acum_embj
)

# VPA
vpa_trimestral_embj = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_embj[3:],
    cod_cvm=20087,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_embj = f_br.indicador_p_vpa(
    df_fechamento=df_embj_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_embj
)

# Margem líquida
ml_trimestral_embj = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=20087,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_embj = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=20087,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_embj = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_embj
)

# Proventos
proventos_trimestral_embj = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=20087,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_embj = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=20087,
    df_proventos=proventos_trimestral_embj,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_embj = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'buyback': [
        # 2020
        0, 0, 0,
        # 2021
        13_479, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        82_212, 0, 0,
    ]
}

buyback_trimestral_embj = f_br.dict_to_polars_df(dict_buyback_trimestral_embj)

# FCO
fco_trimestral_embj = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_embj = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_embj = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_embj = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_embj = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_embj = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisições de imobilizado'
    '6.02.03', # 'Adições ao intangível'
    '6.02.04', # 'Adições investimentos em subsidiárias e coligadas, líquido do caixa adquirido'
]

capex_trimestral_embj = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_embj = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_embj = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_embj,
    df_capex=capex_trimestral_acum_embj
)

# Net capex
net_capex_trimestral_embj = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_embj
)

# R&D
rd_trimestral_embj = f_br.indicador_rd(df_dre=dre, cod_cvm=20087, cod_rd='3.04.05.01', anual=False)

# Adjusted net capex
adjusted_net_capex_trimestral_embj = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_embj,
    df_rd=rd_trimestral_embj
)

# Working capital
working_capital_trimestral_embj = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=20087,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_embj = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_embj = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_embj = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_embj,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_embj
)

# Effective tax rate
effective_tax_rate_trimestral_embj = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=20087,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_embj = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=20087,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_embj,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Novos financiamentos obtidos'
]

new_borrowing_trimestral_embj = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Financiamentos pagos' - '6.03.02';
    - 'debt_paid_item_2' - 'Pagamentos de arrendamentos' - código varia durante os anos.
* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho;
* No item 'debt_paid_item_2', adicionar manualmente os dados.

In [ ]:
# Debt paid

# DFC da EMBJ
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 20087)

################################################################
# 'Financiamentos pagos'
DATAS_ALVO_DP_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Calculando o new borrowing não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

debt_paid_nao_acum_item_1 = debt_paid_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o new borrowing trimestral acumulado
expr_debt_paid_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

debt_paid_acum_item_1 = debt_paid_nao_acum_item_1.with_columns([expr_debt_paid_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
debt_paid_acum_item_1 = debt_paid_acum_item_1[3:]

################################################################
# 'Pagamentos de arrendamentos'
# '6.03.03' (03/24, 06/24, 09/24, 06/25 e 09/25)
# '6.03.04' (03/25)
# '6.03.06' (03/20, 06/20, 09/20, 03/21, 06/21, 09/21, 03/22, 06/22, 09/22, 03/23, 06/23 e 09/23)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        # 2020
        -8_741, -20_742,  -34_792,
        # 2021
        -13_623, -27_586, -40_997,
        # 2022
        -14_069, -30_167, -46_068,
        # 2023
        -17_610, -34_010, -49_887,
        # 2024
        -18_439, -39_985, -67_186,
        # 2025
        -33_872, -69_265, -102_694,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2,
]
debt_paid_trimestral_embj = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_embj = debt_paid_trimestral_embj.with_columns(
    ((pl.col('vl_conta_1_acum') + pl.col('vl_conta_2_acum')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_trimestral_embj = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_embj,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_embj,
    df_new_borrowing=new_borrowing_trimestral_embj,
    df_debt_paid=debt_paid_trimestral_embj,
)

# FCFF
fcff_trimestral_embj = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=20087,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_embj,
    df_capex=capex_trimestral_acum_embj,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_embj,
)

In [ ]:
# Lista dos df
lst_df_trimestral_embj = [
    vm_trimestral_embj,
    lpa_trimestral_embj,
    lpa_trimestral_acum_embj.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_embj,
    lp_trimestral_embj,
    ebitda_trimestral_embj,
    ebitda_trimestral_acum_embj.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_embj,
    caixa_trimestral_embj,
    divida_liquida_trimestral_embj,
    dl_ebitda_trimestral_embj,
    dl_pl_trimestral_embj,
    ev_ebitda_trimestral_embj,
    vpa_trimestral_embj,
    p_vp_trimestral_embj,
    ml_trimestral_embj,
    roe_trimestral_embj,
    roic_trimestral_embj,
    proventos_trimestral_embj,
    payout_trimestral_embj,
    buyback_trimestral_embj,
    fco_trimestral_embj,
    fci_trimestral_embj,
    fcf_trimestral_embj,
    fco_trimestral_acum_embj.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_embj.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_embj.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_embj,
    capex_trimestral_acum_embj.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_embj,
    net_capex_trimestral_embj,
    rd_trimestral_embj,
    adjusted_net_capex_trimestral_embj,
    working_capital_trimestral_embj,
    change_in_non_cash_wc_trimestral_embj,
    change_in_non_cash_wc_trimestral_acum_embj.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_embj,
    effective_tax_rate_trimestral_embj,
    fcfe_trimestral_embj,
    fcff_trimestral_embj
]

# Juntando os dfs
df_indicadores_trimestral_embj = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_embj
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_embj = df_indicadores_trimestral_embj.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_embj

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_embj.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//material-aeronautico//embj//embj_indicadores_trimestral.csv'),
    separator=';'
)

# **Setor de Mineração**

## VALE3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_vale'.

In [ ]:
# Número de ações (2017 a 2019) - VALE
dict_composicao_capital_17_18_19_vale = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        5_284_475_000,
        5_284_475_000,
        5_284_475_000,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_vale = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_vale)

# Composição de capital (2020 a 2024) - VALE
composicao_capital_vale = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('VALE S.A.')).select('dt_refer', 'num_on')

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Arrumando a escala dos anos específicos
composicao_capital_vale = composicao_capital_vale.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on')
]).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_vale = pl.concat([df_composicao_capital_17_18_19_vale, composicao_capital_vale])

##############################################################################
# Preços de fechamento - VALE
df_vale_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='VALE3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_vale.tail())
print('=' * 80)
print(df_vale_fechamento_anual.tail())

In [ ]:
# VALE3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='VALE3.SA', ticker_pn=None)

# Valor de mercado
vm_vale = f_br.indicador_vm(df_capital_social=df_composicao_capital_vale, df_fechamento=df_vale_fechamento_anual, novo_mercado=True)

# LPA
lpa_vale = f_br.indicador_lpa(df_capital_social=df_composicao_capital_vale, df_dre=dre, cod_cvm=4170, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_vale = f_br.indicador_pl_lp(df_fechamento=df_vale_fechamento_anual, tipo_acao='on', df_lpa=lpa_vale)[0]

# L/P
lp_vale = f_br.indicador_pl_lp(df_fechamento=df_vale_fechamento_anual, tipo_acao='on', df_lpa=lpa_vale)[1]

# EBITDA
ebitda_vale = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=4170, cod_ebit='3.05', cod_depreciacao='6.01.01.06')

# Dívida bruta
divida_bruta_vale = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=4170, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_vale = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=4170, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_vale = f_br.indicador_divida_liquida(df_caixa_total=caixa_vale, df_divida_bruta=divida_bruta_vale)

# DL/EBITDA
dl_ebitda_vale = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_vale, df_ebitda=ebitda_vale)

# DL/PL
dl_pl_vale = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=4170, cod_pl='2.03', df_divida_liquida=divida_liquida_vale)

# EV/EBITDA
ev_ebitda_vale = f_br.indicador_ev_ebitda(df_vm=vm_vale, df_divida_liquida=divida_liquida_vale, df_ebitda=ebitda_vale)

# VPA
vpa_vale = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_vale, cod_cvm=4170, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_vale = f_br.indicador_p_vpa(df_fechamento=df_vale_fechamento_anual, tipo_acao='on', df_vpa=vpa_vale)

# Margem líquida
ml_vale = f_br.indicador_ml(df_dre=dre, cod_cvm=4170, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_vale = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=4170, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_vale = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=4170, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_vale)

# Dividendos
lst_cod_proventos = [
    '6.03.04', # ' Dividendos/JCP Pagos a Acionistas'
]

proventos_vale = f_br.indicador_proventos_dfc(df_dfc=dfc, cod_cvm=4170, lst_cod_proventos=lst_cod_proventos)

# Payout
payout_vale = f_br.indicador_payout(df_dre=dre, cod_cvm=4170, df_proventos=proventos_vale, cod_ll='3.11')

# Buyback
buyback_vale = f_br.indicador_buyback(df_dfc=dfc, cod_cvm=4170, cod_buyback='6.03.06')

dict_buyback_vale = {
    'dt_refer': [
        '2017-12-31'
    ],

    'buyback': [
        0.0
    ]
}

# Dados complementares
buyback_complemento_vale = f_br.dict_to_polars_df(dict_buyback_vale)
buyback_vale = pl.concat([buyback_complemento_vale, buyback_vale])

# FCO
fco_vale = f_br.indicador_fco(df_dfc=dfc, cod_cvm=4170, cod_fco='6.01')

# FCI
fci_vale = f_br.indicador_fci(df_dfc=dfc, cod_cvm=4170, cod_fci='6.02')

# FCF
fcf_vale = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=4170, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.05', # 'Adições ao Imobilizado'
]

capex_vale = f_br.indicador_capex(df_dfc=dfc, cod_cvm=4170, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_vale = f_br.indicador_free_cash_flow(df_fco=fco_vale, df_capex=capex_vale)

# Net capex
net_capex_vale = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=4170, cod_depreciacao='6.01.01.06', df_capex=capex_vale)

# R&D - Está informação está no arquivo do DFP/ITR - na parte "Informações Contábeis" na linha "Despesas com pesquisa e desenvolvimento".
dict_rd_vale = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        1_125_000,
        1_376_000,
        1_765_000,
        2_151_000,
        2_964_000,
        3_411_000,
        3_598_000,
        4_307_000,
    ]
}

rd_vale = f_br.dict_to_polars_df(dict_rd_vale)

# Adjusted net capex
adjusted_net_capex_vale = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_vale, df_rd=rd_vale)

# Working capital
working_capital_vale = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=4170,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_vale = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=4170, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_vale = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_vale,
    df_change_in_non_cash_wc=change_in_non_cash_wc_vale
)

# Effective tax rate
effective_tax_rate_vale = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=4170, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_vale = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=4170, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_vale, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Captação de empréstimos e financiamentos obtidos'
]

new_borrowing_vale = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=4170, lst_cod_new_borrowing=lst_cod_new_borrowing)

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de empréstimos e financiamentos'' - '6.03.02';
    - 'debt_paid_item_2' - 'Arrendamentos - pagamentos' - '6.03.05'.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da VALE
dfc_vale = dfc.filter(pl.col('cd_cvm') == 4170)

################################################################
# 'Pagamento de empréstimos e financiamentos'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2017, 12, 31),
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_vale
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

################################################################
# 'Arrendamentos - pagamentos'
DATAS_ALVO_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_vale
    .filter(
        (pl.col('cd_conta') == '6.03.05') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
    ],

    'vl_conta_2': [
        0.0,
    ]
}

debt_paid_item_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_paid_item_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_item_1, debt_paid_item_2]
debt_paid_vale = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_vale = debt_paid_vale.with_columns(
    ((
        pl.col('vl_conta_1') +
        pl.col('vl_conta_2')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_vale = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=4170,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.06',
    df_capex=capex_vale,
    df_change_in_non_cash_wc=change_in_non_cash_wc_vale,
    df_new_borrowing=new_borrowing_vale,
    df_debt_paid=debt_paid_vale,
)

# FCFF
fcff_vale = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.06',
    df_taxes_on_operating_income=taxes_on_operating_income_vale,
    df_capex=capex_vale,
    df_change_in_non_cash_wc=change_in_non_cash_wc_vale,
)

In [ ]:
# Lista dos df
lst_df_vale = [
    vm_vale,
    lpa_vale,
    pl_vale,
    lp_vale,
    ebitda_vale,
    divida_bruta_vale,
    caixa_vale,
    divida_liquida_vale,
    dl_ebitda_vale,
    dl_pl_vale,
    ev_ebitda_vale,
    vpa_vale,
    p_vp_vale,
    ml_vale,
    roe_vale,
    roic_vale,
    proventos_vale,
    payout_vale,
    buyback_vale,
    fco_vale,
    fci_vale,
    fcf_vale,
    capex_vale,
    free_cash_flow_vale,
    net_capex_vale,
    rd_vale,
    adjusted_net_capex_vale,
    working_capital_vale,
    change_in_non_cash_wc_vale,
    reinvestment_rate_vale,
    effective_tax_rate_vale,
    fcfe_vale,
    fcff_vale
]

# Juntando os dfs
df_indicadores_vale = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_vale
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_vale = df_indicadores_vale.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_vale

In [ ]:
# Transformando em um arquivo csv
df_indicadores_vale.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//mineracao//vale//vale_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - Proventos no dicionário 'dict_proventos_trimestral_vale';
    - Buyback no dicionário 'dict_buyback_trimestral_vale';
    - P&D no dicionário 'dict_rd_trimestral_vale'.

In [ ]:
# Composição de capital - VALE
df_composicao_capital_trimestral_vale = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('VALE S.A.')).select('dt_refer', 'num_on')

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),
]

# Arrumando a escala dos anos específicos
df_composicao_capital_trimestral_vale = df_composicao_capital_trimestral_vale.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on')

]).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - VALE
df_vale_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='VALE3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_vale['dt_refer']
)

print(df_composicao_capital_trimestral_vale.tail())
print('=' * 80)
print(df_vale_fechamento_trimestral.tail())

In [ ]:
# VALE3

# Valor de mercado
vm_trimestral_vale = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_vale[3:],
    df_fechamento=df_vale_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_vale = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_vale,
    df_dre=dre_trimestral,
    cod_cvm=4170,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_vale = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_vale,
    df_dre=dre_trimestral,
    cod_cvm=4170,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_vale = f_br.indicador_pl_lp(
    df_fechamento=df_vale_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_vale
)[0]

# L/P
lp_trimestral_vale = f_br.indicador_pl_lp(
    df_fechamento=df_vale_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_vale
)[1]

# EBITDA
ebitda_trimestral_vale = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.06'
)[0]

ebitda_trimestral_acum_vale = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.06'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_vale = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=4170,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_vale = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=4170,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_vale = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_vale,
    df_divida_bruta=divida_bruta_trimestral_vale
)

# DL/EBITDA
dl_ebitda_trimestral_vale = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_vale,
    df_ebitda=ebitda_trimestral_acum_vale
)

# DL/PL
dl_pl_trimestral_vale = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=4170,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_vale
)

# EV/EBITDA
ev_ebitda_trimestral_vale = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_vale,
    df_divida_liquida=divida_liquida_trimestral_vale,
    df_ebitda=ebitda_trimestral_acum_vale
)

# VPA
vpa_trimestral_vale = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_vale[3:],
    cod_cvm=4170,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_vale = f_br.indicador_p_vpa(
    df_fechamento=df_vale_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_vale
)

# Margem líquida
ml_trimestral_vale = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=4170,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_vale = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=4170,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_vale = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_vale
)

# Proventos - utilizei os números do dfc (no dva não aparece os pagamentos dos proventos)
dict_proventos_trimestral_vale = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'proventos': [
        # 2020
        0, 41_316, 18_547_976,
        # 2021
        21_866_000, 32_942_768, 73_158_646,
        # 2022
        17_849_000, 17_884_000, 34_143_000,
        # 2023
        9_449_000, 9_449_000, 17_726_000,
        # 2024
        11_722_000, 11_722_000, 20_662_000,
        # 2025
        11_365_000, 11_365_000, 19_456_000,
    ]
}

proventos_trimestral_vale = f_br.dict_to_polars_df(dict_proventos_trimestral_vale)

# Calculando o provento não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_proventos_nao_acum = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se provento é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('proventos') == 0)
        .then(0)
        .otherwise(pl.col('proventos') - pl.col('proventos').shift(1))
    )
    .otherwise(pl.col('proventos')) # Mantém o valor original para outros meses
    .alias('proventos')
)

proventos_trimestral_vale_nao_acum = proventos_trimestral_vale.with_columns([expr_proventos_nao_acum]).select('dt_refer', 'proventos').fill_null(0)

# Payout
payout_trimestral_vale = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=4170,
    df_proventos=proventos_trimestral_vale_nao_acum,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_vale = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'buyback': [
        # 2020
        0, 0, 0,
        # 2021
        0, 10_407_302, 25_260_819,
        # 2022
        9_176_000, 21_928_000, 25_564_000,
        # 2023
        4_116_000, 10_714_000, 13_374_000,
        # 2024
        1_357_000, 1_942_000 , 2_054_000,
        # 2025
        0, 0, 0,
    ]
}

buyback_trimestral_vale = f_br.dict_to_polars_df(dict_buyback_trimestral_vale)

# Calculando o buyback não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_buyback_nao_acum = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se buyback é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('buyback') == 0)
        .then(0)
        .otherwise(pl.col('buyback') - pl.col('buyback').shift(1))
    )
    .otherwise(pl.col('buyback')) # Mantém o valor original para outros meses
    .alias('buyback_nao_acum')
)

buyback_nao_acum = buyback_trimestral_vale.with_columns([expr_buyback_nao_acum]).select('dt_refer', 'buyback_nao_acum').fill_null(0)

# Calculando o buyback trimestral acumulado
expr_buyback_acum = (
    pl.col('buyback_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('buyback_acum')
)

buyback_trimestral_acum_vale = buyback_nao_acum.with_columns([expr_buyback_acum]).select('dt_refer', 'buyback_acum').fill_null(0)
buyback_trimestral_acum_vale = buyback_trimestral_acum_vale[3:]

# FCO
fco_trimestral_vale = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_vale = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_vale = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_vale = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_vale = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_vale = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.05', # 'Adições ao Imobilizado'
]

capex_trimestral_vale = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_vale = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_vale = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_vale,
    df_capex=capex_trimestral_acum_vale
)

# Net capex
net_capex_trimestral_vale = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_depreciacao='6.01.01.06',
    df_capex=capex_trimestral_acum_vale
)

# R&D - está informação está no arquivo do DFP/ITR - na parte "Informações Contábeis" na linha "Despesas com pesquisa e desenvolvimento".
dict_rd_trimestral_vale = {
    'dt_refer':[
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2020
        429_000, 484_000, 563_000,
        # 2021
        552_000, 743_000, 708_000,
        # 2022
        631_000, 745_000, 888_000,
        # 2023
        723_000, 816_000, 917_000,
        # 2024
        772_000, 988_000, 1_066_000,
        # 2025
        719_000, 898_000, 821_000,
    ]
}

rd_trimestral_vale = f_br.dict_to_polars_df(dict_rd_trimestral_vale)

# Calculando o R&D trimestral acumulado
expr_rd_nao_acum = (
    pl.col('rd')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('rd') # Não mudei para 'rd_acum', por causa da função 'indicador_adjusted_net_capex'
)

rd_trimestral_acum_vale = rd_trimestral_vale.with_columns([expr_rd_nao_acum]).select('dt_refer', 'rd').fill_null(0)
rd_trimestral_acum_vale = rd_trimestral_acum_vale[3:]

# Adjusted net capex
adjusted_net_capex_trimestral_vale = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_vale,
    df_rd=rd_trimestral_acum_vale
)

# Working capital
working_capital_trimestral_vale = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=4170,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_vale = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_vale = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_vale = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_vale,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_vale
)

# Effective tax rate
effective_tax_rate_trimestral_vale = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=4170,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_vale = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=4170,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_vale,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.01' # 'Empréstimos e Financiamentos - Adições'
]

new_borrowing_trimestral_vale = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Empréstimos e Financiamentos - Baixas' - '6.03.02';
    - 'debt_paid_item_2' - 'Arrendamentos - pagamentos' - código varia durantes os anos.
* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.
* No item 'debt_paid_item_2', adicionar manualmente os dados.

In [ ]:
# Debt paid

# DFC da VALE
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 4170)

################################################################
# 'Empréstimos e Financiamentos - Baixas'
DATAS_ALVO_DP_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_DP_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Calculando o new borrowing não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

debt_paid_nao_acum_item_1 = debt_paid_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o new borrowing trimestral acumulado
expr_debt_paid_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

debt_paid_acum_item_1 = debt_paid_nao_acum_item_1.with_columns([expr_debt_paid_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
debt_paid_acum_item_1 = debt_paid_acum_item_1[3:]

################################################################
# 'Pagamentos de arrendamento'
# '6.03.03' (06/20, 06/21, 06/22, 06/23, 06/24 e 06/25)
# '6.03.04' (09/20, 09/21, 09/22, 09/23, 09/24 e 09/25)
# '6.03.05' (03/20, 03/21, 03/22, 03/23, 03/24 e 03/25)
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'vl_conta_2': [
        # 2020
        -218_468, -481_530, -726_226,
        # 2021
        -304_004, -563_000, -859_959,
        # 2022
        -216_000, -492_000, -744_000,
        # 2023
        -246_000, -468_000, -694_000,
        # 2024
        -205_000, -430_000, -695_000,
        # 2025
        -174_000, -361_000, -590_000,
    ]
}

debt_paid_item_2 = f_br.dict_to_polars_df(dict_debt_paid_item_2)

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2,
]
debt_paid_trimestral_vale = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_vale = debt_paid_trimestral_vale.with_columns(
    ((pl.col('vl_conta_1_acum') + pl.col('vl_conta_2_acum')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_trimestral_vale = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.06',
    df_capex=capex_trimestral_acum_vale,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_vale,
    df_new_borrowing=new_borrowing_trimestral_vale,
    df_debt_paid=debt_paid_trimestral_vale,
)

# FCFF
fcff_trimestral_vale = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=4170,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.06',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_vale,
    df_capex=capex_trimestral_acum_vale,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_vale,
)

In [ ]:
# Lista dos df
lst_df_trimestral_vale = [
    vm_trimestral_vale,
    lpa_trimestral_vale,
    lpa_trimestral_acum_vale.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_vale,
    lp_trimestral_vale,
    ebitda_trimestral_vale,
    ebitda_trimestral_acum_vale.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_vale,
    caixa_trimestral_vale,
    divida_liquida_trimestral_vale,
    dl_ebitda_trimestral_vale,
    dl_pl_trimestral_vale,
    ev_ebitda_trimestral_vale,
    vpa_trimestral_vale,
    p_vp_trimestral_vale,
    ml_trimestral_vale,
    roe_trimestral_vale,
    roic_trimestral_vale,
    proventos_trimestral_vale_nao_acum,
    payout_trimestral_vale,
    buyback_trimestral_vale,
    fco_trimestral_vale,
    fci_trimestral_vale,
    fcf_trimestral_vale,
    fco_trimestral_acum_vale.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_vale.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_vale.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_vale,
    capex_trimestral_acum_vale.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_vale,
    net_capex_trimestral_vale,
    rd_trimestral_vale,
    rd_trimestral_acum_vale.rename({'rd': 'rd_acum'}),
    adjusted_net_capex_trimestral_vale,
    working_capital_trimestral_vale,
    change_in_non_cash_wc_trimestral_vale,
    change_in_non_cash_wc_trimestral_acum_vale.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_vale,
    effective_tax_rate_trimestral_vale,
    fcfe_trimestral_vale,
    fcff_trimestral_vale
]

# Juntando os dfs
df_indicadores_trimestral_vale = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_vale
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_vale = df_indicadores_trimestral_vale.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_vale

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_vale.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//mineracao//vale//vale_indicadores_trimestral.csv'),
    separator=';'
)

# **Setor de Petróleo**

## PETR4

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - Buyback no dicionário 'dict_buyback_petr'.

In [ ]:
# Número de ações - PETR
dict_composicao_capital_17_18_19_petr = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        7_442_454_000,
        7_442_454_000,
        7_442_454_000,
    ],

    'num_pn':[
        5_602_043_000,
        5_602_043_000,
        5_602_043_000,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_petr = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_petr)

# Calculando o total de ações
df_composicao_capital_17_18_19_petr = df_composicao_capital_17_18_19_petr.with_columns(
    (pl.col('num_on') + pl.col('num_pn')).alias('num_total')
)

# Composição de capital (2020 a 2024) - PETR
composicao_capital_petr = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('PETROLEO BRASILEIRO S.A.'))

# Arrumando a escala da linha de '2020-12-31'
composicao_capital_petr = composicao_capital_petr.with_columns([
    pl.when(pl.col('dt_refer') == pl.datetime(2020, 12, 31))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on'),

    pl.when(pl.col('dt_refer') == pl.datetime(2020, 12, 31))
      .then(pl.col('num_pn') * 1000)
      .otherwise(pl.col('num_pn'))
      .alias('num_pn'),

    pl.when(pl.col('dt_refer') == pl.datetime(2020, 12, 31))
      .then(pl.col('num_total') * 1000)
      .otherwise(pl.col('num_total'))
      .alias('num_total'),
]).select('dt_refer', 'num_on', 'num_pn', 'num_total')

# Juntando os dfs
df_composicao_capital_petr = pl.concat([df_composicao_capital_17_18_19_petr, composicao_capital_petr])

##############################################################################
# Preços de fechamento - PETR
df_petr_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit=None,
    list_ticker_on_pn=['PETR3.SA', 'PETR4.SA'],
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_petr.tail())
print('=' * 80)
print(df_petr_fechamento_anual.tail())

In [ ]:
# PETR

# Valor de mercado atual
vm_atual_petr = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='PETR3.SA', ticker_pn='PETR4.SA')

# Valor de mercado
vm_petr = f_br.indicador_vm(df_capital_social=df_composicao_capital_petr, df_fechamento=df_petr_fechamento_anual, novo_mercado=False)

# LPA
lpa_petr = f_br.indicador_lpa(df_capital_social=df_composicao_capital_petr, df_dre=dre, cod_cvm=9512, cod_ll='3.11.01', novo_mercado=False)

# P/L
pl_petr = f_br.indicador_pl_lp(df_fechamento=df_petr_fechamento_anual, tipo_acao='pn', df_lpa=lpa_petr)[0]

# L/P
lp_petr = f_br.indicador_pl_lp(df_fechamento=df_petr_fechamento_anual, tipo_acao='pn', df_lpa=lpa_petr)[1]

# EBITDA
ebitda_petr = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=9512, cod_ebit='3.05', cod_depreciacao='6.01.01.04')

# Dívida bruta
divida_bruta_petr = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=9512, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_petr = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=9512, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_petr = f_br.indicador_divida_liquida(df_caixa_total=caixa_petr, df_divida_bruta=divida_bruta_petr)

# DL/EBITDA
dl_ebitda_petr = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_petr, df_ebitda=ebitda_petr)

# DL/PL
dl_pl_petr = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=9512, cod_pl='2.03', df_divida_liquida=divida_liquida_petr)

# EV/EBITDA
ev_ebitda_petr = f_br.indicador_ev_ebitda(df_vm=vm_petr, df_divida_liquida=divida_liquida_petr, df_ebitda=ebitda_petr)

# VPA
vpa_petr = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_petr, cod_cvm=9512, cod_pl='2.03', novo_mercado=False)

# P/VP
p_vp_petr = f_br.indicador_p_vpa(df_fechamento=df_petr_fechamento_anual, tipo_acao='pn', df_vpa=vpa_petr)

# Margem líquida
ml_petr = f_br.indicador_ml(df_dre=dre, cod_cvm=9512, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_petr = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=9512, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_petr = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=9512, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_petr)

# Dividendos
proventos_petr = f_br.indicador_proventos(df_dva=dva, cod_cvm=9512, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_petr = f_br.indicador_payout(df_dre=dre, cod_cvm=9512, df_proventos=proventos_petr, cod_ll='3.11')

# Buyback
dict_buyback_petr = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback': [
        0,
        0,
        0,
        0,
        0,
        0,
        3_644_000,
        1_919_000,
    ]
}

buyback_petr = f_br.dict_to_polars_df(dict_buyback_petr)

# FCO
fco_petr = f_br.indicador_fco(df_dfc=dfc, cod_cvm=9512, cod_fco='6.01')

# FCI
fci_petr = f_br.indicador_fci(df_dfc=dfc, cod_cvm=9512, cod_fci='6.02')

# FCF
fcf_petr = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=9512, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.01' # 'Aquisições de ativos imobilizados e intangíveis'
]

capex_petr = f_br.indicador_capex(df_dfc=dfc, cod_cvm=9512, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_petr = f_br.indicador_free_cash_flow(df_fco=fco_petr, df_capex=capex_petr)

# Net capex
net_capex_petr = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=9512, cod_depreciacao='6.01.01.04', df_capex=capex_petr)

# R&D
rd_petr = f_br.indicador_rd(df_dre=dre, cod_cvm=9512, cod_rd='3.04.05.02', anual=True)

# Adjusted net capex
adjusted_net_capex_petr = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_petr, df_rd=rd_petr)

# Working capital
working_capital_petr = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=9512,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_petr = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=9512, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_petr = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_petr,
    df_change_in_non_cash_wc=change_in_non_cash_wc_petr
)

# Effective tax rate
effective_tax_rate_petr = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=9512, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_petr = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=9512, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_petr, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.02' # 'Captações'
]

new_borrowing_petr = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=9512, lst_cod_new_borrowing=lst_cod_new_borrowing)

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Amortizações de principal - financiamentos' - '6.03.03'
    - 'debt_paid_item_2' - 'Amortizações de juros - financiamentos' - '6.03.04'
    - 'debt_paid_item_3' - 'Amortizações de arrendamentos' - '6.03.08'

* Os itens 'debt_paid_item_1' e 'debt_paid_item_2' estão corretos, apenas o 'debt_paid_item_3' estava com falta de dados;
* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid
lst_cod_debt_paid = [
    '6.03.03', # 'Amortizações de principal - financiamentos'
    '6.03.04', # 'Amortizações de juros - financiamentos'
]

debt_paid_petr_item_1_2 = f_br.indicador_debt_paid(df_dfc=dfc, cod_cvm=9512, lst_cod_debt_paid=lst_cod_debt_paid)
debt_paid_petr_item_1_2 = debt_paid_petr_item_1_2.rename({'debt_paid': 'vl_conta_1_2'})

################################################################
# DFC da PETR
dfc_petr = dfc.filter(pl.col('cd_cvm') == 9512)

################################################################
# 'Amortizações de arrendamentos'
DATAS_ALVO_3 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_3 = (
    dfc_petr
    .filter(
        (pl.col('cd_conta') == '6.03.08') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_3))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_3')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
    ],

    'vl_conta_3': [
        0.0,
        0.0,
    ]
}

debt_paid_item_3_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_3)
debt_paid_item_3 = pl.concat([debt_paid_item_3_complementos, debt_paid_item_3])
debt_paid_item_3 = debt_paid_item_3.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_petr_item_1_2, debt_paid_item_3]
debt_paid_petr = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_petr = debt_paid_petr.with_columns(
    ((pl.col('vl_conta_1_2') + pl.col('vl_conta_3')) * -1).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_petr = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=9512,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.04',
    df_capex=capex_petr,
    df_change_in_non_cash_wc=change_in_non_cash_wc_petr,
    df_new_borrowing=new_borrowing_petr,
    df_debt_paid=debt_paid_petr,
)

# FCFF
fcff_petr = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.04',
    df_taxes_on_operating_income=taxes_on_operating_income_petr,
    df_capex=capex_petr,
    df_change_in_non_cash_wc=change_in_non_cash_wc_petr,
)

In [ ]:
# Lista dos df
lst_df_petr = [
    vm_petr,
    lpa_petr,
    pl_petr,
    lp_petr,
    ebitda_petr,
    divida_bruta_petr,
    caixa_petr,
    divida_liquida_petr,
    dl_ebitda_petr,
    dl_pl_petr,
    ev_ebitda_petr,
    vpa_petr,
    p_vp_petr,
    ml_petr,
    roe_petr,
    roic_petr,
    proventos_petr,
    payout_petr,
    buyback_petr,
    fco_petr,
    fci_petr,
    fcf_petr,
    capex_petr,
    free_cash_flow_petr,
    net_capex_petr,
    rd_petr,
    adjusted_net_capex_petr,
    working_capital_petr,
    change_in_non_cash_wc_petr,
    reinvestment_rate_petr,
    effective_tax_rate_petr,
    fcfe_petr,
    fcff_petr
]

# Juntando os dfs
df_indicadores_petr = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_petr
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_petr = df_indicadores_petr.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_petr

In [ ]:
# Transformando em um arquivo csv
df_indicadores_petr.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//petroleo//petr//petr_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - Buyback no dicionário 'dict_buyback_trimestral_petr'.

In [ ]:
# Composição de capital - PETR
composicao_capital_trimestral_petr = composicao_capital_trimestral.filter(
    composicao_capital_trimestral['denom_cia'].str.contains('PETROLEO BRASILEIRO S.A.')
)

# Calculando o total de ações
df_composicao_capital_trimestral_petr = composicao_capital_trimestral_petr.with_columns(
    (pl.col('num_on') + pl.col('num_pn')).alias('num_total')
).select('dt_refer', 'num_on', 'num_pn', 'num_total')

##############################################################################
# Preços de fechamento - PETR
df_petr_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit=None,
    list_ticker_on_pn=['PETR3.SA', 'PETR4.SA'],
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=composicao_capital_trimestral_petr['dt_refer']
)

print(df_composicao_capital_trimestral_petr.tail())
print('=' * 80)
print(df_petr_fechamento_trimestral.tail())

In [ ]:
# PETR

# Valor de mercado
vm_trimestral_petr = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_petr[3:],
    df_fechamento=df_petr_fechamento_trimestral[3:],
    novo_mercado=False
)

# LPA
lpa_trimestral_petr = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_petr,
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_ll='3.11.01',
    novo_mercado=False
)[0]

lpa_trimestral_acum_petr = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_petr,
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_ll='3.11.01',
    novo_mercado=False
)[1]

# P/L
pl_trimestral_petr = f_br.indicador_pl_lp(
    df_fechamento=df_petr_fechamento_trimestral[3:],
    tipo_acao='pn',
    df_lpa=lpa_trimestral_acum_petr
)[0]

# L/P
lp_trimestral_petr = f_br.indicador_pl_lp(
    df_fechamento=df_petr_fechamento_trimestral[3:],
    tipo_acao='pn',
    df_lpa=lpa_trimestral_acum_petr
)[1]

# EBITDA
ebitda_trimestral_petr = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.04'
)[0]

ebitda_trimestral_acum_petr = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.04'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_petr = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=9512,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_petr = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=9512,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_petr = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_petr,
    df_divida_bruta=divida_bruta_trimestral_petr
)

# DL/EBITDA
dl_ebitda_trimestral_petr = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_petr,
    df_ebitda=ebitda_trimestral_acum_petr
)

# DL/PL
dl_pl_trimestral_petr = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=9512,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_petr
)

# EV/EBITDA
ev_ebitda_trimestral_petr = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_petr,
    df_divida_liquida=divida_liquida_trimestral_petr,
    df_ebitda=ebitda_trimestral_acum_petr
)

# VPA
vpa_trimestral_petr = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_petr[3:],
    cod_cvm=9512,
    cod_pl='2.03',
    novo_mercado=False
)

# P/VP
p_vp_trimestral_petr = f_br.indicador_p_vpa(
    df_fechamento=df_petr_fechamento_trimestral[3:],
    tipo_acao='pn',
    df_vpa=vpa_trimestral_petr
)

# Margem líquida
ml_trimestral_petr = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_petr = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=9512,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_petr = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_petr
)

# Proventos
proventos_trimestral_petr = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=9512,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_petr = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    df_proventos=proventos_trimestral_petr,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_petr = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'buyback': [
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 975_000,
        # 2024
        1_147_000, 1_919_000, 0,
        # 2025
        0, 0, 0,

    ]
}

buyback_trimestral_petr = f_br.dict_to_polars_df(dict_buyback_trimestral_petr)

# Calculando o buyback não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_buyback_nao_acum = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se buyback é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('buyback') == 0)
        .then(0)
        .otherwise(pl.col('buyback') - pl.col('buyback').shift(1))
    )
    .otherwise(pl.col('buyback')) # Mantém o valor original para outros meses
    .alias('buyback_nao_acum')
)

buyback_trimestral_petr = buyback_trimestral_petr.with_columns([expr_buyback_nao_acum]).select('dt_refer', 'buyback_nao_acum').fill_null(0)

# FCO
fco_trimestral_petr = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_petr = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_petr = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_petr = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_petr = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_petr = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01' # 'Aquisições de ativos imobilizados e intangíveis'
]

capex_trimestral_petr = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_petr = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_petr = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_petr,
    df_capex=capex_trimestral_acum_petr
)

# Net capex
net_capex_trimestral_petr = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_depreciacao='6.01.01.04',
    df_capex=capex_trimestral_acum_petr
)

# R&D
rd_trimestral_petr = f_br.indicador_rd(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_rd='3.04.05.02',
    anual=False
)[0]

rd_trimestral_acum_petr = f_br.indicador_rd(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_rd='3.04.05.02',
    anual=False
)[1]

# Adjusted net capex
adjusted_net_capex_trimestral_petr = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_petr,
    df_rd=rd_trimestral_acum_petr
)

# Working capital
working_capital_trimestral_petr = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=9512,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)[3:]

# Change in non-cash wc
change_in_non_cash_wc_trimestral_petr = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_petr = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_petr = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_petr,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_petr
)

# Effective tax rate
effective_tax_rate_trimestral_petr = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_petr = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=9512,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_petr,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.02' # 'Captações'
]

new_borrowing_trimestral_petr = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

# Debt paid
lst_cod_debt_paid = [
    '6.03.03', # 'Amortizações de principal - financiamentos'
    '6.03.04', # 'Amortizações de juros - financiamentos'
    '6.03.08'  # 'Amortizações de arrendamentos'
]

debt_paid_trimestral_petr = f_br.indicador_debt_paid_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    lst_cod_debt_paid=lst_cod_debt_paid
)[3:]

# FCFE
fcfe_trimestral_petr = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.04',
    df_capex=capex_trimestral_acum_petr,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_petr,
    df_new_borrowing=new_borrowing_trimestral_petr,
    df_debt_paid=debt_paid_trimestral_petr,
)

# FCFF
fcff_trimestral_petr = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=9512,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.04',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_petr,
    df_capex=capex_trimestral_acum_petr,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_petr,
)

In [ ]:
# Lista dos df
lst_df_trimestral_petr = [
    vm_trimestral_petr,
    lpa_trimestral_petr,
    lpa_trimestral_acum_petr.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_petr,
    lp_trimestral_petr,
    ebitda_trimestral_petr,
    ebitda_trimestral_acum_petr.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_petr,
    caixa_trimestral_petr,
    divida_liquida_trimestral_petr,
    dl_ebitda_trimestral_petr,
    dl_pl_trimestral_petr,
    ev_ebitda_trimestral_petr,
    vpa_trimestral_petr,
    p_vp_trimestral_petr,
    ml_trimestral_petr,
    roe_trimestral_petr,
    roic_trimestral_petr,
    proventos_trimestral_petr,
    payout_trimestral_petr,
    buyback_trimestral_petr,
    fco_trimestral_petr,
    fci_trimestral_petr,
    fcf_trimestral_petr,
    fco_trimestral_acum_petr.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_petr.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_petr.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_petr,
    capex_trimestral_acum_petr.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_petr,
    net_capex_trimestral_petr,
    rd_trimestral_petr,
    rd_trimestral_acum_petr.rename({'rd': 'rd_acum'}),
    adjusted_net_capex_trimestral_petr,
    working_capital_trimestral_petr,
    change_in_non_cash_wc_trimestral_petr,
    change_in_non_cash_wc_trimestral_acum_petr.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_petr,
    effective_tax_rate_trimestral_petr,
    fcfe_trimestral_petr,
    fcff_trimestral_petr
]

# Juntando os dfs
df_indicadores_trimestral_petr = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_petr
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_petr = df_indicadores_trimestral_petr.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_petr

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_petr.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//petroleo//petr//petr_indicadores_trimestral.csv'),
    separator=';'
)

## PRIO3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_prio'.

In [ ]:
# Número de ações - PRIO
dict_composicao_capital_17_18_19_prio = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        13_191_932,
        13_336_517,
        143_185_890,
    ],

    'num_pn':[
        0,
        0,
        0,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_prio = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_prio)

# Calculando o total de ações
df_composicao_capital_17_18_19_prio = df_composicao_capital_17_18_19_prio.with_columns(
    (pl.col('num_on') + pl.col('num_pn')).alias('num_total')
)

# Composição de capital (2020 a 2024) - PRIO
composicao_capital_prio = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('PRIO S.A')).select('dt_refer', 'num_on', 'num_pn', 'num_total')

# Juntando os dfs
df_composicao_capital_prio = pl.concat([df_composicao_capital_17_18_19_prio, composicao_capital_prio])

##############################################################################
# Preços de fechamento - PRIO
df_prio_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='PRIO3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_prio.tail())
print('=' * 80)
print(df_prio_fechamento_anual.tail())

In [ ]:
# PRIO

# Valor de mercado atual
vm_atual_prio = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='PRIO3.SA', ticker_pn=None)

# Valor de mercado
vm_prio = f_br.indicador_vm(df_capital_social=df_composicao_capital_prio, df_fechamento=df_prio_fechamento_anual, novo_mercado=True)

# LPA
lpa_prio = f_br.indicador_lpa(df_capital_social=df_composicao_capital_prio, df_dre=dre, cod_cvm=22187, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_prio = f_br.indicador_pl_lp(df_fechamento=df_prio_fechamento_anual, tipo_acao='on', df_lpa=lpa_prio)[0]

# L/P
lp_prio = f_br.indicador_pl_lp(df_fechamento=df_prio_fechamento_anual, tipo_acao='on', df_lpa=lpa_prio)[1]

# EBITDA
ebitda_prio = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=22187, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_prio = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=22187, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_prio = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=22187, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_prio = f_br.indicador_divida_liquida(df_caixa_total=caixa_prio, df_divida_bruta=divida_bruta_prio)

# DL/EBITDA
dl_ebitda_prio = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_prio, df_ebitda=ebitda_prio)

# DL/PL
dl_pl_prio = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=22187, cod_pl='2.03', df_divida_liquida=divida_liquida_prio)

# EV/EBITDA
ev_ebitda_prio = f_br.indicador_ev_ebitda(df_vm=vm_prio, df_divida_liquida=divida_liquida_prio, df_ebitda=ebitda_prio)

# VPA
vpa_prio = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_prio, cod_cvm=22187, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_prio = f_br.indicador_p_vpa(df_fechamento=df_prio_fechamento_anual, tipo_acao='on', df_vpa=vpa_prio)

# Margem líquida
ml_prio = f_br.indicador_ml(df_dre=dre, cod_cvm=22187, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_prio = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=22187, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_prio = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=22187, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_prio)

# Dividendos
proventos_prio = f_br.indicador_proventos(df_dva=dva, cod_cvm=22187, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')

# Payout
payout_prio = f_br.indicador_payout(df_dre=dre, cod_cvm=22187, df_proventos=proventos_prio, cod_ll='3.11')

# Buyback
buyback_prio = f_br.indicador_buyback(df_dfc=dfc, cod_cvm=22187, cod_buyback='6.03.06')

# FCO
fco_prio = f_br.indicador_fco(df_dfc=dfc, cod_cvm=22187, cod_fco='6.01')

# FCI
fci_prio = f_br.indicador_fci(df_dfc=dfc, cod_cvm=22187, cod_fci='6.02')

# FCF
fcf_prio = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=22187, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.01' # 'Compra de ativo imobilizado'
]

capex_prio = f_br.indicador_capex(df_dfc=dfc, cod_cvm=22187, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_prio = f_br.indicador_free_cash_flow(df_fco=fco_prio, df_capex=capex_prio)

# Net capex
net_capex_prio = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=22187, cod_depreciacao='6.01.01.02', df_capex=capex_prio)

# R&D
dict_rd_prio = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31'
    ],

    'rd': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
    ]
}

rd_prio = f_br.dict_to_polars_df(dict_rd_prio)

# Adjusted net capex
adjusted_net_capex_prio = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_prio, df_rd=rd_prio)

# Working capital
working_capital_prio = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=22187,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_prio = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=22187, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_prio = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_prio,
    df_change_in_non_cash_wc=change_in_non_cash_wc_prio
)

# Effective tax rate
effective_tax_rate_prio = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=22187, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_prio = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=22187, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_prio, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.02', # Captações de empréstimos
    '6.03.12'  # Captação de debêntures
]

new_borrowing_prio = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=22187, lst_cod_new_borrowing=lst_cod_new_borrowing)

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de principal sobre empréstimos' - '6.03.03';
    - 'debt_paid_item_2' - 'Juros pagos sobre empréstimos' - '6.03.04';
    - 'debt_paid_item_3' - 'Encargos contratuais Leasing IFRS 16 - Principal' - '6.03.10';
    - 'debt_paid_item_4' - 'Encargos contratuais Leasing IFRS 16 - Juros' - '6.03.11';
    - 'debt_paid_item_5' - 'Juros pagos sobre debêntures' - '6.03.13'.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da PRIO
dfc_prio = dfc.filter(pl.col('cd_cvm') == 22187)

################################################################
# 'Pagamento de principal sobre empréstimos'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_prio
    .filter(
        (pl.col('cd_conta') == '6.03.03') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'vl_conta_1': [
        0.0,
        0.0,
        0.0,
    ]
}

debt_paid_item_1_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_1)
debt_paid_item_1 = pl.concat([debt_paid_item_1_complementos, debt_paid_item_1])
debt_paid_item_1 = debt_paid_item_1.sort('dt_refer')

################################################################
# 'Juros pagos sobre empréstimos'
DATAS_ALVO_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_prio
    .filter(
        (pl.col('cd_conta') == '6.03.04') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

debt_paid_item_2 = debt_paid_item_2.with_columns(
    pl.when(pl.col('dt_refer') == datetime(2022, 12, 31))
    .then(-230139)
    .otherwise(pl.col('vl_conta_2'))
    .alias('vl_conta_2')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'vl_conta_2': [
        0.0,
        0.0,
        0.0,
    ]
}

debt_paid_item_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_paid_item_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

################################################################
# 'Encargos contratuais Leasing IFRS 16 - Principal'
DATAS_ALVO_3 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_3 = (
    dfc_prio
    .filter(
        (pl.col('cd_conta') == '6.03.10') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_3))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_3')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'vl_conta_3': [
        0.0,
        0.0,
        -158495.0,
    ]
}

debt_paid_item_3_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_3)
debt_paid_item_3 = pl.concat([debt_paid_item_3_complementos, debt_paid_item_3])
debt_paid_item_3 = debt_paid_item_3.sort('dt_refer')

################################################################
# 'Encargos contratuais Leasing IFRS 16 - Juros'
DATAS_ALVO_4 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_4 = (
    dfc_prio
    .filter(
        (pl.col('cd_conta') == '6.03.11') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_4))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_4')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'vl_conta_4': [
        0.0,
        0.0,
        0.0,
    ]
}

debt_paid_item_4_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_4)
debt_paid_item_4 = pl.concat([debt_paid_item_4_complementos, debt_paid_item_4])
debt_paid_item_4 = debt_paid_item_4.sort('dt_refer')

################################################################
# 'Juros pagos sobre debêntures'
DATAS_ALVO_5 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_5 = (
    dfc_prio
    .filter(
        (pl.col('cd_conta') == '6.03.13') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_5))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_5')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_5 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31'
    ],

    'vl_conta_5': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
    ]
}

debt_paid_item_5_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_5)
debt_paid_item_5 = pl.concat([debt_paid_item_5_complementos, debt_paid_item_5])
debt_paid_item_5 = debt_paid_item_5.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_item_1, debt_paid_item_2, debt_paid_item_3, debt_paid_item_4, debt_paid_item_5]
debt_paid_prio = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_prio = debt_paid_prio.with_columns(
    ((
        pl.col('vl_conta_1') +
        pl.col('vl_conta_2') +
        pl.col('vl_conta_3') +
        pl.col('vl_conta_4') +
        pl.col('vl_conta_5')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')
################################################################

# FCFE
fcfe_prio = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=22187,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_prio,
    df_change_in_non_cash_wc=change_in_non_cash_wc_prio,
    df_new_borrowing=new_borrowing_prio,
    df_debt_paid=debt_paid_prio,
)

# FCFF
fcff_prio = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_prio,
    df_capex=capex_prio,
    df_change_in_non_cash_wc=change_in_non_cash_wc_prio,
)

In [ ]:
# Lista dos df
lst_df_prio = [
    vm_prio,
    lpa_prio,
    pl_prio,
    lp_prio,
    ebitda_prio,
    divida_bruta_prio,
    caixa_prio,
    divida_liquida_prio,
    dl_ebitda_prio,
    dl_pl_prio,
    ev_ebitda_prio,
    vpa_prio,
    p_vp_prio,
    ml_prio,
    roe_prio,
    roic_prio,
    proventos_prio,
    payout_prio,
    buyback_prio,
    fco_prio,
    fci_prio,
    fcf_prio,
    capex_prio,
    free_cash_flow_prio,
    net_capex_prio,
    rd_prio,
    adjusted_net_capex_prio,
    working_capital_prio,
    change_in_non_cash_wc_prio,
    reinvestment_rate_prio,
    effective_tax_rate_prio,
    fcfe_prio,
    fcff_prio
]

# Juntando os dfs
df_indicadores_prio = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_prio
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_prio = df_indicadores_prio.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_prio

In [ ]:
# Transformando em um arquivo csv
df_indicadores_prio.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//petroleo//prio//prio_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_trimestral_prio'.

In [ ]:
# Composição de capital - PRIO
df_composicao_capital_trimestral_prio = composicao_capital_trimestral.filter(
    composicao_capital_trimestral['denom_cia'].str.contains('PRIO S.A')
).select('dt_refer', 'num_on', 'num_pn', 'num_total')

##############################################################################
# Preços de fechamento - PRIO
df_prio_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='PRIO3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_prio['dt_refer']
)

print(df_composicao_capital_trimestral_prio.tail())
print('=' * 80)
print(df_prio_fechamento_trimestral.tail())

In [ ]:
# PRIO

# Valor de mercado
vm_trimestral_prio = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_prio[3:],
    df_fechamento=df_prio_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_prio = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_prio,
    df_dre=dre_trimestral,
    cod_cvm=22187,
    cod_ll='3.11',
    novo_mercado=True
)[0]

lpa_trimestral_acum_prio = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_prio,
    df_dre=dre_trimestral,
    cod_cvm=22187,
    cod_ll='3.11',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_prio = f_br.indicador_pl_lp(
    df_fechamento=df_prio_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_prio
)[0]

# L/P
lp_trimestral_prio = f_br.indicador_pl_lp(
    df_fechamento=df_prio_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_prio
)[1]

# EBITDA
ebitda_trimestral_prio = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_prio = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_prio = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=22187,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_prio = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=22187,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_prio = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_prio,
    df_divida_bruta=divida_bruta_trimestral_prio
)

# DL/EBITDA
dl_ebitda_trimestral_prio = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_prio,
    df_ebitda=ebitda_trimestral_acum_prio
)

# DL/PL
dl_pl_trimestral_prio = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=22187,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_prio
)

# EV/EBITDA
ev_ebitda_trimestral_prio = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_prio,
    df_divida_liquida=divida_liquida_trimestral_prio,
    df_ebitda=ebitda_trimestral_acum_prio
)

# VPA
vpa_trimestral_prio = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_prio[3:],
    cod_cvm=22187,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_prio = f_br.indicador_p_vpa(
    df_fechamento=df_prio_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_prio
)

# Margem líquida
ml_trimestral_prio = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=22187,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_prio = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=22187,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_prio = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_prio
)

# Proventos
proventos_trimestral_prio = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=22187,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

# Payout
payout_trimestral_prio = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=22187,
    df_proventos=proventos_trimestral_prio,
    cod_ll='3.11'
)

# Buyback - a prioobras só fez recompra em 2023 e 2024
buyback_trimestral_prio = f_br.indicador_buyback_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_buyback='6.03.06'
)[3:]

# FCO
fco_trimestral_prio = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_prio = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_prio = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_prio = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_prio = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_prio = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01' # 'Compra de ativo imobilizado'
]

capex_trimestral_prio = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_prio = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_prio = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_prio,
    df_capex=capex_trimestral_acum_prio
)

# Net capex
net_capex_trimestral_prio = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_prio
)

# R&D
dict_rd_trimestral_prio = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30'
    ],

    'rd': [
        # 2021
        0.0, 0.0, 0.0,
        # 2022
        0.0, 0.0, 0.0,
        # 2023
        0.0, 0.0, 0.0,
        # 2024
        0.0, 0.0, 0.0,
        # 2025
        0.0, 0.0, 0.0,
    ]
}

rd_trimestral_prio = f_br.dict_to_polars_df(dict_rd_trimestral_prio)

# Adjusted net capex
adjusted_net_capex_trimestral_prio = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_prio,
    df_rd=rd_trimestral_prio
)

# Working capital
working_capital_trimestral_prio = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=22187,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)[3:]

# Change in non-cash wc
change_in_non_cash_wc_trimestral_prio = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_prio = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_prio = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_prio,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_prio
)

# Effective tax rate
effective_tax_rate_trimestral_prio = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=22187,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_prio = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=22187,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_prio,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.02', # Captações de empréstimos
    '6.03.12'  # Captação de debêntures
]

new_borrowing_trimestral_prio = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de juros s/ arrendamentos (IFRS 16)' - '6.03.04';
    - 'debt_paid_item_2' - 'Pagamento de arrendamentos (IFRS 16)' - '6.03.05';
    - 'debt_paid_item_3' - 'Encargos contratuais Leasing IFRS 16' - '6.03.07';
    - 'debt_paid_item_4' - 'Pagamento de Principal sobre Empréstimos' - '6.03.09';
    - 'debt_paid_item_5' - 'Juros Pagos sobre Empréstimos' - '6.03.10';
    - 'debt_paid_item_6' - 'Juros pagos sobre debêntures' - '6.03.12';
    - 'debt_paid_item_7' - 'Empréstimos' - '6.03.02'.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da PRIO
dfc_empresa = dfc_trimestral.filter(pl.col('cd_cvm') == 22187)

################################################################
# 'Pagamento de juros s/ arrendamentos (IFRS 16)'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2021, 3, 31),
    datetime(2022, 3, 31),
    datetime(2023, 3, 31),
    datetime(2024, 3, 31),
    datetime(2025, 3, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.04') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2020-06-30', '2020-09-30',
        '2021-06-30', '2021-09-30',
        '2022-06-30', '2022-09-30',
        '2023-06-30', '2023-09-30',
        '2024-06-30', '2024-09-30',
        '2025-06-30',
    ],

    'vl_conta_1': [
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0,
    ]
}

debt_paid_item_1_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_1)
debt_paid_item_1 = pl.concat([debt_paid_item_1_complementos, debt_paid_item_1])
debt_paid_item_1 = debt_paid_item_1.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_1 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_1 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_1') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_1') - pl.col('vl_conta_1').shift(1))
    )
    .otherwise(pl.col('vl_conta_1')) # Mantém o valor original para outros meses
    .alias('vl_conta_1_nao_acum')
)

debt_paid_nao_acum_item_1 = debt_paid_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_1 = (
    pl.col('vl_conta_1_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_1_acum')
)

debt_paid_acum_item_1 = debt_paid_nao_acum_item_1.with_columns([expr_debt_paid_nao_acum_item_1]).select('dt_refer', 'vl_conta_1_acum').fill_null(0)
debt_paid_acum_item_1 = debt_paid_acum_item_1[3:]

################################################################
# 'Pagamento de arrendamentos (IFRS 16)'
DATAS_ALVO_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 3, 31),
    datetime(2021, 3, 31),
    datetime(2022, 3, 31),
    datetime(2023, 3, 31),
    datetime(2024, 3, 31),
    datetime(2025, 3, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.05') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2020-06-30', '2020-09-30',
        '2021-06-30', '2021-09-30',
        '2022-06-30', '2022-09-30',
        '2023-06-30', '2023-09-30',
        '2024-06-30', '2024-09-30',
        '2025-06-30',
    ],

    'vl_conta_2': [
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0,
    ]
}

debt_paid_item_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_paid_item_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_2 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_2 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_2') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_2') - pl.col('vl_conta_2').shift(1))
    )
    .otherwise(pl.col('vl_conta_2')) # Mantém o valor original para outros meses
    .alias('vl_conta_2_nao_acum')
)

debt_paid_nao_acum_item_2 = debt_paid_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_2 = (
    pl.col('vl_conta_2_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_2_acum')
)

debt_paid_acum_item_2 = debt_paid_nao_acum_item_2.with_columns([expr_debt_paid_nao_acum_item_2]).select('dt_refer', 'vl_conta_2_acum').fill_null(0)
debt_paid_acum_item_2 = debt_paid_acum_item_2[3:]

################################################################
# 'Encargos contratuais Leasing IFRS 16'
DATAS_ALVO_3 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 9, 30),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30)
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_3 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.07') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_3))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_3')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_3 = {
    'dt_refer': [
        '2020-03-31',
        '2021-03-31',
        '2022-03-31',
        '2023-03-31',
        '2024-03-31', '2024-06-30',
        '2025-03-31'
    ],

    'vl_conta_3': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0, 0.0,
        0.0,
    ]
}

debt_paid_item_3_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_3)
debt_paid_item_3 = pl.concat([debt_paid_item_3_complementos, debt_paid_item_3])
debt_paid_item_3 = debt_paid_item_3.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_3 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_3 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_3') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_3') - pl.col('vl_conta_3').shift(1))
    )
    .otherwise(pl.col('vl_conta_3')) # Mantém o valor original para outros meses
    .alias('vl_conta_3_nao_acum')
)

debt_paid_nao_acum_item_3 = debt_paid_item_3.with_columns([expr_debt_paid_nao_acum_item_3]).select('dt_refer', 'vl_conta_3_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_3 = (
    pl.col('vl_conta_3_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_3_acum')
)

debt_paid_acum_item_3 = debt_paid_nao_acum_item_3.with_columns([expr_debt_paid_nao_acum_item_3]).select('dt_refer', 'vl_conta_3_acum').fill_null(0)
debt_paid_acum_item_3 = debt_paid_acum_item_3[3:]

################################################################
# 'Pagamento de Principal sobre Empréstimos'
DATAS_ALVO_4 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2024, 3, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_4 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.09') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_4))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_4')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_4 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30'
    ],

    'vl_conta_4': [
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
    ]
}

debt_paid_item_4_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_4)
debt_paid_item_4 = pl.concat([debt_paid_item_4_complementos, debt_paid_item_4])
debt_paid_item_4 = debt_paid_item_4.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_4 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_4 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_4') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_4') - pl.col('vl_conta_4').shift(1))
    )
    .otherwise(pl.col('vl_conta_4')) # Mantém o valor original para outros meses
    .alias('vl_conta_4_nao_acum')
)

debt_paid_nao_acum_item_4 = debt_paid_item_4.with_columns([expr_debt_paid_nao_acum_item_4]).select('dt_refer', 'vl_conta_4_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_4 = (
    pl.col('vl_conta_4_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_4_acum')
)

debt_paid_acum_item_4 = debt_paid_nao_acum_item_4.with_columns([expr_debt_paid_nao_acum_item_4]).select('dt_refer', 'vl_conta_4_acum').fill_null(0)
debt_paid_acum_item_4 = debt_paid_acum_item_4[3:]

################################################################
# 'Juros Pagos sobre Empréstimos'
DATAS_ALVO_5 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2021, 3, 31),
    datetime(2023, 3, 31),
    datetime(2024, 3, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_5 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.10') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_5))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_5')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_5 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-06-30', '2023-09-30',
        '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30'
    ],

    'vl_conta_5': [
        0.0, 0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
        0.0, 0.0,
    ]
}

debt_paid_item_5_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_5)
debt_paid_item_5 = pl.concat([debt_paid_item_5_complementos, debt_paid_item_5])
debt_paid_item_5 = debt_paid_item_5.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_5 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_5 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_5') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_5') - pl.col('vl_conta_5').shift(1))
    )
    .otherwise(pl.col('vl_conta_5')) # Mantém o valor original para outros meses
    .alias('vl_conta_5_nao_acum')
)

debt_paid_nao_acum_item_5 = debt_paid_item_5.with_columns([expr_debt_paid_nao_acum_item_5]).select('dt_refer', 'vl_conta_5_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_5 = (
    pl.col('vl_conta_5_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_5_acum')
)

debt_paid_acum_item_5 = debt_paid_nao_acum_item_5.with_columns([expr_debt_paid_nao_acum_item_5]).select('dt_refer', 'vl_conta_5_acum').fill_null(0)
debt_paid_acum_item_5 = debt_paid_acum_item_5[3:]

################################################################
# 'Juros pagos sobre debêntures'
DATAS_ALVO_6 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2025, 3, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_6 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.12') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_6))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_6')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_6 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-06-30'
    ],

    'vl_conta_6': [
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0,
    ]
}

debt_paid_item_6_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_6)
debt_paid_item_6 = pl.concat([debt_paid_item_6_complementos, debt_paid_item_6])
debt_paid_item_6 = debt_paid_item_6.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_6 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_6 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_6') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_6') - pl.col('vl_conta_6').shift(1))
    )
    .otherwise(pl.col('vl_conta_6')) # Mantém o valor original para outros meses
    .alias('vl_conta_6_nao_acum')
)

debt_paid_nao_acum_item_6 = debt_paid_item_6.with_columns([expr_debt_paid_nao_acum_item_6]).select('dt_refer', 'vl_conta_6_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_6 = (
    pl.col('vl_conta_6_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_6_acum')
)

debt_paid_acum_item_6 = debt_paid_nao_acum_item_6.with_columns([expr_debt_paid_nao_acum_item_6]).select('dt_refer', 'vl_conta_6_acum').fill_null(0)
debt_paid_acum_item_6 = debt_paid_acum_item_6[3:]

################################################################
# 'Empréstimos'
DATAS_ALVO_7 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2024, 6, 30),
    datetime(2025, 3, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_7 = (
    dfc_empresa
    .filter(
        (pl.col('cd_conta') == '6.03.02') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_7))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_7')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_7 = {
    'dt_refer': [
        '2020-03-31', '2020-06-30', '2020-09-30',
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-09-30',
        '2025-06-30'
    ],

    'vl_conta_7': [
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0, 0.0,
        0.0, 0.0,
        0.0,
    ]
}

debt_paid_item_7_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_7)
debt_paid_item_7 = pl.concat([debt_paid_item_7_complementos, debt_paid_item_7])
debt_paid_item_7 = debt_paid_item_7.sort('dt_refer')

# Calculando o debt paid não acumulado para os meses 06 e 09 (e mantendo caso contrário)
expr_debt_paid_nao_acum_item_7 = (
    pl.when(
        (pl.col('dt_refer').dt.month() == 6) | (pl.col('dt_refer').dt.month() == 9)
    )
    .then(
        # Verifica se vl_conta_7 é 0 para manter 0. Caso contrário, faz a subtração.
        pl.when(pl.col('vl_conta_7') == 0)
        .then(0)
        .otherwise(pl.col('vl_conta_7') - pl.col('vl_conta_7').shift(1))
    )
    .otherwise(pl.col('vl_conta_7')) # Mantém o valor original para outros meses
    .alias('vl_conta_7_nao_acum')
)

debt_paid_nao_acum_item_7 = debt_paid_item_7.with_columns([expr_debt_paid_nao_acum_item_7]).select('dt_refer', 'vl_conta_7_nao_acum').fill_null(0)

# Calculando o debt paid trimestral acumulado
expr_debt_paid_nao_acum_item_7 = (
    pl.col('vl_conta_7_nao_acum')
    .rolling_sum(window_size=4, min_periods=4)
    .round(2)
    .alias('vl_conta_7_acum')
)

debt_paid_acum_item_7 = debt_paid_nao_acum_item_7.with_columns([expr_debt_paid_nao_acum_item_7]).select('dt_refer', 'vl_conta_7_acum').fill_null(0)
debt_paid_acum_item_7 = debt_paid_acum_item_7[3:]

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid_trimestral = [
    debt_paid_acum_item_1,
    debt_paid_acum_item_2,
    debt_paid_acum_item_3,
    debt_paid_acum_item_4,
    debt_paid_acum_item_5,
    debt_paid_acum_item_6,
    debt_paid_acum_item_7
]
debt_paid_trimestral_prio = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid_trimestral)

# Somando as colunas
debt_paid_trimestral_prio = debt_paid_trimestral_prio.with_columns(
    ((
        pl.col('vl_conta_1_acum') +
        pl.col('vl_conta_2_acum') +
        pl.col('vl_conta_3_acum') +
        pl.col('vl_conta_4_acum') +
        pl.col('vl_conta_5_acum') +
        pl.col('vl_conta_6_acum') +
        pl.col('vl_conta_7_acum')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_trimestral_prio = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_prio,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_prio,
    df_new_borrowing=new_borrowing_trimestral_prio,
    df_debt_paid=debt_paid_trimestral_prio,
)

# FCFF
fcff_trimestral_prio = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=22187,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_prio,
    df_capex=capex_trimestral_acum_prio,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_prio,
)

In [ ]:
# Lista dos df
lst_df_trimestral_prio = [
    vm_trimestral_prio,
    lpa_trimestral_prio,
    lpa_trimestral_acum_prio.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_prio,
    lp_trimestral_prio,
    ebitda_trimestral_prio,
    ebitda_trimestral_acum_prio.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_prio,
    caixa_trimestral_prio,
    divida_liquida_trimestral_prio,
    dl_ebitda_trimestral_prio,
    dl_pl_trimestral_prio,
    ev_ebitda_trimestral_prio,
    vpa_trimestral_prio,
    p_vp_trimestral_prio,
    ml_trimestral_prio,
    roe_trimestral_prio,
    roic_trimestral_prio,
    proventos_trimestral_prio,
    payout_trimestral_prio,
    buyback_trimestral_prio,
    fco_trimestral_prio,
    fci_trimestral_prio,
    fcf_trimestral_prio,
    fco_trimestral_acum_prio.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_prio.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_prio.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_prio,
    capex_trimestral_acum_prio.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_prio,
    net_capex_trimestral_prio,
    rd_trimestral_prio,
    adjusted_net_capex_trimestral_prio,
    working_capital_trimestral_prio,
    change_in_non_cash_wc_trimestral_prio,
    change_in_non_cash_wc_trimestral_acum_prio.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_prio,
    effective_tax_rate_trimestral_prio,
    fcfe_trimestral_prio,
    fcff_trimestral_prio
]

# Juntando os dfs
df_indicadores_trimestral_prio = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_prio
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_prio = df_indicadores_trimestral_prio.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_prio

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_prio.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//petroleo//prio//prio_indicadores_trimestral.csv'),
    separator=';'
)

# **Setor de Varejo**

## VULC3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_vulc'.

In [ ]:
# Número de ações (2017 a 2019) - VULC
dict_composicao_capital_17_18_19_vulc = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],

    'num_on':[
        245_756_000,
        245_756_000,
        245_756_000,
    ]
}

# Transformando em um df
df_composicao_capital_17_18_19_vulc = f_br.dict_to_polars_df(dict_composicao_capital_17_18_19_vulc)

# Composição de capital (2020 a 2024) - VULC
composicao_capital_vulc = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('VULCABRAS')).select('dt_refer', 'num_on')

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Arrumando a escala dos anos específicos
composicao_capital_vulc = composicao_capital_vulc.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on')

]).select('dt_refer', 'num_on')


# Juntando os dfs
df_composicao_capital_vulc = pl.concat([df_composicao_capital_17_18_19_vulc, composicao_capital_vulc])

##############################################################################
# Preços de fechamento - VULC
df_vulc_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='VULC3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2017,
    ultimo_ano=2024
)

print(df_composicao_capital_vulc.tail())
print('=' * 80)
print(df_vulc_fechamento_anual.tail())

In [ ]:
# VULC3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='VULC3.SA', ticker_pn=None)

# Valor de mercado
vm_vulc = f_br.indicador_vm(df_capital_social=df_composicao_capital_vulc, df_fechamento=df_vulc_fechamento_anual, novo_mercado=True)

# LPA
lpa_vulc = f_br.indicador_lpa(df_capital_social=df_composicao_capital_vulc, df_dre=dre, cod_cvm=11762, cod_ll='3.11.01', novo_mercado=True)

# P/L
pl_vulc = f_br.indicador_pl_lp(df_fechamento=df_vulc_fechamento_anual, tipo_acao='on', df_lpa=lpa_vulc)[0]

# L/P
lp_vulc = f_br.indicador_pl_lp(df_fechamento=df_vulc_fechamento_anual, tipo_acao='on', df_lpa=lpa_vulc)[1]

# EBITDA
ebitda_vulc = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=11762, cod_ebit='3.05', cod_depreciacao='6.01.01.02')

# Dívida bruta
divida_bruta_vulc = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=11762, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')

# Caixa e equivalentes
caixa_vulc = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=11762, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')

# Dívida líquida
divida_liquida_vulc = f_br.indicador_divida_liquida(df_caixa_total=caixa_vulc, df_divida_bruta=divida_bruta_vulc)

# DL/EBITDA
dl_ebitda_vulc = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_vulc, df_ebitda=ebitda_vulc)

# DL/PL
dl_pl_vulc = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=11762, cod_pl='2.03', df_divida_liquida=divida_liquida_vulc)

# EV/EBITDA
ev_ebitda_vulc = f_br.indicador_ev_ebitda(df_vm=vm_vulc, df_divida_liquida=divida_liquida_vulc, df_ebitda=ebitda_vulc)

# VPA
vpa_vulc = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_vulc, cod_cvm=11762, cod_pl='2.03', novo_mercado=True)

# P/VP
p_vp_vulc = f_br.indicador_p_vpa(df_fechamento=df_vulc_fechamento_anual, tipo_acao='on', df_vpa=vpa_vulc)

# Margem líquida
ml_vulc = f_br.indicador_ml(df_dre=dre, cod_cvm=11762, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_vulc = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=11762, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_vulc = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=11762, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_vulc)

# Dividendos
proventos_vulc = f_br.indicador_proventos(df_dva=dva, cod_cvm=11762, cod_dividendos='7.08.04.02', cod_jcp=None)

# Payout
payout_vulc = f_br.indicador_payout(df_dre=dre, cod_cvm=11762, df_proventos=proventos_vulc, cod_ll='3.11')

# Buyback
buyback_vulc = f_br.indicador_buyback(df_dfc=dfc, cod_cvm=11762, cod_buyback='6.03.12')

dict_buyback_vulc = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
    ],

    'buyback': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
    ]
}

# Dados complementares
buyback_complemento_vulc = f_br.dict_to_polars_df(dict_buyback_vulc)
buyback_vulc = pl.concat([buyback_complemento_vulc, buyback_vulc])

# FCO
fco_vulc = f_br.indicador_fco(df_dfc=dfc, cod_cvm=11762, cod_fco='6.01')

# FCI
fci_vulc = f_br.indicador_fci(df_dfc=dfc, cod_cvm=11762, cod_fci='6.02')

# FCF
fcf_vulc = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=11762, cod_fcf='6.03')

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisições de imobilizado'
    '6.02.02', # 'Pagamento por aquisição de controlada'
    '6.02.05', # 'Aquisições de intangível'
]

capex_vulc = f_br.indicador_capex(df_dfc=dfc, cod_cvm=11762, lst_cod_capex=lst_cod_capex)

# Free cash flow
free_cash_flow_vulc = f_br.indicador_free_cash_flow(df_fco=fco_vulc, df_capex=capex_vulc)

# Net capex
net_capex_vulc = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=11762, cod_depreciacao='6.01.01.02', df_capex=capex_vulc)

# R&D
dict_rd_vulc = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
}

rd_vulc = f_br.dict_to_polars_df(dict_rd_vulc)

# Adjusted net capex
adjusted_net_capex_vulc = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_vulc, df_rd=rd_vulc)

# Working capital
working_capital_vulc = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=11762,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_vulc = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=11762, cod_change_in_non_cash_wc='6.01.02')

# Reinvestment rate
reinvestment_rate_vulc = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_vulc,
    df_change_in_non_cash_wc=change_in_non_cash_wc_vulc
)

# Effective tax rate
effective_tax_rate_vulc = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=11762, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_vulc = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=11762, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_vulc, anual=True)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.02' # 'Empréstimos tomados - Principal'
]

new_borrowing_vulc = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=11762, lst_cod_new_borrowing=lst_cod_new_borrowing)

* Itens que compõem o 'debt paid':
    - 'debt_paid_item_1' - 'Pagamento de empréstimos tomados - Principal' - '6.03.04';
    - 'debt_paid_item_2' - 'Pagamento de passivos de arrendamentos financeiros' - '6.03.08'.

* Se o item **aparece** no DFC, eu tenho que adicionar manualmente a data do arquivo na lista 'DATAS_ALVO_X';
* Se o item **não** apareça no DFC, eu tenho que adicionar manualmente a data e 0 no dicionário 'dict_debt_paid_item_X', que são os dados complementares que estão faltando para todas as séries ficarem no mesmo tamanho.

In [ ]:
# Debt paid - não utilizei a função 'indicador_debt_paid', porque os dados de alguns itens estavam errados

# DFC da VULC
dfc_vulc = dfc.filter(pl.col('cd_cvm') == 11762)

################################################################
# 'Pagamento de empréstimos tomados - Principal'
DATAS_ALVO_1 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2018, 12, 31),
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_1 = (
    dfc_vulc
    .filter(
        (pl.col('cd_conta') == '6.03.04') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_1))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_1')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_1 = {
    'dt_refer': [
        '2017-12-31',
    ],

    'vl_conta_1': [
        -509_603.0,
    ]
}

debt_paid_item_1_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_1)
debt_paid_item_1 = pl.concat([debt_paid_item_1_complementos, debt_paid_item_1])
debt_paid_item_1 = debt_paid_item_1.sort('dt_refer')

################################################################
# 'Pagamento de passivos de arrendamentos financeiros'
DATAS_ALVO_2 = [
    # Todas as datas que esse item aparece no DFC
    datetime(2019, 12, 31),
    datetime(2020, 12, 31),
    datetime(2021, 12, 31),
    datetime(2022, 12, 31),
    datetime(2023, 12, 31),
    datetime(2024, 12, 31),
]

# Filtrando a série do item nas datas que ele aparece no DFC
debt_paid_item_2 = (
    dfc_vulc
    .filter(
        (pl.col('cd_conta') == '6.03.08') &
        (pl.col('dt_refer').is_in(DATAS_ALVO_2))
    )
    .select([
        'dt_refer',
        pl.col('vl_conta').alias('vl_conta_2')
    ])
    .sort('dt_refer')
)

# Dados complementares
dict_debt_paid_item_2 = {
    'dt_refer': [
        '2017-12-31',
        '2018-12-31',
    ],

    'vl_conta_2': [
        0.0,
        0.0,
    ]
}

debt_paid_item_2_complementos = f_br.dict_to_polars_df(dict_debt_paid_item_2)
debt_paid_item_2 = pl.concat([debt_paid_item_2_complementos, debt_paid_item_2])
debt_paid_item_2 = debt_paid_item_2.sort('dt_refer')

################################################################
# Lista dos dfs para concatenar
dfs_debt_paid = [debt_paid_item_1, debt_paid_item_2]
debt_paid_vulc = reduce(lambda left, right: left.join(right, on='dt_refer', how='inner'), dfs_debt_paid)

# Somando as colunas
debt_paid_vulc = debt_paid_vulc.with_columns(
    ((
        pl.col('vl_conta_1') +
        pl.col('vl_conta_2')
    ) * -1
    ).alias('debt_paid')
).select('dt_refer', 'debt_paid')

################################################################
# FCFE
fcfe_vulc = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=11762,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_vulc,
    df_change_in_non_cash_wc=change_in_non_cash_wc_vulc,
    df_new_borrowing=new_borrowing_vulc,
    df_debt_paid=debt_paid_vulc,
)

# FCFF
fcff_vulc = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_vulc,
    df_capex=capex_vulc,
    df_change_in_non_cash_wc=change_in_non_cash_wc_vulc,
)

In [ ]:
# Lista dos df
lst_df_vulc = [
    vm_vulc,
    lpa_vulc,
    pl_vulc,
    lp_vulc,
    ebitda_vulc,
    divida_bruta_vulc,
    caixa_vulc,
    divida_liquida_vulc,
    dl_ebitda_vulc,
    dl_pl_vulc,
    ev_ebitda_vulc,
    vpa_vulc,
    p_vp_vulc,
    ml_vulc,
    roe_vulc,
    roic_vulc,
    proventos_vulc,
    payout_vulc,
    buyback_vulc,
    fco_vulc,
    fci_vulc,
    fcf_vulc,
    capex_vulc,
    free_cash_flow_vulc,
    net_capex_vulc,
    rd_vulc,
    adjusted_net_capex_vulc,
    working_capital_vulc,
    change_in_non_cash_wc_vulc,
    reinvestment_rate_vulc,
    effective_tax_rate_vulc,
    fcfe_vulc,
    fcff_vulc
]

# Juntando os dfs
df_indicadores_vulc = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_vulc
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_vulc = df_indicadores_vulc.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_vulc

In [ ]:
# Transformando em um arquivo csv
df_indicadores_vulc.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//varejo//vulc//vulc_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - P&D no dicionário 'dict_rd_trimestral_vulc'.

In [ ]:
# Composição de capital - VULC
df_composicao_capital_trimestral_vulc = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('VULCABRAS')).select('dt_refer', 'num_on')

DATAS_NUM_ACOES = [
    # Todas as datas que o nº de ações estão com escalas erradas
    datetime(2020, 3, 31),
    datetime(2020, 6, 30),
    datetime(2020, 9, 30),
    datetime(2021, 3, 31),
    datetime(2021, 6, 30),
    datetime(2021, 9, 30),
    datetime(2022, 3, 31),
    datetime(2022, 6, 30),
    datetime(2022, 9, 30),
    datetime(2023, 3, 31),
    datetime(2023, 6, 30),
    datetime(2023, 9, 30),
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 9, 30),
    datetime(2025, 3, 31),
    datetime(2025, 6, 30),
    datetime(2025, 9, 30),
]

# Arrumando a escala dos anos específicos
df_composicao_capital_trimestral_vulc = df_composicao_capital_trimestral_vulc.with_columns([
    pl.when(pl.col('dt_refer').is_in(DATAS_NUM_ACOES))
      .then(pl.col('num_on') * 1000)
      .otherwise(pl.col('num_on'))
      .alias('num_on')

]).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - VULC
df_vulc_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='VULC3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_vulc['dt_refer']
)

print(df_composicao_capital_trimestral_vulc.tail())
print('=' * 80)
print(df_vulc_fechamento_trimestral.tail())

In [ ]:
# VULC3

# Valor de mercado
vm_trimestral_vulc = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_vulc[3:],
    df_fechamento=df_vulc_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_vulc = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_vulc,
    df_dre=dre_trimestral,
    cod_cvm=11762,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_vulc = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_vulc,
    df_dre=dre_trimestral,
    cod_cvm=11762,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

# P/L
pl_trimestral_vulc = f_br.indicador_pl_lp(
    df_fechamento=df_vulc_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_vulc
)[0]

# L/P
lp_trimestral_vulc = f_br.indicador_pl_lp(
    df_fechamento=df_vulc_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_vulc
)[1]

# EBITDA
ebitda_trimestral_vulc = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_vulc = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

# Dívida bruta
divida_bruta_trimestral_vulc = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=11762,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

# Caixa e equivalentes
caixa_trimestral_vulc = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=11762,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

# Dívida líquida
divida_liquida_trimestral_vulc = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_vulc,
    df_divida_bruta=divida_bruta_trimestral_vulc
)

# DL/EBITDA
dl_ebitda_trimestral_vulc = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_vulc,
    df_ebitda=ebitda_trimestral_acum_vulc
)

# DL/PL
dl_pl_trimestral_vulc = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=11762,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_vulc
)

# EV/EBITDA
ev_ebitda_trimestral_vulc = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_vulc,
    df_divida_liquida=divida_liquida_trimestral_vulc,
    df_ebitda=ebitda_trimestral_acum_vulc
)

# VPA
vpa_trimestral_vulc = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_vulc[3:],
    cod_cvm=11762,
    cod_pl='2.03',
    novo_mercado=True
)

# P/VP
p_vp_trimestral_vulc = f_br.indicador_p_vpa(
    df_fechamento=df_vulc_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_vulc
)

# Margem líquida
ml_trimestral_vulc = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=11762,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_vulc = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=11762,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_vulc = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_vulc
)

# Proventos
proventos_trimestral_vulc = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=11762,
    cod_dividendos='7.08.04.02',
    cod_jcp=None
)

# Payout
payout_trimestral_vulc = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=11762,
    df_proventos=proventos_trimestral_vulc,
    cod_ll='3.11'
)

# Buyback
buyback_trimestral_vulc = f_br.indicador_buyback_trimestral(df_dfc=dfc_trimestral, cod_cvm=11762, cod_buyback='6.03.12')

dict_buyback_trimestral_vulc = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
    ],

    'buyback_nao_acum': [
        # 2021
        0.0, 0.0, 0.0,
        # 2022
        0.0, 0.0, 0.0,
    ]
}

# Dados complementares
buyback_complemento_trimestral_vulc = f_br.dict_to_polars_df(dict_buyback_trimestral_vulc)
buyback_trimestral_vulc = pl.concat([buyback_complemento_trimestral_vulc, buyback_trimestral_vulc])

# FCO
fco_trimestral_vulc = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_vulc = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_fco='6.01'
)[1][3:]

# FCI
fci_trimestral_vulc = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_vulc = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_fci='6.02'
)[1][3:]

# FCF
fcf_trimestral_vulc = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_vulc = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_fcf='6.03'
)[1][3:]

# CAPEX
lst_cod_capex = [
    '6.02.01', # 'Aquisições de imobilizado'
    '6.02.05', # 'Aquisições de intangível'
]

capex_trimestral_vulc = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_vulc = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    lst_cod_capex=lst_cod_capex
)[1][3:]

# Free cash flow
free_cash_flow_trimestral_vulc = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_vulc,
    df_capex=capex_trimestral_acum_vulc
)

# Net capex
net_capex_trimestral_vulc = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_vulc
)

# R&D
dict_rd_trimestral_vulc = {
    'dt_refer':[
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2021
        0.0, 0.0, 0.0,
        # 2022
        0.0, 0.0, 0.0,
        # 2023
        0.0, 0.0, 0.0,
        # 2024
        0.0, 0.0, 0.0,
        # 2025
        0.0, 0.0, 0.0,
    ]
}

rd_trimestral_vulc = f_br.dict_to_polars_df(dict_rd_trimestral_vulc)

# Adjusted net capex
adjusted_net_capex_trimestral_vulc = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_vulc,
    df_rd=rd_trimestral_vulc
)

# Working capital
working_capital_trimestral_vulc = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=11762,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

# Change in non-cash wc
change_in_non_cash_wc_trimestral_vulc = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_vulc = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

# Reinvestment rate
reinvestment_rate_trimestral_vulc = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_vulc,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_vulc
)

# Effective tax rate
effective_tax_rate_trimestral_vulc = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=11762,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_vulc = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=11762,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_vulc,
    anual=False
)

# New borrowing
lst_cod_new_borrowing = [
    '6.03.02' # 'Empréstimos tomados - Principal'
]

new_borrowing_trimestral_vulc = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

# Debt paid
lst_cod_debt_paid = [
    '6.03.04', # 'Pagamento de empréstimos tomados - Principal'
    '6.03.08', # 'Pagamento de passivos de arrendamentos financeiros'
]

debt_paid_trimestral_vulc = f_br.indicador_debt_paid_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    lst_cod_debt_paid=lst_cod_debt_paid
)[3:]

# FCFE
fcfe_trimestral_vulc = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_vulc,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_vulc,
    df_new_borrowing=new_borrowing_trimestral_vulc,
    df_debt_paid=debt_paid_trimestral_vulc,
)

# FCFF
fcff_trimestral_vulc = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=11762,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_vulc,
    df_capex=capex_trimestral_acum_vulc,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_vulc,
)

In [ ]:
# Lista dos df
lst_df_trimestral_vulc = [
    vm_trimestral_vulc,
    lpa_trimestral_vulc,
    lpa_trimestral_acum_vulc.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_vulc,
    lp_trimestral_vulc,
    ebitda_trimestral_vulc,
    ebitda_trimestral_acum_vulc.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_vulc,
    caixa_trimestral_vulc,
    divida_liquida_trimestral_vulc,
    dl_ebitda_trimestral_vulc,
    dl_pl_trimestral_vulc,
    ev_ebitda_trimestral_vulc,
    vpa_trimestral_vulc,
    p_vp_trimestral_vulc,
    ml_trimestral_vulc,
    roe_trimestral_vulc,
    roic_trimestral_vulc,
    proventos_trimestral_vulc,
    payout_trimestral_vulc,
    buyback_trimestral_vulc,
    fco_trimestral_vulc,
    fci_trimestral_vulc,
    fcf_trimestral_vulc,
    fco_trimestral_acum_vulc.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_vulc.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_vulc.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_vulc,
    capex_trimestral_acum_vulc.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_vulc,
    net_capex_trimestral_vulc,
    rd_trimestral_vulc,
    adjusted_net_capex_trimestral_vulc,
    working_capital_trimestral_vulc,
    change_in_non_cash_wc_trimestral_vulc,
    change_in_non_cash_wc_trimestral_acum_vulc.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_vulc,
    effective_tax_rate_trimestral_vulc,
    fcfe_trimestral_vulc,
    fcff_trimestral_vulc
]

# Juntando os dfs
df_indicadores_trimestral_vulc = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_vulc
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_vulc = df_indicadores_trimestral_vulc.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_vulc

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_vulc.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//varejo//vulc//vulc_indicadores_trimestral.csv'),
    separator=';'
)

## VIVA3

### Indicadores Fundamentalistas - Anual

* Eu tenho que adicionar manualmente os dados de:
    - Buyback no dicionário 'dict_buyback_viva';
    - P&D no dicionário 'dict_rd_viva'.

In [ ]:
# Número de ações (2019) - VIVA
dict_composicao_capital_19_viva = {
    'dt_refer':[
        '2019-12-31',
    ],

    'num_on':[
        236_197_769,
    ]
}

# Transformando em um df
df_composicao_capital_19_viva = f_br.dict_to_polars_df(dict_composicao_capital_19_viva)

# Composição de capital (2020 a 2024) - VIVA
composicao_capital_viva = composicao_capital.filter(composicao_capital['denom_cia'].str.contains('VIVARA')).select('dt_refer', 'num_on')

# Juntando os dfs
df_composicao_capital_viva = pl.concat([df_composicao_capital_19_viva, composicao_capital_viva])

##############################################################################
# Preços de fechamento - VIVA
df_viva_fechamento_anual = f_br.ativo_fechamento_anual(
    ticker_on_unit='VIVA3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2019,
    ultimo_ano=2024
)

print(df_composicao_capital_viva.tail())
print('=' * 80)
print(df_viva_fechamento_anual.tail())

In [ ]:
# VIVA3

# Valor de mercado atual
vm_atual = f_br.indicador_vm_atual(df_capital_social=df_capital_social, ticker_on='VIVA3.SA', ticker_pn=None)

# Valor de mercado
vm_viva = f_br.indicador_vm(df_capital_social=df_composicao_capital_viva, df_fechamento=df_viva_fechamento_anual, novo_mercado=True)

# LPA
lpa_viva = f_br.indicador_lpa(df_capital_social=df_composicao_capital_viva, df_dre=dre, cod_cvm=24805, cod_ll='3.11.01', novo_mercado=True)
lpa_viva = lpa_viva.with_columns((pl.col('lpa') / 1000).alias('lpa'))

# P/L
pl_viva = f_br.indicador_pl_lp(df_fechamento=df_viva_fechamento_anual, tipo_acao='on', df_lpa=lpa_viva)[0]

# L/P
lp_viva = f_br.indicador_pl_lp(df_fechamento=df_viva_fechamento_anual, tipo_acao='on', df_lpa=lpa_viva)[1]

# EBITDA
ebitda_viva = f_br.indicador_ebitda(df_dre=dre, df_dfc=dfc, cod_cvm=24805, cod_ebit='3.05', cod_depreciacao='6.01.01.02')
ebitda_viva = ebitda_viva.with_columns((pl.col('ebitda') / 1000).alias('ebitda'))

# Dívida bruta
divida_bruta_viva = f_br.indicador_divida_bruta(df_bpp=bpp, cod_cvm=24805, cod_emprest_circ='2.01.04', cod_emprest_n_circ='2.02.01')
divida_bruta_viva = divida_bruta_viva.with_columns((pl.col('divida_bruta') / 1000).alias('divida_bruta'))
divida_bruta_viva_2 = divida_bruta_viva.with_columns((pl.col('divida_bruta') * 1000).alias('divida_bruta')) # Multipliquei por 1000 p/ calcular o 'indicador_roic'

# Caixa e equivalentes
caixa_viva = f_br.indicador_caixa(df_bpa=bpa, cod_cvm=24805, cod_caixa='1.01.01', cod_aplic_financ='1.01.02')
caixa_viva = caixa_viva.with_columns((pl.col('caixa_total') / 1000).alias('caixa_total'))

# Dívida líquida
divida_liquida_viva = f_br.indicador_divida_liquida(df_caixa_total=caixa_viva, df_divida_bruta=divida_bruta_viva)
divida_liquida_viva_2 = divida_liquida_viva.with_columns((pl.col('divida_liquida') * 1000).alias('divida_liquida'))  # Multipliquei por 1000 p/ calcular o 'indicador_dl_pl'

# DL/EBITDA
dl_ebitda_viva = f_br.indicador_dl_ebitda(df_divida_liquida=divida_liquida_viva, df_ebitda=ebitda_viva)

# DL/PL
dl_pl_viva = f_br.indicador_dl_pl(df_bpp=bpp, cod_cvm=24805, cod_pl='2.03', df_divida_liquida=divida_liquida_viva_2)

# EV/EBITDA
ev_ebitda_viva = f_br.indicador_ev_ebitda(df_vm=vm_viva, df_divida_liquida=divida_liquida_viva, df_ebitda=ebitda_viva)

# VPA
vpa_viva = f_br.indicador_vpa(df_bpp=bpp, df_capital_social=df_composicao_capital_viva, cod_cvm=24805, cod_pl='2.03', novo_mercado=True)
vpa_viva = vpa_viva.with_columns((pl.col('vpa') / 1000).alias('vpa'))

# P/VP
p_vp_viva = f_br.indicador_p_vpa(df_fechamento=df_viva_fechamento_anual, tipo_acao='on', df_vpa=vpa_viva)

# Margem líquida
ml_viva = f_br.indicador_ml(df_dre=dre, cod_cvm=24805, cod_ll='3.11', cod_rl='3.01')

# ROE
roe_viva = f_br.indicador_roe(df_dre=dre, df_bpp=bpp, cod_cvm=24805, cod_ll='3.11', cod_pl='2.03')

# ROIC
roic_viva = f_br.indicador_roic(df_dre=dre, df_bpp=bpp, cod_cvm=24805, cod_ebit='3.05', cod_ir='3.08', cod_pl='2.03', df_divida_bruta=divida_bruta_viva_2)

# Dividendos
proventos_viva = f_br.indicador_proventos(df_dva=dva, cod_cvm=24805, cod_dividendos='7.08.04.02', cod_jcp='7.08.04.01')
proventos_viva = proventos_viva.with_columns((pl.col('proventos') / 1000).alias('proventos'))
proventos_viva_2 = proventos_viva.with_columns((pl.col('proventos') * 1000).alias('proventos'))  # Multipliquei por 1000 p/ calcular o 'indicador_payout'

# Payout
payout_viva = f_br.indicador_payout(df_dre=dre, cod_cvm=24805, df_proventos=proventos_viva_2, cod_ll='3.11')

# Buyback
dict_buyback_viva = {
    'dt_refer': [
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'buyback': [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        6972955/1000,
    ]
}

buyback_viva  = f_br.dict_to_polars_df(dict_buyback_viva)

# FCO
fco_viva = f_br.indicador_fco(df_dfc=dfc, cod_cvm=24805, cod_fco='6.01')
fco_viva = fco_viva.with_columns((pl.col('fco') / 1000).alias('fco'))

# FCI
fci_viva = f_br.indicador_fci(df_dfc=dfc, cod_cvm=24805, cod_fci='6.02')
fci_viva = fci_viva.with_columns((pl.col('fci') / 1000).alias('fci'))

# FCF
fcf_viva = f_br.indicador_fcf(df_dfc=dfc, cod_cvm=24805, cod_fcf='6.03')
fcf_viva = fcf_viva.with_columns((pl.col('fcf') / 1000).alias('fcf'))

# CAPEX
lst_cod_capex = [
    '6.02.02', # 'Aquisição de imobilizado'
    '6.02.03', # 'Aquisição de intangível'
]

capex_viva = f_br.indicador_capex(df_dfc=dfc, cod_cvm=24805, lst_cod_capex=lst_cod_capex)
capex_viva = capex_viva.with_columns((pl.col('capex') / 1000).alias('capex'))
capex_viva_2 = capex_viva.with_columns((pl.col('capex') * 1000).alias('capex')) # Multipliquei por 1000 p/ calcular o 'indicador_net_capex', 'indicador_fcfe' e 'indicador_fcff'

# Free cash flow
free_cash_flow_viva = f_br.indicador_free_cash_flow(df_fco=fco_viva, df_capex=capex_viva)

# Net capex
net_capex_viva = f_br.indicador_net_capex(df_dfc=dfc, cod_cvm=24805, cod_depreciacao='6.01.01.02', df_capex=capex_viva_2)
net_capex_viva = net_capex_viva.with_columns((pl.col('net_capex') / 1000).alias('net_capex'))

# R&D
dict_rd_viva = {
    'dt_refer':[
        '2019-12-31',
        '2020-12-31',
        '2021-12-31',
        '2022-12-31',
        '2023-12-31',
        '2024-12-31',
    ],

    'rd':[
        0,
        0,
        0,
        0,
        0,
        0,
    ]
}

rd_viva = f_br.dict_to_polars_df(dict_rd_viva)

# Adjusted net capex
adjusted_net_capex_viva = f_br.indicador_adjusted_net_capex(df_net_capex=net_capex_viva, df_rd=rd_viva)
adjusted_net_capex_viva_2 = adjusted_net_capex_viva.with_columns((pl.col('adjusted_net_capex') * 1000).alias('adjusted_net_capex')) # Multipliquei por 1000 p/ calcular o 'indicador_reinvestment_rate'

# Working capital
working_capital_viva = f_br.indicador_working_capital(
    df_bpa=bpa,
    df_bpp=bpp,
    cod_cvm=24805,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)
working_capital_viva = working_capital_viva.with_columns((pl.col('working_capital') / 1000).alias('working_capital'))

# Change in non-cash wc
change_in_non_cash_wc_viva = f_br.indicador_change_in_non_cash_wc(df_dfc=dfc, cod_cvm=24805, cod_change_in_non_cash_wc='6.01.02')
change_in_non_cash_wc_viva = change_in_non_cash_wc_viva.with_columns((pl.col('change_in_non_cash_wc') / 1000).alias('change_in_non_cash_wc'))
change_in_non_cash_wc_viva_2 = change_in_non_cash_wc_viva.with_columns((pl.col('change_in_non_cash_wc') * 1000).alias('change_in_non_cash_wc')) # Multipliquei por 1000 p/ calcular o 'indicador_reinvestment_rate', 'indicador_fcfe' e 'indicador_fcff'

# Reinvestment rate
reinvestment_rate_viva = f_br.indicador_reinvestment_rate(
    df_dre=dre,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_viva_2,
    df_change_in_non_cash_wc=change_in_non_cash_wc_viva_2
)

# Effective tax rate
effective_tax_rate_viva = f_br.indicador_effective_tax_rate(df_dre=dre, cod_cvm=24805, cod_ir='3.08', cod_ebt='3.07', anual=True)

# Taxes on operating income
taxes_on_operating_income_viva = f_br.indicador_taxes_on_operating_income(df_dre=dre, cod_cvm=24805, cod_ebit='3.05', df_effective_tax_rate=effective_tax_rate_viva, anual=True)
taxes_on_operating_income_viva = taxes_on_operating_income_viva.with_columns((pl.col('taxes_on_operating_income') / 1000).alias('taxes_on_operating_income'))
taxes_on_operating_income_viva_2 = taxes_on_operating_income_viva.with_columns((pl.col('taxes_on_operating_income') * 1000).alias('taxes_on_operating_income')) # Multipliquei por 1000 p/ calcular o 'indicador_fcff'

# New borrowing
lst_cod_new_borrowing = [
    '6.03.04' # 'Captação de empréstimos e financiamentos'
]

new_borrowing_viva = f_br.indicador_new_borrowing(df_dfc=dfc, cod_cvm=24805, lst_cod_new_borrowing=lst_cod_new_borrowing)
new_borrowing_viva = new_borrowing_viva.with_columns((pl.col('new_borrowing') / 1000).alias('new_borrowing'))
new_borrowing_viva_2 = new_borrowing_viva.with_columns((pl.col('new_borrowing') * 1000).alias('new_borrowing')) # Multipliquei por 1000 p/ calcular o 'indicador_fcfe'

# Debt paid
lst_cod_debt_paid = [
    '6.03.05', # 'Amortização de empréstimos e financiamentos'
    '6.03.07', # 'Amortização de arrendamentos direito de uso'
]

debt_paid_viva = f_br.indicador_debt_paid(df_dfc=dfc, cod_cvm=24805, lst_cod_debt_paid=lst_cod_debt_paid)
debt_paid_viva = debt_paid_viva.with_columns((pl.col('debt_paid') / 1000).alias('debt_paid'))
debt_paid_viva_2 = debt_paid_viva.with_columns((pl.col('debt_paid') * 1000).alias('debt_paid')) # Multipliquei por 1000 p/ calcular o 'indicador_fcfe'

# FCFE
fcfe_viva = f_br.indicador_fcfe(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=24805,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_viva_2,
    df_change_in_non_cash_wc=change_in_non_cash_wc_viva_2,
    df_new_borrowing=new_borrowing_viva_2,
    df_debt_paid=debt_paid_viva_2,
)
fcfe_viva = fcfe_viva.with_columns((pl.col('fcfe') / 1000).alias('fcfe'))

# FCFF
fcff_viva = f_br.indicador_fcff(
    df_dre=dre,
    df_dfc=dfc,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_viva_2,
    df_capex=capex_viva_2,
    df_change_in_non_cash_wc=change_in_non_cash_wc_viva_2,
)
fcff_viva = fcff_viva.with_columns((pl.col('fcff') / 1000).alias('fcff'))

In [ ]:
# Lista dos df
lst_df_viva = [
    vm_viva,
    lpa_viva,
    pl_viva,
    lp_viva,
    ebitda_viva,
    divida_bruta_viva,
    caixa_viva,
    divida_liquida_viva,
    dl_ebitda_viva,
    dl_pl_viva,
    ev_ebitda_viva,
    vpa_viva,
    p_vp_viva,
    ml_viva,
    roe_viva,
    roic_viva,
    proventos_viva,
    payout_viva,
    buyback_viva,
    fco_viva,
    fci_viva,
    fcf_viva,
    capex_viva,
    free_cash_flow_viva,
    net_capex_viva,
    rd_viva,
    adjusted_net_capex_viva,
    working_capital_viva,
    change_in_non_cash_wc_viva,
    reinvestment_rate_viva,
    effective_tax_rate_viva,
    fcfe_viva,
    fcff_viva
]

# Juntando os dfs
df_indicadores_viva = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_viva
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_viva = df_indicadores_viva.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_viva

In [ ]:
# Transformando em um arquivo csv
df_indicadores_viva.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//varejo//viva//viva_indicadores_anual.csv'),
    separator=';'
)

### Indicadores Fundamentalistas - Trimestral

* Eu tenho que adicionar manualmente os dados de:
    - Buyback no dicionário 'dict_buyback_trimestral_viva';
    - P&D no dicionário 'dict_rd_trimestral_viva'.

In [ ]:
# Composição de capital - VIVA
df_composicao_capital_trimestral_viva = composicao_capital_trimestral.filter(composicao_capital_trimestral['denom_cia'].str.contains('VIVARA')).select('dt_refer', 'num_on')

##############################################################################
# Preços de fechamento - VIVA
df_viva_fechamento_trimestral = f_br.ativo_fechamento_trimestral(
    ticker_on_unit='VIVA3.SA',
    list_ticker_on_pn=None,
    primeiro_ano=2020,
    ultimo_ano=2026,
    pl_series_dt_refer=df_composicao_capital_trimestral_viva['dt_refer']
)

print(df_composicao_capital_trimestral_viva.tail())
print('=' * 80)
print(df_viva_fechamento_trimestral.tail())

In [ ]:
# VIVA3

# Valor de mercado
vm_trimestral_viva = f_br.indicador_vm(
    df_capital_social=df_composicao_capital_trimestral_viva[3:],
    df_fechamento=df_viva_fechamento_trimestral[3:],
    novo_mercado=True
)

# LPA
lpa_trimestral_viva = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_viva,
    df_dre=dre_trimestral,
    cod_cvm=24805,
    cod_ll='3.11.01',
    novo_mercado=True
)[0]

lpa_trimestral_acum_viva = f_br.indicador_lpa_trimestral(
    df_capital_social=df_composicao_capital_trimestral_viva,
    df_dre=dre_trimestral,
    cod_cvm=24805,
    cod_ll='3.11.01',
    novo_mercado=True
)[1]

lpa_trimestral_acum_viva = lpa_trimestral_acum_viva.with_columns((pl.col('lpa') / 1000).alias('lpa'))

# P/L
pl_trimestral_viva = f_br.indicador_pl_lp(
    df_fechamento=df_viva_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_viva
)[0]

# L/P
lp_trimestral_viva = f_br.indicador_pl_lp(
    df_fechamento=df_viva_fechamento_trimestral[3:],
    tipo_acao='on',
    df_lpa=lpa_trimestral_acum_viva
)[1]

# EBITDA
ebitda_trimestral_viva = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[0]

ebitda_trimestral_acum_viva = f_br.indicador_ebitda_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02'
)[1][3:]

ebitda_trimestral_acum_viva = ebitda_trimestral_acum_viva.with_columns((pl.col('ebitda') / 1000).alias('ebitda'))

# Dívida bruta
divida_bruta_trimestral_viva = f_br.indicador_divida_bruta(
    df_bpp=bpp_trimestral,
    cod_cvm=24805,
    cod_emprest_circ='2.01.04',
    cod_emprest_n_circ='2.02.01'
)[3:]

divida_bruta_trimestral_viva = divida_bruta_trimestral_viva.with_columns((pl.col('divida_bruta') / 1000).alias('divida_bruta'))
divida_bruta_trimestral_viva_2 = divida_bruta_trimestral_viva.with_columns((pl.col('divida_bruta') * 1000).alias('divida_bruta')) # Multipliquei por 1000 p/ calcular o 'indicador_roic_trimestral'

# Caixa e equivalentes
caixa_trimestral_viva = f_br.indicador_caixa(
    df_bpa=bpa_trimestral,
    cod_cvm=24805,
    cod_caixa='1.01.01',
    cod_aplic_financ='1.01.02'
)[3:]

caixa_trimestral_viva = caixa_trimestral_viva.with_columns((pl.col('caixa_total') / 1000).alias('caixa_total'))

# Dívida líquida
divida_liquida_trimestral_viva = f_br.indicador_divida_liquida(
    df_caixa_total=caixa_trimestral_viva,
    df_divida_bruta=divida_bruta_trimestral_viva
)

divida_liquida_trimestral_viva_2 = divida_liquida_trimestral_viva.with_columns((pl.col('divida_liquida') * 1000).alias('divida_liquida'))  # Multipliquei por 1000 p/ calcular o 'indicador_dl_pl'

# DL/EBITDA
dl_ebitda_trimestral_viva = f_br.indicador_dl_ebitda(
    df_divida_liquida=divida_liquida_trimestral_viva,
    df_ebitda=ebitda_trimestral_acum_viva
)

# DL/PL
dl_pl_trimestral_viva = f_br.indicador_dl_pl(
    df_bpp=bpp_trimestral,
    cod_cvm=24805,
    cod_pl='2.03',
    df_divida_liquida=divida_liquida_trimestral_viva_2
)

# EV/EBITDA
ev_ebitda_trimestral_viva = f_br.indicador_ev_ebitda(
    df_vm=vm_trimestral_viva,
    df_divida_liquida=divida_liquida_trimestral_viva,
    df_ebitda=ebitda_trimestral_acum_viva
)

# VPA
vpa_trimestral_viva = f_br.indicador_vpa(
    df_bpp=bpp_trimestral,
    df_capital_social=df_composicao_capital_trimestral_viva[3:],
    cod_cvm=24805,
    cod_pl='2.03',
    novo_mercado=True
)

vpa_trimestral_viva = vpa_trimestral_viva.with_columns((pl.col('vpa') / 1000).alias('vpa'))

# P/VP
p_vp_trimestral_viva = f_br.indicador_p_vpa(
    df_fechamento=df_viva_fechamento_trimestral[3:],
    tipo_acao='on',
    df_vpa=vpa_trimestral_viva
)

# Margem líquida
ml_trimestral_viva = f_br.indicador_ml_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=24805,
    cod_ll='3.11',
    cod_rl='3.01'
)

# ROE
roe_trimestral_viva = f_br.indicador_roe_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=24805,
    cod_ll='3.11',
    cod_pl='2.03'
)

# ROIC
roic_trimestral_viva = f_br.indicador_roic_trimestral(
    df_dre=dre_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_ir='3.08',
    cod_pl='2.03',
    df_divida_bruta=divida_bruta_trimestral_viva_2
)

# Proventos
proventos_trimestral_viva = f_br.indicador_proventos_trimestral(
    df_dva=dva_trimestral,
    cod_cvm=24805,
    cod_dividendos='7.08.04.02',
    cod_jcp='7.08.04.01'
)

proventos_trimestral_viva = proventos_trimestral_viva.with_columns((pl.col('proventos') / 1000).alias('proventos'))
proventos_trimestral_viva_2 = proventos_trimestral_viva.with_columns((pl.col('proventos') * 1000).alias('proventos'))  # Multipliquei por 1000 p/ calcular o 'indicador_payout_trimestral'

# Payout
payout_trimestral_viva = f_br.indicador_payout_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=24805,
    df_proventos=proventos_trimestral_viva_2,
    cod_ll='3.11'
)

# Buyback
dict_buyback_trimestral_viva = {
    'dt_refer': [
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2023-09-30',
        '2025-03-31', '2025-06-30', '2023-09-30',
    ],

    'buyback_nao_acum': [
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

buyback_trimestral_viva = f_br.dict_to_polars_df(dict_buyback_trimestral_viva)

# FCO
fco_trimestral_viva = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_fco='6.01'
)[0][3:]

fco_trimestral_acum_viva = f_br.indicador_fco_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_fco='6.01'
)[1][3:]

fco_trimestral_acum_viva = fco_trimestral_acum_viva.with_columns((pl.col('fco') / 1000).alias('fco'))

# FCI
fci_trimestral_viva = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_fci='6.02'
)[0][3:]

fci_trimestral_acum_viva = f_br.indicador_fci_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_fci='6.02'
)[1][3:]

fci_trimestral_acum_viva = fci_trimestral_acum_viva.with_columns((pl.col('fci') / 1000).alias('fci'))

# FCF
fcf_trimestral_viva = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_fcf='6.03'
)[0][3:]

fcf_trimestral_acum_viva = f_br.indicador_fcf_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_fcf='6.03'
)[1][3:]

fcf_trimestral_acum_viva = fcf_trimestral_acum_viva.with_columns((pl.col('fcf') / 1000).alias('fcf'))

# CAPEX
lst_cod_capex = [
    '6.02.02', # 'Aquisição de imobilizado'
    '6.02.03', # 'Aquisição de intangível'
]

capex_trimestral_viva = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    lst_cod_capex=lst_cod_capex
)[0][3:]

capex_trimestral_acum_viva = f_br.indicador_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    lst_cod_capex=lst_cod_capex
)[1][3:]

capex_trimestral_acum_viva = capex_trimestral_acum_viva.with_columns((pl.col('capex') / 1000).alias('capex'))
capex_trimestral_acum_viva_2 = capex_trimestral_acum_viva.with_columns((pl.col('capex') * 1000).alias('capex')) # Multipliquei por 1000 p/ calcular o 'indicador_net_capex_trimestral'

# Free cash flow
free_cash_flow_trimestral_viva = f_br.indicador_free_cash_flow(
    df_fco=fco_trimestral_acum_viva,
    df_capex=capex_trimestral_acum_viva
)

# Net capex
net_capex_trimestral_viva = f_br.indicador_net_capex_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_viva_2
)

net_capex_trimestral_viva = net_capex_trimestral_viva.with_columns((pl.col('net_capex') / 1000).alias('net_capex'))

# R&D
dict_rd_trimestral_viva = {
    'dt_refer':[
        '2021-03-31', '2021-06-30', '2021-09-30',
        '2022-03-31', '2022-06-30', '2022-09-30',
        '2023-03-31', '2023-06-30', '2023-09-30',
        '2024-03-31', '2024-06-30', '2024-09-30',
        '2025-03-31', '2025-06-30', '2025-09-30',
    ],

    'rd':[
        # 2021
        0, 0, 0,
        # 2022
        0, 0, 0,
        # 2023
        0, 0, 0,
        # 2024
        0, 0, 0,
        # 2025
        0, 0, 0,
    ]
}

rd_trimestral_viva = f_br.dict_to_polars_df(dict_rd_trimestral_viva)

# Adjusted net capex
adjusted_net_capex_trimestral_viva = f_br.indicador_adjusted_net_capex(
    df_net_capex=net_capex_trimestral_viva,
    df_rd=rd_trimestral_viva
)

adjusted_net_capex_trimestral_viva_2 = adjusted_net_capex_trimestral_viva.with_columns((pl.col('adjusted_net_capex') * 1000).alias('adjusted_net_capex')) # Multipliquei por 1000 p/ calcular o 'indicador_reinvestment_rate_trimestral'

# Working capital
working_capital_trimestral_viva = f_br.indicador_working_capital(
    df_bpa=bpa_trimestral,
    df_bpp=bpp_trimestral,
    cod_cvm=24805,
    cod_ativo_circ='1.01',
    cod_passivo_circ='2.01'
)

working_capital_trimestral_viva = working_capital_trimestral_viva.with_columns((pl.col('working_capital') / 1000).alias('working_capital'))

# Change in non-cash wc
change_in_non_cash_wc_trimestral_viva = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_change_in_non_cash_wc='6.01.02'
)[0][3:]

change_in_non_cash_wc_trimestral_acum_viva = f_br.indicador_change_in_non_cash_wc_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_change_in_non_cash_wc='6.01.02'
)[1][3:]

change_in_non_cash_wc_trimestral_acum_viva = change_in_non_cash_wc_trimestral_acum_viva.with_columns((pl.col('change_in_non_cash_wc') / 1000).alias('change_in_non_cash_wc'))
change_in_non_cash_wc_trimestral_acum_viva_2 = change_in_non_cash_wc_trimestral_acum_viva.with_columns((pl.col('change_in_non_cash_wc') * 1000).alias('change_in_non_cash_wc')) # Multipliquei por 1000 p/ calcular o 'indicador_reinvestment_rate_trimestral'

# Reinvestment rate
reinvestment_rate_trimestral_viva = f_br.indicador_reinvestment_rate_trimestral(
    df_dre=dre_trimestral,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_ir='3.08',
    df_net_capex=adjusted_net_capex_trimestral_viva_2,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_viva_2
)

# Effective tax rate
effective_tax_rate_trimestral_viva = f_br.indicador_effective_tax_rate(
    df_dre=dre_trimestral,
    cod_cvm=24805,
    cod_ir='3.08',
    cod_ebt='3.07',
    anual=False
)[3:]

# Taxes on operating income
taxes_on_operating_income_trimestral_viva = f_br.indicador_taxes_on_operating_income(
    df_dre=dre_trimestral,
    cod_cvm=24805,
    cod_ebit='3.05',
    df_effective_tax_rate=effective_tax_rate_trimestral_viva,
    anual=False
)

taxes_on_operating_income_trimestral_viva = taxes_on_operating_income_trimestral_viva.with_columns((pl.col('taxes_on_operating_income') / 1000).alias('taxes_on_operating_income'))
taxes_on_operating_income_trimestral_viva_2 = taxes_on_operating_income_trimestral_viva.with_columns((pl.col('taxes_on_operating_income') * 1000).alias('taxes_on_operating_income')) # Multipliquei por 1000 p/ calcular o 'indicador_fcff_trimestral'

# New borrowing
lst_cod_new_borrowing = [
    '6.03.04' # 'Captação de empréstimos e financiamentos'
]

new_borrowing_trimestral_viva = f_br.indicador_new_borrowing_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    lst_cod_new_borrowing=lst_cod_new_borrowing
)[3:]

new_borrowing_trimestral_viva = new_borrowing_trimestral_viva.with_columns((pl.col('new_borrowing') / 1000).alias('new_borrowing'))
new_borrowing_trimestral_viva_2 = new_borrowing_trimestral_viva.with_columns((pl.col('new_borrowing') * 1000).alias('new_borrowing')) # Multipliquei por 1000 p/ calcular o 'indicador_fcfe_trimestral'

# Debt paid
lst_cod_debt_paid = [
    '6.03.05', # 'Amortização de empréstimos e financiamentos'
    '6.03.07', # 'Amortização de arrendamentos direito de uso'
]

debt_paid_trimestral_viva = f_br.indicador_debt_paid_trimestral(
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    lst_cod_debt_paid=lst_cod_debt_paid
)[3:]

debt_paid_trimestral_viva = debt_paid_trimestral_viva.with_columns((pl.col('debt_paid') / 1000).alias('debt_paid'))
debt_paid_trimestral_viva_2 = debt_paid_trimestral_viva.with_columns((pl.col('debt_paid') * 1000).alias('debt_paid')) # Multipliquei por 1000 p/ calcular o 'indicador_fcfe_trimestral'

# FCFE
fcfe_trimestral_viva = f_br.indicador_fcfe_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_ll='3.11',
    cod_depreciacao='6.01.01.02',
    df_capex=capex_trimestral_acum_viva_2,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_viva_2,
    df_new_borrowing=new_borrowing_trimestral_viva_2,
    df_debt_paid=debt_paid_trimestral_viva_2,
)

fcfe_trimestral_viva = fcfe_trimestral_viva.with_columns((pl.col('fcfe') / 1000).alias('fcfe'))

# FCFF
fcff_trimestral_viva = f_br.indicador_fcff_trimestral(
    df_dre=dre_trimestral,
    df_dfc=dfc_trimestral,
    cod_cvm=24805,
    cod_ebit='3.05',
    cod_depreciacao='6.01.01.02',
    df_taxes_on_operating_income=taxes_on_operating_income_trimestral_viva_2,
    df_capex=capex_trimestral_acum_viva_2,
    df_change_in_non_cash_wc=change_in_non_cash_wc_trimestral_acum_viva_2,
)

fcff_trimestral_viva = fcff_trimestral_viva.with_columns((pl.col('fcff') / 1000).alias('fcff'))

In [ ]:
# Lista dos df
lst_df_trimestral_viva = [
    vm_trimestral_viva,
    lpa_trimestral_viva,
    lpa_trimestral_acum_viva.rename({'lpa': 'lpa_acum'}),
    pl_trimestral_viva,
    lp_trimestral_viva,
    ebitda_trimestral_viva,
    ebitda_trimestral_acum_viva.rename({'ebitda': 'ebitda_acum'}),
    divida_bruta_trimestral_viva,
    caixa_trimestral_viva,
    divida_liquida_trimestral_viva,
    dl_ebitda_trimestral_viva,
    dl_pl_trimestral_viva,
    ev_ebitda_trimestral_viva,
    vpa_trimestral_viva,
    p_vp_trimestral_viva,
    ml_trimestral_viva,
    roe_trimestral_viva,
    roic_trimestral_viva,
    proventos_trimestral_viva,
    payout_trimestral_viva,
    buyback_trimestral_viva,
    fco_trimestral_viva,
    fci_trimestral_viva,
    fcf_trimestral_viva,
    fco_trimestral_acum_viva.rename({'fco': 'fco_acum'}),
    fci_trimestral_acum_viva.rename({'fci': 'fci_acum'}),
    fcf_trimestral_acum_viva.rename({'fcf': 'fcf_acum'}),
    capex_trimestral_viva,
    capex_trimestral_acum_viva.rename({'capex': 'capex_acum'}),
    free_cash_flow_trimestral_viva,
    net_capex_trimestral_viva,
    rd_trimestral_viva,
    adjusted_net_capex_trimestral_viva,
    working_capital_trimestral_viva,
    change_in_non_cash_wc_trimestral_viva,
    change_in_non_cash_wc_trimestral_acum_viva.rename({'change_in_non_cash_wc': 'change_in_non_cash_wc_acum'}),
    reinvestment_rate_trimestral_viva,
    effective_tax_rate_trimestral_viva,
    fcfe_trimestral_viva,
    fcff_trimestral_viva
]

# Juntando os dfs
df_indicadores_trimestral_viva = reduce(
    lambda left, right: left.join(right, on='dt_refer', how='inner'), lst_df_trimestral_viva
)

# Transformando a coluna 'dt_refer' no tipo 'Date' (não aparece o horário '00:00:00')
df_indicadores_trimestral_viva = df_indicadores_trimestral_viva.with_columns(
    pl.col('dt_refer').dt.date()
)

df_indicadores_trimestral_viva

In [ ]:
# Transformando em um arquivo csv
df_indicadores_trimestral_viva.write_csv(
    Path('C://B3//analise-fundamentalista//analise_brasil//setores//varejo//viva//viva_indicadores_trimestral.csv'),
    separator=';'
)

## AZZA3

In [ ]:
# Número de ações (2017 a 2019) - AZZA
dict_composicao_capital_17_18_19_azza = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        89_766_000,
        90_303_000,
        90_954_000,
    ]
}

## LREN3

In [ ]:
# Número de ações (2017 a 2019) - LREN
dict_composicao_capital_17_18_19_lren = {
    'dt_refer':[
        '2017-12-31',
        '2018-12-31',
        '2019-12-31',
    ],
    'num_on':[
        713_235_000,
        720_024_000,
        795_558_000,
    ]
}

## CEAB3

In [ ]:
# Número de ações (2019) - CEAB
dict_composicao_capital_19_ceab = {
    'dt_refer':[
        '2019-12-31'
    ],
    'num_on':[
        308_245_068
    ]
}

## SBFG3

In [ ]:
# Número de ações (2019) - SBFG
dict_composicao_capital_17_18_19_sbfg = {
    'dt_refer':[
        '2019-12-31'
    ],
    'num_on':[
        210_140_608
    ]
}